# Imports

In [1]:
import numpy as np
import pandas as pd
from itertools import product
from numba import njit
import time
%matplotlib inline

# Preparing Data

In [2]:
pauwel = pd.read_csv("datasets/pauwel-dataset.txt", sep='\t', header=0)
pauwel

,abdominal cramps,abdominal distention,abdominal pain,malformations,spontaneous abortion,missed abortion,abscess,acanthosis nigricans,acidosis,renal tubular acidosis,...,drug dependence,diverticulosis,prostatic hypertrophy,allergic reaction,dysphonia,eosinophilic pneumonia,retinal vein thrombosis,renal insufficiency,glioblastoma multiforme,portal cirrhosis
carnitine,1,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,0,0,0,0
GABA,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
delta-aminolevulinic acid,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
leucovorin,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
PGE2,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
pimecrolimus,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
auranofin,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
cefditoren,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
nitroprusside,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
mizutani = pd.read_csv("datasets/mizutani-dataset.txt", sep='\t', header=0)
mizutani # drug names are coded with pubchem ID

,abdominal.cramps,abdominal.distention,abdominal.pain,malformations,spontaneous.abortion,missed.abortion,abscess,acanthosis.nigricans,acidosis,renal.tubular.acidosis,...,vitamin.deficiency,drug.dependence,diverticulosis,prostatic.hypertrophy,allergic.reaction,dysphonia,eosinophilic.pneumonia,retinal.vein.thrombosis,renal.insufficiency,glioblastoma.multiforme
85,1,0,0,0,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
119,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
137,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
143,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
158,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6398525,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6398970,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,1,0,0,0
6447131,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
6918453,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
liu = pd.read_csv("datasets/liu-dataset.csv", sep=',', header=None)
liu

,0,1,2,3,4,5,6,7,8,9,...,1375,1376,1377,1378,1379,1380,1381,1382,1383,1384
0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
827,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
828,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
829,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
830,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


In [5]:
# Check number of missing values
new_pauwel = pauwel.fillna(999).values
num_missing_pauwel = np.argwhere(new_pauwel == 999)
print("The number of missing values in pauwel's dataset:", len(num_missing_pauwel))
new_mizutani = mizutani.fillna(999).values
num_missing_mizutani = np.argwhere(new_mizutani == 999)
print("The number of missing values in mizutani's dataset:",len(num_missing_mizutani))
new_liu = liu.fillna(999).values
num_missing_liu = np.argwhere(new_liu == 999)
print("The number of missing values in pauwel's dataset:" ,len(num_missing_liu))

The number of missing values in pauwel's dataset: 0
The number of missing values in mizutani's dataset: 0
The number of missing values in pauwel's dataset: 0


In [6]:
def density(df):
    """Calculate the density of the matrix."""
    
    density = float(len(np.nonzero(df.to_numpy())[0]))
    known_se = density
    density /= (df.shape[0]*df.shape[1])
    density *= 100
    return '{:.2f}%'.format(density), known_se

print("Pauwel:", density(pauwel))
print("Liu:", density(liu))
print("Mizutani:", density(mizutani))

Pauwel: ('4.97%', 61102.0)
Liu: ('5.14%', 59205.0)
Mizutani: ('5.57%', 49051.0)


# Implementation

## Original (Non-Numba) Implementation

In [7]:
class SGD_Recommender:
    
    def __init__(self, k:int, lmbda:float, max_iter:int=350, learn_rate=0.005, tolerance=1e-06, seed:int=5):
        """Sets the parameters for SGD."""
        
        self.k=k
        self.lmbda=lmbda
        self.max_iter=max_iter
        self.learn_rate=learn_rate
        self.tolerance=tolerance
        self.seed = seed
            
    def fit(self, train: np.ndarray) -> None:
        """Train the SGD model.
        
        Args:            
            train (np.ndarray): The training set
        Returns: 
            None        
        """     
        np.random.seed(self.seed)
        m, n = train.shape
        
        # Initialize the low rank matrices U and V with values from the normal distribution N(0,0.01)
        mu, sigma = 0, 0.01
        self.U = np.random.normal(loc=mu, scale=sigma, size=(m, self.k))
        self.V = np.random.normal(loc=mu, scale=sigma, size=(n, self.k))
        
        # Get non-zero values in train set
        drug, se = train.nonzero()
        drug_se = list(zip(drug, se))
        
        # Start of training
        for _ in range(self.max_iter):
            np.random.shuffle(drug_se) # Shuffle in place
            U_old = self.U.copy()
            V_old = self.V.copy()
            
            for (drug, se) in drug_se:
                error = train[drug,se] - self.predictions(self.U[drug,:], self.V[se,:])
                temp_u = self.U[drug,:] + self.learn_rate*(error*self.V[se,:] - self.lmbda*self.U[drug,:])
                temp_v = self.V[se,:] + self.learn_rate*(error*self.U[drug,:] - self.lmbda*self.V[se,:])
                self.U[drug,:] = temp_u 
                self.V[se,:] = temp_v
            
            if self.converged(U_old, self.U) and self.converged(V_old, self.V):
                break
    
    def predict(self) -> np.ndarray:
        """Predict the entire drug-side effect matrix values."""
        
        return self.predictions(self.U, self.V)

            
    def predictions(self, U: np.ndarray, V: np.ndarray) -> np.ndarray:
        """Return dot product of the matrices U and V."""
        
        return np.dot(U, V.T)
    
    def converged(self, old: np.ndarray, curr: np.ndarray) -> bool:
        """Check if matrices have reached convergence."""
        
        return np.all(np.abs(np.subtract(old,curr)) <= self.tolerance)
            
    

## Numba Implementation

In [8]:
  
@njit(cache=True)
def predictions(U: np.ndarray, V: np.ndarray) -> np.ndarray:
    """Return dot product of the matrices U and V."""
    return np.dot(U, V.T)

@njit(cache=True)
def converged(old: np.ndarray, curr: np.ndarray, tolerance:float=1e-4) -> bool:
    """Check if matrices have reached convergence."""

    return np.all(np.abs(np.subtract(old,curr)) <= tolerance)
    
@njit(cache=True)
def fit(train: np.ndarray, k: int, lmbda: float, max_iter:int=1000, learn_rate:float=0.005, seed:int=5, tolerance:float=1e-4):
    """Train the SGD model.

    Args:
        train (np.ndarray): The training set
        k (float): Number of latent features
        lmbda (float): Regularization term.
        max_iter (int): Max number of iterations
        learn_rate (float): Learning rate
    Returns: 
        U, V (np.ndarray): The low rank matrix representation of the drug-side effect matrix.        
    """     
    np.random.seed(seed)
    m, n = train.shape
    # Initialize the low rank matrices U and V with values from the normal distribution N(0,0.01)
    mu, sigma = 0, 0.01
    U = np.random.normal(mu, sigma, (m, k))
    V = np.random.normal(mu, sigma, (n, k))
    
    drug, se = train.nonzero()
    drug_se = np.array(list(zip(drug, se)))

    converged_flag = False
    # Start of training
    for epoch in range(max_iter):

        np.random.shuffle(drug_se) # Shuffle in place
        U_old = U.copy()
        V_old = V.copy()

        # Learn from the known  associations in the training set (drug_se)
        for (drug, se) in drug_se:
            error = train[drug,se] - predictions(U[drug,:], V[se,:])
            temp_u = U[drug,:] + learn_rate*(error*V[se,:] - lmbda*U[drug,:])
            temp_v = V[se,:] + learn_rate*(error*U[drug,:] - lmbda*V[se,:])
            U[drug,:] = temp_u 
            V[se,:] = temp_v

        if converged(U_old, U, tolerance) and converged(V_old, V, tolerance):
            print("Convergence reached at epoch", epoch)
            converged_flag=True
            break
    print("Convergence state:", converged_flag)
    return U, V

@njit(cache=True)   
def predict(U:np.ndarray, V:np.ndarray) -> np.ndarray:
        """Predict the entire drug-side effect matrix values."""
        
        return predictions(U, V)

## Performance Metrics

### AUPR

In [9]:
def aupr(truth: np.ndarray, predictions: np.ndarray) -> float:
    """Get the area under the precision-recall curve, using trapezoidal rule.
    
    Args:
        truth: 1-D vector of ground truth values
        predictions: 1-D vector of predictions

    Returns:
        (float): The area.
    """
    
    max_value = predictions.max()
    min_value = predictions.min()
    # Create an array of 99 representing the thresholds
    threshold = min_value + (max_value-min_value)*np.arange(1,100,1)/100
    
    tn = np.zeros((threshold.size, 1))
    tp = np.zeros((threshold.size, 1))
    fn = np.zeros((threshold.size, 1))
    fp = np.zeros((threshold.size, 1))
    
    # Calculate the tp, tn, fp, fn for every threshold.
    for i in range(threshold.size):
        tp[i,0] = np.logical_and(predictions>=threshold[i], truth==1).sum()
        tn[i,0] = np.logical_and(predictions<threshold[i], truth==0).sum()
        fp[i,0] = np.logical_and(predictions>=threshold[i], truth==0).sum()
        fn[i,0] = np.logical_and(predictions<threshold[i], truth==1).sum()
    
    # Calculate the area under the precision-recall curve
    recall = tp/(tp+fn)
    prec = tp/(tp+fp)
    
    x = recall
    y = prec
    
    sorted_index = x.argsort(axis=0)
    y = np.take_along_axis(y, sorted_index, axis=0)
    x = np.take_along_axis(x, sorted_index, axis=0)    

    x[0][0] = 0
    y[0][0] = 1
    x = np.append(x, [[1]], axis=0)
    y = np.append(y, [[0]], axis=0)

    
    # Calculate the area using the trapezoidal rule: (b-a)*0.5*(f(b)+f(a))
    area = 0
    area = 0.5*x[0][0]*(1+y[0][0]) # still 0
    for i in range(threshold.size):
        area += (y[i][0]+ y[i+1][0])*(x[i+1][0]-x[i][0])*0.5
    return area
    

### AUROC

In [10]:

def auroc(truth: np.ndarray, predictions: np.ndarray) -> float:
    """Get the area under the ROC curve.
    
    Args:
        truth: 1-D vector of ground truth values
        predictions: 1-D vector of predictions

    Returns:
        (float): The area.
    """
    
    max_value = predictions.max()
    min_value = predictions.min()
    # Create an array of 99 representing the thresholds
    threshold = min_value + (max_value-min_value)*np.arange(1,100,1)/100
    
    tn = np.zeros((threshold.size, 1))
    tp = np.zeros((threshold.size, 1))
    fn = np.zeros((threshold.size, 1))
    fp = np.zeros((threshold.size, 1))
    
    # Calculate the tp, tn, fp, fn for every threshold.
    for i in range(threshold.size):
        tp[i,0] = np.logical_and(predictions>=threshold[i], truth==1).sum()
        tn[i,0] = np.logical_and(predictions<threshold[i], truth==0).sum()
        fp[i,0] = np.logical_and(predictions>=threshold[i], truth==0).sum()
        fn[i,0] = np.logical_and(predictions<threshold[i], truth==1).sum()
    
    # Calculate the area under the precision-recall curve
    sn = tp/(tp+fn)
    sp = tn/(tn+fp)
    x = 1 - sp
    y = sn

    sorted_index = x.argsort(axis=0)
    y = np.take_along_axis(y, sorted_index, axis=0)
    x = np.take_along_axis(x, sorted_index, axis=0)

    sorted_index = y.argsort(axis=0)
    y = np.take_along_axis(y, sorted_index, axis=0)
    x = np.take_along_axis(x, sorted_index, axis=0)

    x = np.append(x, [[1]], 0)
    y = np.append(y, [[1]], 0)
    
    # Calculate the area using the trapezoidal rule: (b-a)*0.5*(f(b)+f(a))
    area = 0
    area = 0.5*x[0][0]*y[0][0] # still 0
    for i in range(threshold.size):
        area += (y[i][0]+ y[i+1][0])*(x[i+1][0]-x[i][0])*0.5
    
    return area

### Sensitivity, specificity, precision, recall, accuracy, f1-measure

In [11]:
@njit(cache=True)
def classification_metric(truth: np.ndarray, predictions: np.ndarray) -> tuple:
    """Calculate the evaulation metrics given 1-D vector of ground truth and predictions.
    
    Args:
        truth: 1-D vector of ground truth values
        predictions: 1-D vector of predictions

    Returns:
        sensitivity, specificity, precision, accuracy, f1
    """
    
    tp = np.logical_and(predictions==1, truth==1).sum()
    tn = np.logical_and(predictions==0, truth==0).sum()
    fp = np.logical_and(predictions==1, truth==0).sum()
    fn = np.logical_and(predictions==0, truth==1).sum()
    
    try:
        acc = (tp+tn)/(tn+tp+fn+fp)
    except:
        acc = 0
    try:
        sn = tp/(tp+fn)
    except:
        sn=0
    recall = sn
    try:
        sp = tn/(tn+fp)
    except:
        sp = 0
    try:
        prec = tp/(tp+fp)
    except:
        prec = 0
    try:
        f1 = (2.0*prec*recall)/(recall+prec)  
    except:
        f1 = 0
    return sn,sp,prec,acc,f1
    

In [12]:
@njit(cache=True)
def do_stuff():
    try:
        x = 0//0
    except:
        print("O")
do_stuff()

O


In [13]:
def get_metric(truth: np.ndarray, predictions: np.ndarray) -> tuple:
    """Calculate the metrics of the drug-side effect matrix.
    Args:
        truth: 1-D vector of ground truth values
        predictions: 1-D vector of predictions

    Returns:
        sensitivity, specificity, precision, accuracy, f1
    """

    max_value = predictions.max()
    min_value = predictions.min()
    # Create an 1-D array of 999 threshold values in ascending order
    threshold = min_value + (max_value-min_value)*np.arange(1,1000,1)/1000
    temp_sn = np.zeros(threshold.size)
    temp_sp = np.zeros(threshold.size)
    temp_prec = np.zeros(threshold.size)
    temp_acc = np.zeros(threshold.size)
    temp_f1 = np.zeros(threshold.size)

    for i in range(threshold.size):
        # assign values above threshold to 1
        predict_label = predictions>threshold[i]
        # calculate the metrics for the predictions under threshold i
        temp_sn[i],temp_sp[i],temp_prec[i],temp_acc[i],temp_f1[i] = classification_metric(truth, predict_label)
    
    # Get index corresponding to max f1 score (optimal value of prec and recall)
    try:
        indx_max_f1 = np.nanargmax(temp_f1)
        sn = temp_sn[indx_max_f1]
        sp = temp_sp[indx_max_f1]
        prec = temp_prec[indx_max_f1]
        acc = temp_acc[indx_max_f1]
        f1 = temp_f1[indx_max_f1]
    except:
        print("Exception: F1 vector is all 0.0")
        sn, sp, prec, acc, f1  = 1, 1, 1, 1, 1
        
    return sn, sp, prec, acc, f1    

## Cross-validation

In [14]:
def cross_val(cv:int, dataset:np.ndarray, lmda:float, k:int, learn_rate:float=0.005, tolerance:float=1e-4, seed:int=0) -> np.ndarray:
    interaction_matrix = dataset
    row, col = dataset.shape
    np.random.seed(seed)
    cv_matrix = np.ceil(np.random.rand(row,col)*cv)

    predict_score_matrix = np.zeros((row,col))

    for fold in range(cv):
        print("Performing CV fold: {0}".format(fold))
        test_index_matrix = (cv_matrix==fold)
        train_index_matrix = np.logical_not(test_index_matrix)
        train_interaction_matrix = np.multiply(interaction_matrix,train_index_matrix)

        # Uncomment for the non-numba implementation
        # sgd = SGD_Recommender(k=k, lmbda=lmda, seed=seed)
        # sgd.fit(train=train_interaction_matrix)
        # predict_matrix = sgd.predict()

        # Numba implementation
        U, V = fit(train=train_interaction_matrix, k=k, lmbda=lmda, learn_rate=learn_rate, tolerance=tolerance, seed=seed)
        predict_matrix = predict(U, V)

        predict_score_matrix = predict_score_matrix + np.multiply(predict_matrix, test_index_matrix)
    
    auc = auroc(truth=interaction_matrix.flatten(), predictions=predict_score_matrix.flatten())
    auprc = aupr(truth=interaction_matrix.flatten(), predictions=predict_score_matrix.flatten())
    sn, sp, prec, acc, f1 = get_metric(truth=interaction_matrix.flatten(), predictions=predict_score_matrix.flatten())

    return np.array([auprc, auc, sn, sp, prec, acc, f1])
        
        



## Functions to run training

Optimal parameters are those which yield highest AUPR

In [15]:
def param_selection(dataset: np.ndarray, k_values: np.ndarray, lmda_values: np.ndarray, 
                    learn_rates: np.ndarray=np.empty(0), tolerance_values:np.ndarray=np.empty(0)) -> dict:
    """Calculate the metrics of the model, for each pair of given k and lambda parameter values.
    
    Args:
        dataset (np.ndarray): The dataset
        k_values (np.ndarray): List of k values
        lmda_values (np.ndarray): List of lambda values
        learn_rates (np.ndarray): List of learn rates
        tolerance_values (np.ndarray): List of tolerance values
    Returns:
        dict: Dictionary of parameter combinations and their associated score.
    """
    results = {}
    if len(learn_rates) == 0:
        # Do 5 fold CV for each possible combination of lambda and k (product function gives the cartesian product)
        for (k,lmbda) in product(k_values, lmda_values):
            print("Trying parameters: (" , k, ", ", lmbda, ")")
            res = cross_val(cv=5, dataset=dataset, k=k, lmda=lmbda)
            print(res)
            results[(k,lmbda)] = res
    else:
        # Do 5 fold CV for each possible combination of lambda and k and learn rate
        for (k,lmbda,learn_rate,tolerance_rate) in product(k_values, lmda_values, learn_rates, tolerance_values):
            print("Trying parameters: k={0},lambda={1},learn_rate={2},tolerance={3}".format(k,lmbda,learn_rate,tolerance_rate))
            res = cross_val(cv=5, dataset=dataset, k=k, lmda=lmbda, learn_rate = learn_rate, tolerance=tolerance_rate)
            print(res)
            results[(k, lmbda, learn_rate, tolerance_rate)] = res
    
    return results
        
    


20 independent runs of 5 fold CV, using optimal paramters.

In [16]:
def train_optimal_params(dataset: np.ndarray, lmbda: float, k: int) -> dict:
    """20 independent runs of 5 fold CV.
   
    Args:
        dataset (np.ndarray): The dataset
        k (float): k
        lmbda (float): lambda
    Returns:
        np.ndarray: array of resulting metrics.
    """
    # stores the aupr, auroc, sn, sp, prec, acc, f1
    results = np.zeros(7)
    for i in range(20):
        # ensures independence through the different seed value in every run
        res =  cross_val(cv=5, dataset=dataset, lmda=lmbda, k=k, seed=i)
        # element wise addition
        results = results + res    
    
    # return the mean over the 20 independent runs
    return results/20.0

# Training parameters


In [17]:
k_values = np.array([1,3,5,10,15,20,25,30,35,40,50,100])
lambda_values = np.array([0.01,0.1,0.5,1.,5.,10.,15.,20.])
learning_rates = np.array([0.1,0.01,1e-3,1e-4,1e-5])
tolerance_values = np.array([1e-4,1e-3])

In [18]:
k_values, lambda_values, learning_rates, tolerance_values

(array([  1,   3,   5,  10,  15,  20,  25,  30,  35,  40,  50, 100]),
 array([1.0e-02, 1.0e-01, 5.0e-01, 1.0e+00, 5.0e+00, 1.0e+01, 1.5e+01,
        2.0e+01]),
 array([1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05]),
 array([0.0001, 0.001 ]))

## Pauwel

### Grid search over the above parameters

In [19]:
# Commented out cause done 

# %time pauwel_results= param_selection(dataset=pauwel.values.copy(), k_values=k_values, lmda_values=lambda_values, learn_rates=learning_rates, tolerance_values=tolerance_values)


Time taken for below:
CPU times: user 1h 24min 1s, sys: 4min 53s, total: 1h 28min 54s
Wall time: 1h 19min 29s

In [20]:
# TODO:
# Testing the time for tolerance 1e-05 and learn rate 0.1
%time pauwel_results_2= param_selection(dataset=pauwel.values.copy(), k_values=k_values, lmda_values=lambda_values, learn_rates=np.array([0.1]), tolerance_values=np.array([1e-5]))


Trying parameters: k=1,lambda=0.01,learn_rate=0.1,tolerance=1e-05
Performing CV fold: 0
Convergence reached at epoch 117
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 123
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 126
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 124
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 122
Convergence state: True
[0.42855698 0.51524807 0.79544041 0.2352996  0.05157539 0.26312811
 0.09686986]
Trying parameters: k=1,lambda=0.1,learn_rate=0.1,tolerance=1e-05
Performing CV fold: 0
Convergence reached at epoch 94
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 93
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 97
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 95
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 94
Convergence state: True
[0.4

/tmp/ipykernel_525/1879514621.py:30: RuntimeWarning: invalid value encountered in divide
  sn = tp/(tp+fn)
/tmp/ipykernel_525/1879514621.py:31: RuntimeWarning: invalid value encountered in divide
  sp = tn/(tn+fp)


[       nan        nan 0.         1.         0.         0.95031873
 0.        ]
Trying parameters: k=3,lambda=0.01,learn_rate=0.1,tolerance=1e-05
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Performing CV fold: 4
Convergence state: False
[0.4285522  0.51507363 0.79544041 0.2352996  0.05157539 0.26312811
 0.09686986]
Trying parameters: k=3,lambda=0.1,learn_rate=0.1,tolerance=1e-05
Performing CV fold: 0
Convergence reached at epoch 360
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 370
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 369
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 370
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 369
Convergence state: True
[0.42854348 0.5149174  0.79544041 0.2352996  0.05157539 0.26312811
 0.09686986]

In [27]:
# pauwel_results_df_2 = pd.DataFrame.from_dict(pauwel_results_2)
# pauwel_results_df_2.set_index(pd.Series(['auprc','auroc', 'sn', 'sp', 'prec', 'acc', 'f1']),inplace=True)
# pauwel_results_df_2.replace(1.0,0.0,inplace=True)
# # Save to CSV
# pauwel_results_df_2.to_csv('results_pauwel_2.csv', index=True)

In [28]:
pauwel_results_df_2 = pd.read_csv('results_pauwel_2.csv', header=[0,1,2,3],index_col=0)
pauwel_results_df_2

1                                                              \
           0.01       0.1       0.5       1.0       5.0      10.0      15.0   
            0.1       0.1       0.1       0.1       0.1       0.1       0.1   
          1e-05     1e-05     1e-05     1e-05     1e-05     1e-05     1e-05   
auprc  0.428557  0.428549  0.428565  0.027914  0.024841  0.024841  0.024841   
auroc  0.515248  0.515082  0.515311  0.493458  0.500000  0.500000  0.500000   
sn     0.795440  0.795440  0.795440  0.998052  0.000000  0.000000  0.000000   
sp     0.235300  0.235300  0.235300  0.016240  0.000016  0.000016  0.000016   
prec   0.051575  0.051575  0.051575  0.050367  0.049682  0.049682  0.049682   
acc    0.263128  0.263128  0.263128  0.065018  0.049697  0.049697  0.049697   
f1     0.096870  0.096870  0.096870  0.095894  0.094661  0.094661  0.094661   

                        3            ...        50                 100  \
           20.0      0.01       0.1  ...      15.0      20.0      0.01   
            0.1       0.1       0.1  ...       0.1       0.1       0.1   
          1e-05     1e-05     1e-05  ...     1e-05     1e-05     1e-05   
auprc       NaN  0.428552  0.428543  ...  0.024841       NaN  0.428539   
auroc       NaN  0.515074  0.514917  ...  0.500000       NaN  0.514763   
sn     0.000000  0.795440  0.795440  ...  0.000000  0.000000  0.795244   
sp     0.000000  0.235300  0.235300  ...  0.000015  0.000000  0.245800   
prec   0.000000  0.051575  0.051575  ...  0.049682  0.000000  0.052244   
acc    0.950319  0.263128  0.263128  ...  0.049695  0.950319  0.273097   
f1     0.000000  0.096870  0.096870  ...  0.094661  0.000000  0.098046   

                                                                             
            0.1       0.5       1.0       5.0      10.0      15.0      20.0  
            0.1       0.1       0.1       0.1       0.1       0.1       0.1  
          1e-05     1e-05     1e-05     1e-05     1e-05     1e-05     1e-05  
auprc  0.428527  0.428566  0.028269  0.024841  0.024841  0.024841       NaN  
auroc  0.514552  0.515300  0.502491  0.500002  0.500002  0.500002       NaN  
sn     0.795440  0.795440  0.997987  0.000000  0.000000  0.000000  0.000000  
sp     0.235300  0.235300  0.017630  0.000016  0.000016  0.000016  0.000000  
prec   0.051575  0.051575  0.050431  0.049682  0.049682  0.049682  0.000000  
acc    0.263128  0.263128  0.066336  0.049697  0.049697  0.049697  0.950319  
f1     0.096870  0.096870  0.096011  0.094661  0.094661  0.094661  0.000000  

[7 rows x 96 columns]

Time taken for below:
CPU times: user 12h 26min 13s, sys: 19min 45s, total: 12h 45min 58s
Wall time: 12h 7min 53s

In [21]:
# TODO: Compute values for the rest of the learn rates
%time pauwel_results_3= param_selection(dataset=pauwel.values.copy(), k_values=k_values, lmda_values=lambda_values, learn_rates=np.array([0.01,1e-3,1e-4,1e-5]), tolerance_values=np.array([1e-5]))


Trying parameters: k=1,lambda=0.01,learn_rate=0.01,tolerance=1e-05
Performing CV fold: 0
Convergence reached at epoch 857
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 891
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 921
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 916
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 900
Convergence state: True
[0.42855698 0.51524807 0.79161075 0.39446242 0.06397102 0.41419325
 0.11837594]
Trying parameters: k=1,lambda=0.01,learn_rate=0.001,tolerance=1e-05
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Performing CV fold: 4
Convergence state: False
[0.09245565 0.69785501 0.58875323 0.82985819 0.15319037 0.81787979
 0.24312178]
Trying parameters: k=1,lambda=0.01,learn_rate=0.0001,tolerance=1e-05
Perfor

In [30]:
# pauwel_results_df_3 = pd.DataFrame.from_dict(pauwel_results_3)
# pauwel_results_df_3.set_index(pd.Series(['auprc','auroc', 'sn', 'sp', 'prec', 'acc', 'f1']),inplace=True)
# pauwel_results_df_3.replace(1.0,0.0,inplace=True)
# # Save to CSV
# pauwel_results_df_3.to_csv('results_pauwel_3.csv', index=True)

In [32]:
pauwel_results_df_3 = pd.read_csv('results_pauwel_3.csv', header=[0,1,2,3],index_col=0)
pauwel_results_df_3

1                                                              \
           0.01                                     0.1                       
           0.01     0.001    0.0001     1e-05      0.01     0.001    0.0001   
          1e-05     1e-05     1e-05     1e-05     1e-05     1e-05     1e-05   
auprc  0.428557  0.092456  0.395196  0.049266  0.428549  0.093923  0.401552   
auroc  0.515248  0.697855  0.769959  0.501436  0.515082  0.668702  0.775964   
sn     0.791611  0.588753  0.458774  0.931623  0.795260  0.720107  0.451213   
sp     0.394462  0.829858  0.971237  0.074303  0.248086  0.781139  0.971317   
prec   0.063971  0.153190  0.454696  0.049983  0.052395  0.146765  0.451272   
acc    0.414193  0.817880  0.945777  0.116896  0.275271  0.778107  0.945478   
f1     0.118376  0.243122  0.456726  0.094876  0.098313  0.243834  0.451242   

                                     ...       100                      \
                      0.5            ...      10.0                15.0   
          1e-05      0.01     0.001  ...    0.0001     1e-05      0.01   
          1e-05     1e-05     1e-05  ...     1e-05     1e-05     1e-05   
auprc  0.048702  0.428565  0.434288  ...  0.024841  0.027607  0.024841   
auroc  0.508766  0.515311  0.661476  ...  0.500003  0.500188  0.500002   
sn     0.884685  0.791300  0.691172  ...  0.000000  0.994697  0.000000   
sp     0.142569  0.401040  0.708258  ...  0.004853  0.151463  0.000017   
prec   0.051180  0.064604  0.110205  ...  0.049911  0.057745  0.049682   
acc    0.179438  0.420429  0.707409  ...  0.054293  0.193356  0.049698   
f1     0.096761  0.119456  0.190099  ...  0.095077  0.109153  0.094661   

                                                                             
                                         20.0                                
          0.001    0.0001     1e-05      0.01     0.001    0.0001     1e-05  
          1e-05     1e-05     1e-05     1e-05     1e-05     1e-05     1e-05  
auprc  0.024841  0.024841  0.026290  0.024841  0.024841  0.024841  0.025945  
auroc  0.500002  0.500002  0.486961  0.500002  0.500002  0.500002  0.497683  
sn     0.000000  0.000000  0.997087  0.000000  0.000000  0.000000  0.998953  
sp     0.000197  0.002159  0.119426  0.000016  0.000186  0.001037  0.074967  
prec   0.049691  0.049783  0.055887  0.049682  0.049690  0.049730  0.053439  
acc    0.049868  0.051733  0.163030  0.049697  0.049858  0.050667  0.120872  
f1     0.094677  0.094845  0.105842  0.094661  0.094676  0.094749  0.101451  

[7 rows x 384 columns]

### Analysing Pauwel Results

In [ ]:
# result_gridsearch_pauwel.set_index(pd.Series(['auprc','auroc', 'sn', 'sp', 'prec', 'acc', 'f1']), inplace=True)
# result_gridsearch_pauwel.replace(1.0,0.0,inplace=True)
# # Save to CSV
# result_gridsearch_pauwel.to_csv('results_pauwel.csv', index=True)

In [ ]:
result_gridsearch_pauwel = pd.read_csv('results/results_pauwel.csv', header=[0,1,2,3], index_col=0)
result_gridsearch_pauwel

1                                                              \
           0.01                                                               
            0.1                0.01               0.001              0.0001   
         0.0001     0.001    0.0001     0.001    0.0001     0.001    0.0001   
auprc  0.428557  0.428557  0.428557  0.348701  0.092456  0.049287  0.049271   
auroc  0.515248  0.515851  0.525599  0.682339  0.697855  0.501889  0.501535   
sn     0.792527  0.786210  0.783771  0.548165  0.588753  0.927400  0.927891   
sp     0.349199  0.482105  0.514532  0.823717  0.829858  0.078876  0.078127   
prec   0.059853  0.073528  0.077833  0.139832  0.153190  0.050003  0.049989   
acc    0.371224  0.497214  0.527908  0.810027  0.817880  0.121032  0.120344   
f1     0.111300  0.134479  0.141603  0.222824  0.243122  0.094890  0.094868   

                                     ...    100                            \
                                     ...   20.0                             
                    1e-05            ...    0.1            0.01             
          0.001    0.0001     0.001  ... 0.0001 0.001    0.0001     0.001   
auprc  0.049271  0.049266  0.049266  ...    0.0   0.0  0.024841  0.024841   
auroc  0.501535  0.501436  0.501436  ...    0.0   0.0  0.500002  0.500002   
sn     0.927891  0.931623  0.931623  ...    0.0   0.0  0.000000  0.000000   
sp     0.078127  0.074303  0.074303  ...    0.0   0.0  0.000157  0.000614   
prec   0.049989  0.049983  0.049983  ...    0.0   0.0  0.049689  0.049710   
acc    0.120344  0.116896  0.116896  ...    0.0   0.0  0.049831  0.050265   
f1     0.094868  0.094877  0.094877  ...    0.0   0.0  0.094673  0.094712   

                                                                   
                                                                   
          0.001              0.0001               1e-05            
         0.0001     0.001    0.0001     0.001    0.0001     0.001  
auprc  0.024841  0.025427  0.025757  0.034406  0.034956  0.044949  
auroc  0.500002  0.500145  0.507208  0.481844  0.477915  0.502350  
sn     0.000000  0.999722  0.997840  0.924634  0.929446  0.868646  
sp     0.000868  0.035697  0.082207  0.332854  0.325791  0.237342  
prec   0.049722  0.051412  0.053781  0.067561  0.067225  0.056198  
acc    0.050506  0.083591  0.127697  0.362255  0.355781  0.268706  
f1     0.094734  0.097795  0.102062  0.125920  0.125381  0.105565  

[7 rows x 960 columns]

In [ ]:
pauwel_res = pd.DataFrame()
pauwel_res['max'] = result_gridsearch_pauwel.max(axis=1)
pauwel_res['index'] = result_gridsearch_pauwel.idxmax(axis=1)
pauwel_res.set_index(pd.Series(['auprc','auroc', 'sn', 'sp', 'prec', 'acc', 'f1']),inplace=True)
pauwel_res[['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = np.random.randint(10, size=(7,7))
pauwel_res.loc['auprc', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = result_gridsearch_pauwel.loc[:, pauwel_res.loc['auprc','index']].values
pauwel_res.loc['auroc', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = result_gridsearch_pauwel.loc[:, pauwel_res.loc['auroc','index']].values
pauwel_res.loc['sn', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = result_gridsearch_pauwel.loc[:, pauwel_res.loc['sn','index']].values
pauwel_res.loc['sp', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = result_gridsearch_pauwel.loc[:, pauwel_res.loc['sp','index']].values
pauwel_res.loc['prec', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = result_gridsearch_pauwel.loc[:, pauwel_res.loc['prec','index']].values
pauwel_res.loc['acc', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = result_gridsearch_pauwel.loc[:, pauwel_res.loc['acc','index']].values
pauwel_res.loc['f1', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = result_gridsearch_pauwel.loc[:, pauwel_res.loc['f1','index']].values


In [ ]:
pauwel_res

,max,index,aupr_val,auroc_val,sn_val,sp_val,prec_val,acc_val,f1_val
auprc,0.434356,"(15, 0.5, 0.001, 0.0001)",0.434356,0.662233,0.696295,0.705447,0.109989,0.704992,0.189970
auroc,0.775964,"(1, 0.1, 0.0001, 0.0001)",0.401552,0.775964,0.451213,0.971317,0.451272,0.945478,0.451242
sn,0.999984,"(3, 5.0, 0.1, 0.0001)",0.024841,0.499999,0.999984,0.000170,0.049689,0.049842,0.094673
sp,0.984322,"(5, 0.1, 0.0001, 0.0001)",0.134105,0.521267,0.141174,0.984322,0.320074,0.942433,0.195930
prec,0.462555,"(25, 0.1, 0.0001, 0.0001)",0.380704,0.769227,0.428398,0.973978,0.462555,0.946873,0.444822
acc,0.946873,"(25, 0.1, 0.0001, 0.0001)",0.380704,0.769227,0.428398,0.973978,0.462555,0.946873,0.444822
f1,0.451242,"(1, 0.1, 0.0001, 0.0001)",0.401552,0.775964,0.451213,0.971317,0.451272,0.945478,0.451242


## Liu

#### GridSearch over Liu Dataset

Time for running over parameters: 
- k_values = np.array([1,3,5,10,15,20,25,30,35,40,50,100])
- lambda_values = np.array([0.01,0.1,0.5,1,5,10,15,20])
- learning_rates = np.array([0.1,0.01,1e-3,1e-4,1e-5])
- tolerance_values = np.array([1e-4,1e-3,1e-5])

CPU times: user 16h 39min 25s, sys: 1h 11min 17s, total: 17h 50min 42s
Wall time: 15h 33min 8s



In [ ]:
tolerance_values = np.array([1e-5,1e-4,1e-3])


In [ ]:
# Commented out because done
# %time liu_results = param_selection(dataset=liu.values.copy(), k_values=k_values, lmda_values=lambda_values, learn_rates=learning_rates,tolerance_values=tolerance_values)

#### Analysing Results

In [ ]:
# liu_results_df = pd.DataFrame.from_dict(liu_results)
# liu_results_df.set_index(pd.Series(['auprc','auroc', 'sn', 'sp', 'prec', 'acc', 'f1']),inplace=True)
# liu_results_df.replace(1.0,0.0,inplace=True)
# # Save to CSV
# liu_results_df.to_csv('results_liu.csv', index=True)

In [ ]:
# liu_results_df

In [33]:
liu_results_df = pd.read_csv('results/results_liu.csv', header=[0,1,2,3],index_col=0)
liu_results_df

1                                                              \
           0.01                                                               
            0.1                          0.01                         0.001   
          1e-05    0.0001     0.001     1e-05    0.0001     0.001     1e-05   
auprc  0.428709  0.428709  0.428707  0.428709  0.428707  0.434099  0.103508   
auroc  0.515478  0.515478  0.516102  0.515478  0.553022  0.685065  0.688931   
sn     0.793480  0.792078  0.571236  0.789798  0.775965  0.580255  0.682493   
sp     0.239005  0.307429  0.803311  0.398866  0.574342  0.817732  0.799484   
prec   0.053455  0.058330  0.135919  0.066433  0.089863  0.147067  0.155655   
acc    0.267493  0.332330  0.791387  0.418951  0.584701  0.805531  0.793473   
f1     0.100162  0.108659  0.219590  0.122557  0.161072  0.234659  0.253495   

                                     ...       100                      \
                                     ...      20.0                       
                             0.0001  ...      0.01     0.001             
         0.0001     0.001     1e-05  ...     0.001     1e-05    0.0001   
auprc  0.103508  0.051378  0.394419  ...  0.025690  0.025690  0.025690   
auroc  0.688931  0.503129  0.771716  ...  0.500000  0.500000  0.500000   
sn     0.682493  0.872224  0.444523  ...  0.000000  0.000000  0.000000   
sp     0.799484  0.138170  0.973353  ...  0.000641  0.000141  0.001052   
prec   0.155655  0.051966  0.474660  ...  0.051410  0.051386  0.051430   
acc    0.793473  0.175885  0.946182  ...  0.051987  0.051513  0.052377   
f1     0.253495  0.098089  0.459098  ...  0.097793  0.097749  0.097829   

                                                                             
                                                                             
                   0.0001                         1e-05                      
          0.001     1e-05    0.0001     0.001     1e-05    0.0001     0.001  
auprc  0.026141  0.025690  0.026133  0.037364  0.026591  0.039087  0.046247  
auroc  0.501492  0.500000  0.494928  0.549129  0.480687  0.554015  0.498734  
sn     0.988261  0.000000  0.999493  0.946542  0.975830  0.905734  0.859252  
sp     0.162678  0.001308  0.042426  0.311307  0.251177  0.351089  0.238581  
prec   0.060084  0.051443  0.053508  0.069283  0.065928  0.070284  0.057600  
acc    0.205096  0.052620  0.091599  0.343944  0.288409  0.379586  0.270470  
f1     0.113281  0.097852  0.101578  0.129115  0.123511  0.130446  0.107963  

[7 rows x 1440 columns]

In [ ]:
# Looking at results
liu_res = pd.DataFrame()
liu_res['max'] = liu_results_df.max(axis=1)
liu_res['index'] = liu_results_df.idxmax(axis=1)
liu_res.set_index(pd.Series(['auprc','auroc', 'sn', 'sp', 'prec', 'acc', 'f1']),inplace=True)

liu_res[['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = np.random.randint(10, size=(7,7))
liu_res.loc['auprc', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = liu_results_df.loc[:, liu_res.loc['auprc','index']].values
liu_res.loc['auroc', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = liu_results_df.loc[:, liu_res.loc['auroc','index']].values
liu_res.loc['sn', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = liu_results_df.loc[:, liu_res.loc['sn','index']].values
liu_res.loc['sp', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = liu_results_df.loc[:, liu_res.loc['sp','index']].values
liu_res.loc['prec', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = liu_results_df.loc[:, liu_res.loc['prec','index']].values
liu_res.loc['acc', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = liu_results_df.loc[:, liu_res.loc['acc','index']].values
liu_res.loc['f1', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = liu_results_df.loc[:, liu_res.loc['f1','index']].values


liu_res

,max,index,aupr_val,auroc_val,sn_val,sp_val,prec_val,acc_val,f1_val
auprc,0.435891,"(5, 0.1, 0.01, 0.001)",0.435891,0.637053,0.238308,0.926213,0.148881,0.890869,0.183267
auroc,0.804669,"(3, 0.5, 0.0001, 1e-05)",0.422199,0.804669,0.456076,0.969386,0.446558,0.943012,0.451267
sn,0.999983,"(3, 10.0, 0.01, 0.001)",0.025697,0.500000,0.999983,0.002261,0.051489,0.053522,0.097934
sp,0.980183,"(30, 0.1, 0.0001, 0.0001)",0.136187,0.512371,0.152588,0.980183,0.294305,0.937662,0.200977
prec,0.475000,"(5, 0.5, 0.0001, 1e-05)",0.425172,0.800874,0.433392,0.974056,0.475000,0.946277,0.453243
acc,0.946277,"(5, 0.5, 0.0001, 1e-05)",0.425172,0.800874,0.433392,0.974056,0.475000,0.946277,0.453243
f1,0.459098,"(1, 0.01, 0.0001, 1e-05)",0.394419,0.771716,0.444523,0.973353,0.474660,0.946182,0.459098


## Mizutani

#### Gridsearch over parameters

Time for running over parameters: 
- k_values = np.array([1,3,5,10,15,20,25,30,35,40,50,100])
- lambda_values = np.array([0.01,0.1,0.5,1,5,10,15,20])
- learning_rates = np.array([0.1,0.01,1e-3,1e-4,1e-5])
- tolerance_values = np.array([1e-4])

CPU times: user 3h 51min 7s, sys: 23min 18s, total: 4h 14min 25s
Wall time: 3h 29min 28s

Time for running the parameters:
- k_values = np.array([1,3,5,10,15,20,25,30,35,40,50,100])
- lambda_values = np.array([0.01,0.1,0.5,1,5,10,15,20])
- learning_rates = np.array([0.1])
- tolerance_values = np.array([1e-3])

36 minutes

Time for running the parameters:
- k_values = np.array([1,3,5,10,15,20,25,30,35,40,50,100])
- lambda_values = np.array([0.01,0.1,0.5,1,5,10,15,20])
- learning_rates = np.array([0.01])
- tolerance_values = np.array([1e-3])
CPU times: user 26min 55s, sys: 4min 44s, total: 31min 39s
Wall time: 22min 32s


In [ ]:
# %time mizutani_results = param_selection(dataset=mizutani.values.copy(), k_values=k_values, lmda_values=lambda_values, learn_rates=learning_rates,tolerance_values=tolerance_values)

In [ ]:
k_values = np.array([1,3,5,10,15,20,25,30,35,40,50,100])
lambda_values = np.array([0.01,0.1,0.5,1,5,10,15,20])
# learning_rates = np.array([0.1,0.01,1e-3,1e-4,1e-5])
learning_rates = np.array([0.01])
tolerance_values = np.array([1e-3])

# TODO: run for tolerance values 1e-5 and 1e-3
# %time mizutani_results_2 = param_selection(dataset=mizutani.values.copy(), k_values=k_values, lmda_values=lambda_values, learn_rates=learning_rates,tolerance_values=tolerance_values)
# %time mizutani_results_3 = param_selection(dataset=mizutani.values.copy(), k_values=k_values, lmda_values=lambda_values, learn_rates=learning_rates,tolerance_values=tolerance_values)


Trying parameters: k=1,lambda=0.01,learn_rate=0.01,tolerance=0.001
Performing CV fold: 0
Convergence reached at epoch 248
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 245
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 254
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 242
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 242
Convergence state: True
[0.07766459 0.66339072 0.71162667 0.7538434  0.14561701 0.75149308
 0.24176311]
Trying parameters: k=1,lambda=0.1,learn_rate=0.01,tolerance=0.001
Performing CV fold: 0
Convergence reached at epoch 241
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 243
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 252
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 242
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 245
Convergence state: Tr

/tmp/ipykernel_417/1932564076.py:22: RuntimeWarning: invalid value encountered in double_scalars
  f1 = (2.0*prec*recall)/(recall+prec)


[0.03204902 0.52396505 0.95912418 0.26153981 0.07112523 0.30037614
 0.13242994]
Trying parameters: k=1,lambda=5.0,learn_rate=0.01,tolerance=0.001
Performing CV fold: 0
Convergence reached at epoch 10
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 9
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 9
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 10
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 10
Convergence state: True
[0.02852029 0.49617953 0.99896027 0.01787717 0.05657304 0.0724966
 0.10708182]
Trying parameters: k=1,lambda=10.0,learn_rate=0.01,tolerance=0.001
Performing CV fold: 0
Convergence reached at epoch 11
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 10
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 11
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 11
Convergence state: True
Performi

In [ ]:
# mizutani_results_3
# mizutani_results_df_3 = pd.DataFrame.from_dict(mizutani_results_3)
# mizutani_results_df_3.set_index(pd.Series(['auprc','auroc', 'sn', 'sp', 'prec', 'acc', 'f1']),inplace=True)
# mizutani_results_df_3.replace(1.0,0.0,inplace=True)
# # Save to CSV
# mizutani_results_df_3.to_csv('results_mizutani_3.csv', index=True)


In [ ]:
mizutani_results_df_3 = pd.read_csv('results_mizutani_3.csv', header=[0,1,2,3],index_col=0)
mizutani_results_df_3

1                                                              \
           0.01       0.1       0.5       1.0       5.0      10.0      15.0   
           0.01      0.01      0.01      0.01      0.01      0.01      0.01   
          0.001     0.001     0.001     0.001     0.001     0.001     0.001   
auprc  0.077665  0.438365  0.434082  0.032049  0.028520  0.027836  0.028653   
auroc  0.663391  0.646563  0.609644  0.523965  0.496180  0.500002  0.499647   
sn     0.711627  0.272329  0.775356  0.959124  0.998960  0.999735  0.999918   
sp     0.753843  0.915365  0.552860  0.261540  0.017877  0.003073  0.001530   
prec   0.145617  0.159451  0.092748  0.071125  0.056573  0.055821  0.055749   
acc    0.751493  0.879566  0.565247  0.300376  0.072497  0.058560  0.057113   
f1     0.241763  0.201135  0.165678  0.132430  0.107082  0.105737  0.105610   

                        3            ...        50                 100  \
           20.0      0.01       0.1  ...      15.0      20.0      0.01   
           0.01      0.01      0.01  ...      0.01      0.01      0.01   
          0.001     0.001     0.001  ...     0.001     0.001     0.001   
auprc  0.027836  0.075630  0.071709  ...  0.027851  0.027846  0.144387   
auroc  0.500003  0.632685  0.613688  ...  0.500263  0.500043  0.702221   
sn     0.999898  0.638356  0.756723  ...  0.998919  0.999755  0.574993   
sp     0.000861  0.805036  0.663227  ...  0.008618  0.001478  0.899105   
prec   0.055713  0.161799  0.116975  ...  0.056072  0.055738  0.251485   
acc    0.056480  0.795757  0.668432  ...  0.063750  0.057055  0.881061   
f1     0.105544  0.258163  0.202627  ...  0.106184  0.105588  0.349923   

                                                                             
            0.1       0.5       1.0       5.0      10.0      15.0      20.0  
           0.01      0.01      0.01      0.01      0.01      0.01      0.01  
          0.001     0.001     0.001     0.001     0.001     0.001     0.001  
auprc  0.111387  0.437774  0.033402  0.030685  0.031763  0.032243  0.033670  
auroc  0.670142  0.671626  0.527389  0.488040  0.496419  0.497601  0.499047  
sn     0.700495  0.715500  0.711973  0.998328  0.999531  0.999837  0.999817  
sp     0.792738  0.706352  0.543722  0.020756  0.004024  0.001681  0.001411  
prec   0.166148  0.125606  0.084243  0.056696  0.055860  0.055753  0.055737  
acc    0.787603  0.706862  0.553089  0.075180  0.059446  0.057251  0.056995  
f1     0.268590  0.213697  0.150659  0.107299  0.105807  0.105616  0.105588  

[7 rows x 96 columns]

In [ ]:
# mizutani_results_2
# mizutani_results_df_2 = pd.DataFrame.from_dict(mizutani_results_2)
# mizutani_results_df_2.set_index(pd.Series(['auprc','auroc', 'sn', 'sp', 'prec', 'acc', 'f1']),inplace=True)
# mizutani_results_df_2.replace(1.0,0.0,inplace=True)
# # Save to CSV
# mizutani_results_df_2.to_csv('results_mizutani_2.csv', index=True)


In [ ]:
mizutani_results_df_2 = pd.read_csv('results_mizutani_2.csv', header=[0,1,2,3],index_col=0)
mizutani_results_df_2

1                                                              \
           0.01       0.1       0.5       1.0       5.0      10.0      15.0   
            0.1       0.1       0.1       0.1       0.1       0.1       0.1   
          0.001     0.001     0.001     0.001     0.001     0.001     0.001   
auprc  0.430973  0.430942  0.430933  0.033937  0.032102  0.027836  0.034509   
auroc  0.517771  0.518252  0.538773  0.540554  0.499872  0.500001  0.500061   
sn     0.760576  0.786753  0.786753  0.993762  0.998940  0.000000  0.999837   
sp     0.640851  0.407771  0.407335  0.095683  0.006286  0.000023  0.000487   
prec   0.110992  0.072631  0.072581  0.060844  0.055949  0.055674  0.055690   
acc    0.647516  0.428870  0.428458  0.145681  0.061550  0.055694  0.056123   
f1     0.193715  0.132985  0.132901  0.114667  0.105963  0.105475  0.105503   

                    3            ...        50             100            \
       20.0      0.01       0.1  ...      15.0  20.0      0.01       0.1   
        0.1       0.1       0.1  ...       0.1   0.1       0.1       0.1   
      0.001     0.001     0.001  ...     0.001 0.001     0.001     0.001   
auprc   NaN  0.430912  0.430907  ...  0.030474   NaN  0.432274  0.434124   
auroc   NaN  0.545142  0.553890  ...  0.499895   NaN  0.608227  0.582265   
sn      0.0  0.735785  0.783368  ...  0.999511   0.0  0.602088  0.595462   
sp      0.0  0.710255  0.464467  ...  0.002245   0.0  0.893065  0.741410   
prec    0.0  0.130216  0.079391  ...  0.055765   0.0  0.249215  0.119530   
acc     0.0  0.711676  0.482221  ...  0.057766   0.0  0.876866  0.733284   
f1      0.0  0.221273  0.144172  ...  0.105636   0.0  0.352517  0.199094   

                                                               
            0.5       1.0       5.0      10.0      15.0  20.0  
            0.1       0.1       0.1       0.1       0.1   0.1  
          0.001     0.001     0.001     0.001     0.001 0.001  
auprc  0.434032  0.039220  0.035950  0.027837  0.035414   NaN  
auroc  0.579876  0.541131  0.499230  0.500001  0.500592   NaN  
sn     0.783226  0.791278  0.999735  0.000000  0.999653   0.0  
sp     0.463447  0.305551  0.001168  0.000017  0.001632   0.0  
prec   0.079239  0.062947  0.055720  0.055673  0.055740   0.0  
acc    0.481250  0.332593  0.056761  0.055688  0.057195   0.0  
f1     0.143918  0.116616  0.105557  0.105475  0.105593   0.0  

[7 rows x 96 columns]

Time running this: 
PU times: user 42min 40s, sys: 12min 17s, total: 54min 58s
Wall time: 31min 1s

In [22]:
# # TODO: Training parameters for tolerance 1e-3 with the rest of the learn rates >0.1
# k_values = np.array([1,3,5,10,15,20,25,30,35,40,50,100])
# lambda_values = np.array([0.01,0.1,0.5,1,5,10,15,20])
# learning_rates = np.array([1e-3,1e-4,1e-5])
# tolerance_values = np.array([1e-3])
# %time mizutani_results_4 = param_selection(dataset=mizutani.values.copy(), k_values=k_values, lmda_values=lambda_values, learn_rates=learning_rates,tolerance_values=tolerance_values)


Trying parameters: k=1,lambda=0.01,learn_rate=0.001,tolerance=0.001
Performing CV fold: 0
Convergence reached at epoch 0
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 0
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 0
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 0
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 0
Convergence state: True
[5.65429748e-02 5.00719659e-01 9.99633035e-01 4.41099937e-04
 5.56764852e-02 5.60686989e-02 1.05478161e-01]
Trying parameters: k=1,lambda=0.01,learn_rate=0.0001,tolerance=0.001
Performing CV fold: 0
Convergence reached at epoch 0
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 0
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 0
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 0
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 0
Convergen

In [35]:
# mizutani_results_4
# mizutani_results_df_4 = pd.DataFrame.from_dict(mizutani_results_4)
# mizutani_results_df_4.set_index(pd.Series(['auprc','auroc', 'sn', 'sp', 'prec', 'acc', 'f1']),inplace=True)
# mizutani_results_df_4.replace(1.0,0.0,inplace=True)
# # Save to CSV
# mizutani_results_df_4.to_csv('results_mizutani_4.csv', index=True)


In [36]:
mizutani_results_df_4 = pd.read_csv('results_mizutani_4.csv', header=[0,1,2,3],index_col=0)
mizutani_results_df_4

1                                                              \
           0.01                           0.1                           0.5   
          0.001    0.0001     1e-05     0.001    0.0001     1e-05     0.001   
          0.001     0.001     0.001     0.001     0.001     0.001     0.001   
auprc  0.056543  0.056574  0.056576  0.056268  0.056544  0.056572  0.049839   
auroc  0.500720  0.500732  0.500737  0.500746  0.500741  0.500731  0.505503   
sn     0.999633  0.999715  0.999715  0.999266  0.999715  0.999715  0.850380   
sp     0.000441  0.000343  0.000344  0.000897  0.000350  0.000344  0.237493   
prec   0.055676  0.055676  0.055676  0.055681  0.055676  0.055676  0.061693   
acc    0.056069  0.055980  0.055981  0.056478  0.055987  0.055981  0.271614   
f1     0.105478  0.105477  0.105477  0.105484  0.105478  0.105477  0.115040   

                                     ...       100                      \
                                1.0  ...       5.0      10.0             
         0.0001     1e-05     0.001  ...     1e-05     0.001    0.0001   
          0.001     0.001     0.001  ...     0.001     0.001     0.001   
auprc  0.056409  0.056564  0.045941  ...  0.055617  0.030343  0.044572   
auroc  0.500662  0.500741  0.497851  ...  0.500572  0.513421  0.491131   
sn     0.999715  0.999715  0.878371  ...  0.995780  0.995515  0.901429   
sp     0.000367  0.000343  0.269589  ...  0.004565  0.122889  0.325302   
prec   0.055677  0.055676  0.066204  ...  0.055691  0.062717  0.073015   
acc    0.056003  0.055980  0.303481  ...  0.059748  0.171470  0.357377   
f1     0.105479  0.105477  0.123127  ...  0.105482  0.117999  0.135088   

                                                                             
                     15.0                          20.0                      
          1e-05     0.001    0.0001     1e-05     0.001    0.0001     1e-05  
          0.001     0.001     0.001     0.001     0.001     0.001     0.001  
auprc  0.053475  0.028724  0.039176  0.051698  0.028439  0.037175  0.050658  
auroc  0.500348  0.500086  0.497490  0.499891  0.499963  0.455485  0.499655  
sn     0.925486  0.999470  0.922346  0.893845  0.989623  0.944914  0.888178  
sp     0.095960  0.046759  0.325650  0.157795  0.137593  0.305126  0.192915  
prec   0.056918  0.058215  0.074619  0.058885  0.063365  0.074219  0.060926  
acc    0.142142  0.099799  0.358869  0.198773  0.185028  0.340744  0.231622  
f1     0.107241  0.110022  0.138068  0.110491  0.119103  0.137627  0.114029  

[7 rows x 288 columns]

In [37]:
# TODO Running mizutani with tolerance 1e-5 and all the parameters
k_values = np.array([1,3,5,10,15,20,25,30,35,40,50,100])
lambda_values = np.array([0.01,0.1,0.5,1.,5.,10.,15.,20.])
learning_rates = np.array([0.1,0.01,1e-3,1e-4,1e-5])
tolerance_values = np.array([1e-5])

%time mizutani_results_5 = param_selection(dataset=mizutani.values.copy(), k_values=k_values, lmda_values=lambda_values, learn_rates=learning_rates,tolerance_values=tolerance_values)


Trying parameters: k=1,lambda=0.01,learn_rate=0.1,tolerance=1e-05
Performing CV fold: 0
Convergence reached at epoch 121
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 135
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 133
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 129
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 132
Convergence state: True
[0.4309738  0.51669836 0.79248129 0.23872521 0.05782278 0.26955424
 0.10778138]
Trying parameters: k=1,lambda=0.01,learn_rate=0.01,tolerance=1e-05
Performing CV fold: 0
Convergence reached at epoch 890
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 968
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 990
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 946
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 953
Convergence state: Tr

/tmp/ipykernel_525/1879514621.py:30: RuntimeWarning: invalid value encountered in divide
  sn = tp/(tp+fn)
/tmp/ipykernel_525/1879514621.py:31: RuntimeWarning: invalid value encountered in divide
  sp = tn/(tn+fp)


[       nan        nan 0.         1.         0.         0.94432741
 0.        ]
Trying parameters: k=1,lambda=20.0,learn_rate=0.01,tolerance=1e-05
Performing CV fold: 0
Convergence reached at epoch 29
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 29
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 29
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 29
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 29
Convergence state: True
[0.02783648 0.5000018  0.99973497 0.00136176 0.05573032 0.05694378
 0.10557533]
Trying parameters: k=1,lambda=20.0,learn_rate=0.001,tolerance=1e-05
Performing CV fold: 0
Convergence reached at epoch 206
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 196
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 206
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 206
Convergence state: True

In [39]:
mizutani_results_df_5 = pd.DataFrame.from_dict(mizutani_results_5)
mizutani_results_df_5.set_index(pd.Series(['auprc','auroc', 'sn', 'sp', 'prec', 'acc', 'f1']),inplace=True)
mizutani_results_df_5.replace(1.0,0.0,inplace=True)
# Save to CSV
mizutani_results_df_5.to_csv('results_mizutani_5.csv', index=True)

In [40]:
mizutani_results_df_5

1                                                                \
          0.01                                              0.10              
        0.10000   0.01000   0.00100   0.00010   0.00001   0.10000   0.01000   
        0.00001   0.00001   0.00001   0.00001   0.00001   0.00001   0.00001   
auprc  0.430974  0.430974  0.117559  0.415517  0.056576  0.430942  0.430942   
auroc  0.516698  0.516698  0.688247  0.769706  0.500737  0.516172  0.516172   
sn     0.792481  0.786977  0.600579  0.448982  0.999715  0.792481  0.792135   
sp     0.238725  0.405296  0.842325  0.972249  0.000344  0.238725  0.250070   
prec   0.057823  0.072369  0.183378  0.488185  0.055676  0.057823  0.058622   
acc    0.269554  0.426545  0.828867  0.943118  0.055981  0.269554  0.280248   
f1     0.107781  0.132550  0.280967  0.467763  0.105477  0.107781  0.109165   

                                     ...       100                      \
                                     ...     15.00                       
        0.00100   0.00010   0.00001  ...   0.10000   0.01000   0.00100   
        0.00001   0.00001   0.00001  ...   0.00001   0.00001   0.00001   
auprc  0.118212  0.424357  0.056572  ...  0.036564  0.035949  0.035673   
auroc  0.693556  0.768502  0.500731  ...  0.499739  0.499529  0.499588   
sn     0.664777  0.449104  0.999715  ...  0.999796  0.999633  0.999878   
sp     0.817156  0.971407  0.000344  ...  0.000719  0.000734  0.000880   
prec   0.176511  0.480783  0.055676  ...  0.055700  0.055692  0.055712   
acc    0.808673  0.942329  0.055981  ...  0.056340  0.056346  0.056497   
f1     0.278955  0.464404  0.105477  ...  0.105521  0.105506  0.105544   

                                                                             
                              20.00                                          
        0.00010   0.00001   0.10000   0.01000   0.00100   0.00010   0.00001  
        0.00001   0.00001   0.00001   0.00001   0.00001   0.00001   0.00001  
auprc  0.029352  0.030403       NaN  0.034511  0.035865  0.030316  0.029241  
auroc  0.498320  0.515092       NaN  0.499590  0.499628  0.499009  0.501070  
sn     0.999796  0.994108  0.000000  0.999837  0.999918  0.999959  0.998716  
sp     0.003454  0.143046  0.000000  0.000549  0.000581  0.001828  0.075700  
prec   0.055844  0.064013  0.000000  0.055693  0.055699  0.055767  0.059886  
acc    0.058923  0.190427  0.944327  0.056182  0.056216  0.057397  0.127086  
f1     0.105780  0.120280  0.000000  0.105509  0.105520  0.105642  0.112997  

[7 rows x 480 columns]

#### Analysing Results

In [ ]:
# mizutani_results_df = pd.DataFrame.from_dict(mizutani_results)
# mizutani_results_df.set_index(pd.Series(['auprc','auroc', 'sn', 'sp', 'prec', 'acc', 'f1']),inplace=True)
# mizutani_results_df.replace(1.0,0.0,inplace=True)
# # Save to CSV
# mizutani_results_df.to_csv('results_mizutani.csv', index=True)

In [ ]:
# mizutani_results_df

1                                                                \
          0.01                                              0.10              
        0.10000   0.01000   0.00100   0.00010   0.00001   0.10000   0.01000   
         0.0001    0.0001    0.0001    0.0001    0.0001    0.0001    0.0001   
auprc  0.430974  0.430972  0.117559  0.056574  0.056576  0.430942  0.430942   
auroc  0.516698  0.565253  0.688247  0.500732  0.500737  0.516172  0.543389   
sn     0.789933  0.775071  0.600579  0.999715  0.999715  0.792359  0.786671   
sp     0.322845  0.558920  0.842325  0.000343  0.000344  0.242985  0.409240   
prec   0.064348  0.093871  0.183378  0.055676  0.055676  0.058121  0.072791   
acc    0.348849  0.570954  0.828867  0.055980  0.055981  0.273570  0.430252   
f1     0.119002  0.167461  0.280967  0.105477  0.105477  0.108298  0.133252   

                                     ...       100                      \
                                     ...     15.00                       
        0.00100   0.00010   0.00001  ...   0.10000   0.01000   0.00100   
         0.0001    0.0001    0.0001  ...    0.0001    0.0001    0.0001   
auprc  0.118212  0.056544  0.056572  ...  0.038636  0.037720  0.030927   
auroc  0.693556  0.500741  0.500731  ...  0.500301  0.499551  0.498708   
sn     0.664777  0.999715  0.999715  ...  0.999694  0.999837  0.999837   
sp     0.817156  0.000350  0.000344  ...  0.001078  0.000968  0.002407   
prec   0.176511  0.055676  0.055676  ...  0.055713  0.055715  0.055791   
acc    0.808673  0.055987  0.055981  ...  0.056674  0.056577  0.057937   
f1     0.278955  0.105478  0.105477  ...  0.105544  0.105548  0.105685   

                                                                           
                            20.00                                          
        0.00010   0.00001 0.10000   0.01000   0.00100   0.00010   0.00001  
         0.0001    0.0001  0.0001    0.0001    0.0001    0.0001    0.0001  
auprc  0.028837  0.040025     NaN  0.034700  0.030955  0.028461  0.037296  
auroc  0.500226  0.505824     NaN  0.499572  0.498976  0.500063  0.476754  
sn     0.999388  0.916169     0.0  0.999735  0.999939  0.999633  0.899982  
sp     0.053130  0.329676     0.0  0.000721  0.001502  0.032973  0.348245  
prec   0.058580  0.074568     0.0  0.055697  0.055748  0.057442  0.075280  
acc    0.105811  0.362328     0.0  0.056339  0.057088  0.086790  0.378962  
f1     0.110672  0.137912     0.0  0.105515  0.105609  0.108641  0.138938  

[7 rows x 480 columns]

In [ ]:
mizutani_results_df = pd.read_csv('results/results_mizutani.csv', header=[0,1,2,3],index_col=0)
mizutani_results_df

1                                                              \
           0.01                                               0.1             
            0.1      0.01     0.001    0.0001     1e-05       0.1      0.01   
         0.0001    0.0001    0.0001    0.0001    0.0001    0.0001    0.0001   
auprc  0.430974  0.430972  0.117559  0.056574  0.056576  0.430942  0.430942   
auroc  0.516698  0.565253  0.688247  0.500732  0.500737  0.516172  0.543389   
sn     0.789933  0.775071  0.600579  0.999715  0.999715  0.792359  0.786671   
sp     0.322845  0.558920  0.842325  0.000343  0.000344  0.242985  0.409240   
prec   0.064348  0.093871  0.183378  0.055676  0.055676  0.058121  0.072791   
acc    0.348849  0.570954  0.828867  0.055980  0.055981  0.273570  0.430252   
f1     0.119002  0.167461  0.280967  0.105477  0.105477  0.108298  0.133252   

                                     ...       100                      \
                                     ...      15.0                       
          0.001    0.0001     1e-05  ...       0.1      0.01     0.001   
         0.0001    0.0001    0.0001  ...    0.0001    0.0001    0.0001   
auprc  0.118212  0.056544  0.056572  ...  0.038636  0.037720  0.030927   
auroc  0.693556  0.500741  0.500731  ...  0.500301  0.499551  0.498708   
sn     0.664777  0.999715  0.999715  ...  0.999694  0.999837  0.999837   
sp     0.817156  0.000350  0.000344  ...  0.001078  0.000968  0.002407   
prec   0.176511  0.055676  0.055676  ...  0.055713  0.055715  0.055791   
acc    0.808673  0.055987  0.055981  ...  0.056674  0.056577  0.057937   
f1     0.278955  0.105478  0.105477  ...  0.105544  0.105548  0.105685   

                                                                          
                            20.0                                          
         0.0001     1e-05    0.1      0.01     0.001    0.0001     1e-05  
         0.0001    0.0001 0.0001    0.0001    0.0001    0.0001    0.0001  
auprc  0.028837  0.040025    NaN  0.034700  0.030955  0.028461  0.037296  
auroc  0.500226  0.505824    NaN  0.499572  0.498976  0.500063  0.476754  
sn     0.999388  0.916169    0.0  0.999735  0.999939  0.999633  0.899982  
sp     0.053130  0.329676    0.0  0.000721  0.001502  0.032973  0.348245  
prec   0.058580  0.074568    0.0  0.055697  0.055748  0.057442  0.075280  
acc    0.105811  0.362328    0.0  0.056339  0.057088  0.086790  0.378962  
f1     0.110672  0.137912    0.0  0.105515  0.105609  0.108641  0.138938  

[7 rows x 480 columns]

In [ ]:
# Looking at results
mizutani_res = pd.DataFrame()
mizutani_res['max'] = mizutani_results_df.max(axis=1)
mizutani_res['index'] = mizutani_results_df.idxmax(axis=1)
mizutani_res.set_index(pd.Series(['auprc','auroc', 'sn', 'sp', 'prec', 'acc', 'f1']),inplace=True)

mizutani_res[['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = np.random.randint(10, size=(7,7))
mizutani_res.loc['auprc', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = mizutani_results_df.loc[:, mizutani_res.loc['auprc','index']].values
mizutani_res.loc['auroc', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = mizutani_results_df.loc[:, mizutani_res.loc['auroc','index']].values
mizutani_res.loc['sn', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = mizutani_results_df.loc[:, mizutani_res.loc['sn','index']].values
mizutani_res.loc['sp', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = mizutani_results_df.loc[:, mizutani_res.loc['sp','index']].values
mizutani_res.loc['prec', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = mizutani_results_df.loc[:, mizutani_res.loc['prec','index']].values
mizutani_res.loc['acc', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = mizutani_results_df.loc[:, mizutani_res.loc['acc','index']].values
mizutani_res.loc['f1', ['aupr_val', 'auroc_val', 'sn_val', 'sp_val', 'prec_val', 'acc_val', 'f1_val']] = mizutani_results_df.loc[:, mizutani_res.loc['f1','index']].values


mizutani_res

,max,index,aupr_val,auroc_val,sn_val,sp_val,prec_val,acc_val,f1_val
auprc,0.437820,"(1, 0.5, 0.001, 0.0001)",0.437820,0.662883,0.662005,0.718881,0.121907,0.715715,0.205899
auroc,0.764790,"(100, 0.1, 0.0001, 0.0001)",0.400365,0.764790,0.445149,0.970660,0.472149,0.941404,0.458252
sn,0.999980,"(1, 20.0, 0.01, 0.0001)",0.032073,0.499944,0.999980,0.000379,0.055691,0.056029,0.105507
sp,0.980655,"(50, 0.1, 0.0001, 0.0001)",0.147276,0.523637,0.151781,0.980655,0.316270,0.934510,0.205122
prec,0.472149,"(100, 0.1, 0.0001, 0.0001)",0.400365,0.764790,0.445149,0.970660,0.472149,0.941404,0.458252
acc,0.941404,"(100, 0.1, 0.0001, 0.0001)",0.400365,0.764790,0.445149,0.970660,0.472149,0.941404,0.458252
f1,0.458252,"(100, 0.1, 0.0001, 0.0001)",0.400365,0.764790,0.445149,0.970660,0.472149,0.941404,0.458252


##  Measuring Performance
20 rounds of 5-fold CV

In [ ]:
pauwel_lmda=0.1
pauwel_k=1
%time train_optimal_params(dataset=pauwel.values.copy(), lmbda=lmda, k=k)


Now training with optimal params.....
Performing CV fold: 0
<NULL> False
Performing CV fold: 1
<NULL> False


SystemError: CPUDispatcher(<function fit at 0x7f797f1c30a0>) returned a result with an exception set

In [ ]:
print("Now training with optimal params.....")
lmda=0.1
k=100
# 20 runs of 5-CV with optimal parameters
%time train_optimal_params(dataset=mizutani.values.copy(), lmbda=lmda, k=k)


Now training with optimal params.....
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4


/tmp/ipykernel_381/1932564076.py:22: RuntimeWarning: invalid value encountered in double_scalars
  f1 = (2.0*prec*recall)/(recall+prec)


Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing

array([0.0840756 , 0.66741125, 0.58870563, 0.85999602, 0.20774251,
       0.84489258, 0.29912117])

## Training Liu

In [ ]:
print("Now training with optimal params.....")
lmda=0.1
k=15
# 20 runs of 5-CV with optimal parameters
%time train_optimal_params(dataset=liu.values.copy(), lmbda=lmda, k=k)


Now training with optimal params.....
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4


/tmp/ipykernel_381/1932564076.py:22: RuntimeWarning: invalid value encountered in double_scalars
  f1 = (2.0*prec*recall)/(recall+prec)


Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing CV fold: 0
Performing CV fold: 1
Performing CV fold: 2
Performing CV fold: 3
Performing CV fold: 4
Performing

array([0.07327932, 0.64509715, 0.60340174, 0.80435155, 0.1510599 ,
       0.79402696, 0.23483096])

# Experimentation

## Max_iter and Tolerance
Value that indicates convergence. Should be increased or decreased?

while all other parameters the same, and using the pauwel dataset:

tolerance = 1e-6, max_iter = 350: Model converges for values lambda>5.0. The epoch value at which convergence occurs is 190, 130, and 100 for lambda = 10, 15, 20 respectively.

tolerance = 1e-6, max_iter = 1000: Model still does not converge for lambda = 0.1, 1. Convergence reached for lambda >= 5.0. Lambda = 5.0 achieves convergence at epoch 352.
However, the AUPR values for (1,0.1) and (5,0.1) increased to 0.42756658 and 0.42844634 respectively, the rest being more or less the same as before. This is becuase of the increased max epoch value.

tolerance=1e-6, max_iter=2000
So, to discover the epoch where lambda 0.1 and 1 converges, max_iter was set to 2000. At this value, convergence is reached at for lambda=1.0, and for (1,0.1), at around 1800-1900 epochs and 1450-1650 epochs respectively. But the resulting AUPR values remained relatively the same as when max_iter = 1000. However, the other metrics performed better with 1000 max epochs (even though convergence was not reached).

Interesting observation:
- At (10,0.1), for max_iter=1000 AUPR = 0.4278 while at max_iter=2000 AUPR = 0.0367. The rest of the metrics were generally better with max_iter=1000. 
- Same for (15,0.1).
- For lambda = 1, values were relatively the same for when max_iter=2000 or 1000.
- For (20,0.1 and 1) values were just as bad as when max_iter =350. AUPR = 0.039. Will it be better with max_iter = 1000?
- Same with (25,0.1 and 1), (30,0.1,1). Just as bad as max_iter= 350.
- 


This suggests that max_iter does not need to be increased, and that tolerance does not need to be decreased as it is a small enough value.


In [ ]:
# tolerance value = 350, max_iter = 
%time param_selection(dataset=pauwel.values.copy(), k_values=np.array([1,5,10,15,20,25,30,35,40,50,100]), lmda_values=np.array([0.1,1]))

Trying parameters: ( 1 ,  0.1 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Performing CV fold: 4
Convergence state: False
[0.42756658 0.51596062 0.78882852 0.40463801 0.06477968 0.42372508
 0.11972721]
Trying parameters: ( 1 ,  1.0 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Performing CV fold: 4
Convergence state: False


/tmp/ipykernel_677/1932564076.py:22: RuntimeWarning: invalid value encountered in double_scalars
  f1 = (2.0*prec*recall)/(recall+prec)


[0.02484853 0.50014919 0.99839612 0.01336353 0.05024367 0.0623012
 0.09567266]
Trying parameters: ( 5 ,  0.1 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Performing CV fold: 4
Convergence state: False
[0.42844634 0.53786632 0.78794475 0.42536222 0.06688952 0.44337578
 0.12331102]
Trying parameters: ( 5 ,  1.0 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Performing CV fold: 4
Convergence state: False
[0.02794507 0.4981223  0.99774148 0.01762011 0.05041897 0.06631379
 0.0959874 ]
Trying parameters: ( 10 ,  0.1 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Perfor

{(1,
  0.1): array([0.42756658, 0.51596062, 0.78882852, 0.40463801, 0.06477968,
        0.42372508, 0.11972721]),
 (1,
  1.0): array([0.02484853, 0.50014919, 0.99839612, 0.01336353, 0.05024367,
        0.0623012 , 0.09567266]),
 (5,
  0.1): array([0.42844634, 0.53786632, 0.78794475, 0.42536222, 0.06688952,
        0.44337578, 0.12331102]),
 (5,
  1.0): array([0.02794507, 0.4981223 , 0.99774148, 0.01762011, 0.05041897,
        0.06631379, 0.0959874 ]),
 (10,
  0.1): array([0.0367349 , 0.58458458, 0.78833753, 0.41610725, 0.06592984,
        0.43460012, 0.12168314]),
 (10,
  1.0): array([0.02531204, 0.50852889, 0.99754509, 0.01951611, 0.0505021 ,
        0.06810583, 0.09613713]),
 (15,
  0.1): array([0.03533787, 0.57078717, 0.78622631, 0.45141079, 0.06970208,
        0.46804485, 0.12805186]),
 (15,
  1.0): array([0.02784215, 0.49746197, 0.99770875, 0.01897452, 0.05048349,
        0.06759928, 0.09610417]),
 (20,
  0.1): array([0.03236025, 0.53560442, 0.78586626, 0.4530176 , 0.06986276,
   

## Learn_rate, k, lambda, U and V.
Testing possible combinations of parameters k, lambda, and learn_rate. So, 11\*6\*7 combinations of parameters are to be tested, over 2 possibilities of initial values of U and V. The initial values will either be from the normal distribution or the uniform distribution.

Estimated execution time = 11 hours.


Values of k = [1,5,10,15,20,25,30,35,40,50,100] <br>
Values of lambda = [0.1,1,5,10,15,20]<br>
Values of learn_rate = [1.0,0.1,0.01,0.001,0.0001,1e-05,1e-06,1e-07]<br>
Values of U and V = N(0,0.1) or U(0,1)

In [ ]:
learning_rates = np. [1E-0, 1E-1, 1E-2, 1E-3, 1E-4, 1E-5, 1E-6, 1E-7])
k_values = np. [1,5,10,15,20,25,30,35,40,50,100])
lambda_values = np. [0.1,1,5,10,15,20])

learning_rates,lambda_values,k_values

(array([1.e+00, 1.e-01, 1.e-02, 1.e-03, 1.e-04, 1.e-05, 1.e-06, 1.e-07]),
 array([ 0.1,  1. ,  5. , 10. , 15. , 20. ]),
 array([  1,   5,  10,  15,  20,  25,  30,  35,  40,  50, 100]))

### Testing k, learn_rate, and lambda over U,V initialized to N(0,0.1)

In [ ]:
%time param_selection(dataset=pauwel.values.copy(), k_values=k_values, lmda_values=lambda_values, learn_rates=learning_rates)

Trying parameters: ( 1 ,  0.1 ,  1.0 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Performing CV fold: 4
Convergence state: False


/tmp/ipykernel_366/1879514621.py:30: RuntimeWarning: invalid value encountered in divide
  sn = tp/(tp+fn)
/tmp/ipykernel_366/1879514621.py:31: RuntimeWarning: invalid value encountered in divide
  sp = tn/(tn+fp)
/tmp/ipykernel_366/1932564076.py:21: RuntimeWarning: invalid value encountered in long_scalars
  prec = tp/(tp+fp)


Exception: F1 vector is all nan
[nan nan  1.  1.  1.  1.  1.]
Trying parameters: ( 1 ,  0.1 ,  0.1 )
Performing CV fold: 0
Convergence reached at epoch 116
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 120
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 119
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 117
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 122
Convergence state: True
[0.42756656 0.51540116 0.79332919 0.23742148 0.05158131 0.26503968
 0.0968646 ]
Trying parameters: ( 1 ,  0.1 ,  0.01 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Performing CV fold: 4
Convergence state: False
[0.43061436 0.5793632  0.20824196 0.93212312 0.13821871 0.89615979
 0.16615413]
Trying parameters: ( 1 ,  0.1 ,  0.001 )
Performing CV fold: 0
Conver

/tmp/ipykernel_366/1932564076.py:22: RuntimeWarning: invalid value encountered in double_scalars
  f1 = (2.0*prec*recall)/(recall+prec)


[0.16903098 0.73849273 0.57117607 0.89601618 0.22309728 0.87987771
 0.32086643]
Trying parameters: ( 1 ,  0.1 ,  0.0001 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Performing CV fold: 4
Convergence state: False
[0.11710847 0.64648674 0.27756866 0.92349531 0.15943296 0.89140485
 0.20253285]
Trying parameters: ( 1 ,  0.1 ,  1e-05 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Performing CV fold: 4
Convergence state: False
[0.04971631 0.50005045 0.99759419 0.00296036 0.04970753 0.05237503
 0.09469657]
Trying parameters: ( 1 ,  0.1 ,  1e-06 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Con

{(1, 0.1, 1.0): array([nan, nan,  1.,  1.,  1.,  1.,  1.]),
 (1,
  0.1,
  0.1): array([0.42756656, 0.51540116, 0.79332919, 0.23742148, 0.05158131,
        0.26503968, 0.0968646 ]),
 (1,
  0.1,
  0.01): array([0.43061436, 0.5793632 , 0.20824196, 0.93212312, 0.13821871,
        0.89615979, 0.16615413]),
 (1,
  0.1,
  0.001): array([0.16903098, 0.73849273, 0.57117607, 0.89601618, 0.22309728,
        0.87987771, 0.32086643]),
 (1,
  0.1,
  0.0001): array([0.11710847, 0.64648674, 0.27756866, 0.92349531, 0.15943296,
        0.89140485, 0.20253285]),
 (1,
  0.1,
  1e-05): array([0.04971631, 0.50005045, 0.99759419, 0.00296036, 0.04970753,
        0.05237503, 0.09469657]),
 (1,
  0.1,
  1e-06): array([5.08651531e-02, 5.00977733e-01, 9.99885437e-01, 1.98497918e-04,
        4.96852333e-02, 4.98642144e-02, 9.46664053e-02]),
 (1,
  0.1,
  1e-07): array([5.10447736e-02, 5.01792035e-01, 9.99885437e-01, 1.81386029e-04,
        4.96844252e-02, 4.98479526e-02, 9.46649385e-02]),
 (1, 1.0, 1.0): array([na

U and V are initialized with numbers from uniform distribution.

In [ ]:
%time param_selection(dataset=pauwel.values.copy(), k_values=k_values, lmda_values=lambda_values, learn_rates=learning_rates)

Trying parameters: ( 1 ,  0.1 ,  1.0 )
Performing CV fold: 0
Convergence reached at epoch 4
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 5
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 5
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 5
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 5
Convergence state: True
[0.02827034 0.51060327 0.79357468 0.2354673  0.05147135 0.26319478
 0.09667251]
Trying parameters: ( 1 ,  0.1 ,  0.1 )
Performing CV fold: 0
Convergence reached at epoch 113
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 115
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 115
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 114
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 116
Convergence state: True
[0.02827034 0.51060327 0.79357468 0.2354673  0.05147135 0.2631

/tmp/ipykernel_422/1932564076.py:22: RuntimeWarning: invalid value encountered in double_scalars
  f1 = (2.0*prec*recall)/(recall+prec)


[0.03339866 0.55575068 0.78778109 0.42957602 0.06733731 0.4473721
 0.12406951]
Trying parameters: ( 1 ,  0.1 ,  0.001 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Performing CV fold: 4
Convergence state: False
[0.09344865 0.69007059 0.67622991 0.81629189 0.16138154 0.80933343
 0.26057673]
Trying parameters: ( 1 ,  0.1 ,  0.0001 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Convergence state: False
Performing CV fold: 4
Convergence state: False
[0.24605426 0.72730721 0.37663252 0.95659997 0.31209146 0.92778645
 0.34133788]
Trying parameters: ( 1 ,  0.1 ,  1e-05 )
Performing CV fold: 0
Convergence state: False
Performing CV fold: 1
Convergence state: False
Performing CV fold: 2
Convergence state: False
Performing CV fold: 3
Conv

/tmp/ipykernel_422/1879514621.py:30: RuntimeWarning: invalid value encountered in divide
  sn = tp/(tp+fn)
/tmp/ipykernel_422/1879514621.py:31: RuntimeWarning: invalid value encountered in divide
  sp = tn/(tn+fp)
/tmp/ipykernel_422/1932564076.py:21: RuntimeWarning: invalid value encountered in long_scalars
  prec = tp/(tp+fp)


Exception: F1 vector is all nan
[nan nan  1.  1.  1.  1.  1.]
Trying parameters: ( 1 ,  5.0 ,  0.1 )
Performing CV fold: 0
Convergence reached at epoch 19
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 19
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 19
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 19
Convergence state: True
Performing CV fold: 4
Convergence reached at epoch 19
Convergence state: True
Exception: F1 vector is all nan
[0.         0.49998032 1.         1.         1.         1.
 1.        ]
Trying parameters: ( 1 ,  5.0 ,  0.01 )
Performing CV fold: 0
Convergence reached at epoch 212
Convergence state: True
Performing CV fold: 1
Convergence reached at epoch 212
Convergence state: True
Performing CV fold: 2
Convergence reached at epoch 211
Convergence state: True
Performing CV fold: 3
Convergence reached at epoch 212
Convergence state: True
Performing CV fold: 4
Convergence reached at epoc

{(1,
  0.1,
  1.0): array([0.02827034, 0.51060327, 0.79357468, 0.2354673 , 0.05147135,
        0.26319478, 0.09667251]),
 (1,
  0.1,
  0.1): array([0.02827034, 0.51060327, 0.79357468, 0.2354673 , 0.05147135,
        0.26319478, 0.09667251]),
 (1,
  0.1,
  0.01): array([0.03339866, 0.55575068, 0.78778109, 0.42957602, 0.06733731,
        0.4473721 , 0.12406951]),
 (1,
  0.1,
  0.001): array([0.09344865, 0.69007059, 0.67622991, 0.81629189, 0.16138154,
        0.80933343, 0.26057673]),
 (1,
  0.1,
  0.0001): array([0.24605426, 0.72730721, 0.37663252, 0.95659997, 0.31209146,
        0.92778645, 0.34133788]),
 (1,
  0.1,
  1e-05): array([0.06777455, 0.58356923, 0.45437138, 0.68383132, 0.06988024,
        0.67243146, 0.12113107]),
 (1,
  0.1,
  1e-06): array([0.04627416, 0.51273351, 0.74058132, 0.28143411, 0.05112559,
        0.30424513, 0.09564816]),
 (1,
  0.1,
  1e-07): array([0.04431661, 0.50268896, 0.7845897 , 0.22126358, 0.05003601,
        0.24925033, 0.09407268]),
 (1,
  1.0,
  1.0): 

In [ ]:

results_normal_dist = pd.DataFrame.from_dict({(1, 0.1, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (1,
  0.1,
  0.1):  [0.42756656, 0.51540116, 0.79332919, 0.23742148, 0.05158131,
        0.26503968, 0.0968646  ],
 (1,
  0.1,
  0.01):  [0.43061436, 0.5793632 , 0.20824196, 0.93212312, 0.13821871,
        0.89615979, 0.16615413 ],
 (1,
  0.1,
  0.001):  [0.16903098, 0.73849273, 0.57117607, 0.89601618, 0.22309728,
        0.87987771, 0.32086643 ],
 (1,
  0.1,
  0.0001):  [0.11710847, 0.64648674, 0.27756866, 0.92349531, 0.15943296,
        0.89140485, 0.20253285 ],
 (1,
  0.1,
  1e-05):  [0.04971631, 0.50005045, 0.99759419, 0.00296036, 0.04970753,
        0.05237503, 0.09469657 ],
 (1,
  0.1,
  1e-06):  [5.08651531e-02, 5.00977733e-01, 9.99885437e-01, 1.98497918e-04,
        4.96852333e-02, 4.98642144e-02, 9.46664053e-02 ],
 (1,
  0.1,
  1e-07):  [5.10447736e-02, 5.01792035e-01, 9.99885437e-01, 1.81386029e-04,
        4.96844252e-02, 4.98479526e-02, 9.46649385e-02 ],
 (1, 1.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (1,
  1.0,
  0.1):  [0.02484791, 0.50013593, 0.99841249, 0.01334556, 0.05024358,
        0.06228494, 0.09567258 ],
 (1,
  1.0,
  0.01):  [0.02484991, 0.50017828, 0.99842886, 0.01342   , 0.05024796,
        0.06235649, 0.0956806  ],
 (1,
  1.0,
  0.001):  [0.02656265, 0.52725832, 0.99761055, 0.06173114, 0.05265795,
        0.10822682, 0.10003561 ],
 (1,
  1.0,
  0.0001):  [0.02977388, 0.47506165, 0.94345521, 0.27752234, 0.06390574,
        0.31060673, 0.11970327 ],
 (1,
  1.0,
  1e-05):  [0.0444445 , 0.50064416, 0.8696442 , 0.20275963, 0.05394981,
        0.23589131, 0.10159689 ],
 (1,
  1.0,
  1e-06):  [0.04969937, 0.49943226, 0.99765965, 0.00283373, 0.04970463,
        0.05225794, 0.09469161 ],
 (1,
  1.0,
  1e-07):  [5.08347210e-02, 5.00446636e-01, 9.99885437e-01, 1.98497918e-04,
        4.96852333e-02, 4.98642144e-02, 9.46664053e-02 ],
 (1, 5.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (1,
  5.0,
  0.1):  [2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 1.36895116e-05,
        4.96819160e-02, 4.96942791e-02, 9.46608973e-02 ],
 (1,
  5.0,
  0.01):  [2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 1.36895116e-05,
        4.96819160e-02, 4.96942791e-02, 9.46608973e-02 ],
 (1,
  5.0,
  0.001):  [0.02484072, 0.50000086, 1.        , 0.00144681, 0.04974967,
        0.0510562 , 0.09478388 ],
 (1,
  5.0,
  0.0001):  [0.02611027, 0.51301847, 0.9989853 , 0.06449557, 0.05287426,
        0.1109222 , 0.10043281 ],
 (1,
  5.0,
  1e-05):  [0.03413701, 0.46210565, 0.90370201, 0.30733895, 0.06385171,
        0.33696702, 0.11927589 ],
 (1,
  5.0,
  1e-06):  [0.04657674, 0.49893415, 0.90492946, 0.12898686, 0.05151615,
        0.16753667, 0.09748276 ],
 (1,
  5.0,
  1e-07):  [5.02906468e-02, 4.99628225e-01, 9.99721777e-01, 4.53465072e-04,
        4.96895471e-02, 5.00983836e-02, 9.46735016e-02 ],
 (1, 10.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (1,
  10.0,
  0.1):  [2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 1.36895116e-05,
        4.96819160e-02, 4.96942791e-02, 9.46608973e-02 ],
 (1,
  10.0,
  0.01):  [2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 1.36895116e-05,
        4.96819160e-02, 4.96942791e-02, 9.46608973e-02 ],
 (1,
  10.0,
  0.001):  [2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 2.16465402e-04,
        4.96914918e-02, 4.98869808e-02, 9.46782787e-02 ],
 (1,
  10.0,
  0.0001):  [0.02539623, 0.51143206, 0.99985271, 0.02301635, 0.05078514,
        0.07154682, 0.09666063 ],
 (1,
  10.0,
  1e-05):  [0.02945843, 0.47665262, 0.96744787, 0.25209407, 0.06334108,
        0.28763375, 0.11889765 ],
 (1,
  10.0,
  1e-06):  [0.04448072, 0.50052664, 0.87186999, 0.19993446, 0.05389975,
        0.23331707, 0.10152324 ],
 (1,
  10.0,
  1e-07):  [0.04970768, 0.49941402, 0.99797061, 0.0024932 , 0.04970322,
        0.05194978, 0.09469045 ],
 (1, 15.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (1,
  15.0,
  0.1):  [2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 1.36895116e-05,
        4.96819160e-02, 4.96942791e-02, 9.46608973e-02 ],
 (1,
  15.0,
  0.01):  [2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 1.36895116e-05,
        4.96819160e-02, 4.96942791e-02, 9.46608973e-02 ],
 (1,
  15.0,
  0.001):  [2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 1.04382526e-04,
        4.96861984e-02, 4.97804664e-02, 9.46686705e-02 ],
 (1,
  15.0,
  0.0001):  [0.02484094, 0.50000385, 0.99996727, 0.00933368, 0.05012433,
        0.05854961, 0.09546345 ],
 (1,
  15.0,
  1e-05):  [0.0285683 , 0.51328896, 0.98520507, 0.20163453, 0.06060344,
        0.24056331, 0.11418307 ],
 (1,
  15.0,
  1e-06):  [0.04301625, 0.50080664, 0.86938234, 0.23454668, 0.05604864,
        0.26608612, 0.10530813 ],
 (1,
  15.0,
  1e-07):  [0.0491948 , 0.49941882, 0.93224444, 0.07332958, 0.04996518,
        0.11600156, 0.09484688 ],
 (1, 20.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (1, 20.0, 0.1):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (1,
  20.0,
  0.01):  [2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 1.36895116e-05,
        4.96819160e-02, 4.96942791e-02, 9.46608973e-02 ],
 (1,
  20.0,
  0.001):  [2.48407156e-02, 5.00000856e-01, 1.00000000e+00, 2.73790232e-05,
        4.96825624e-02, 4.97072885e-02, 9.46620706e-02 ],
 (1,
  20.0,
  0.0001):  [0.02484074, 0.50000086, 0.99963995, 0.01927141, 0.05059081,
        0.06797736, 0.09630758 ],
 (1,
  20.0,
  1e-05):  [0.03006325, 0.55771767, 0.99173513, 0.16896793, 0.05872434,
        0.20984405, 0.11088289 ],
 (1,
  20.0,
  1e-06):  [0.04177209, 0.49626131, 0.87129718, 0.25905176, 0.05791511,
        0.28946889, 0.10861085 ],
 (1,
  20.0,
  1e-07):  [0.04871125, 0.49916937, 0.93674511, 0.07211036, 0.05013164,
        0.11506651, 0.09517008 ],
 (5, 0.1, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (5,
  0.1,
  0.1):  [0.42755725, 0.51524126, 0.79332919, 0.23742148, 0.05158131,
        0.26503968, 0.0968646  ],
 (5,
  0.1,
  0.01):  [0.43136499, 0.59170114, 0.78287126, 0.5109807 , 0.07722921,
        0.52448857, 0.14058946 ],
 (5,
  0.1,
  0.001):  [0.1720721 , 0.73676378, 0.57145429, 0.89610602, 0.22333159,
        0.87997691, 0.32115264 ],
 (5,
  0.1,
  0.0001):  [0.37631035, 0.77164126, 0.40438938, 0.97261071, 0.43562349,
        0.94438075, 0.41942575 ],
 (5,
  0.1,
  1e-05):  [0.04857219, 0.50132092, 0.87407941, 0.14103705, 0.05051142,
        0.17745552, 0.09550385 ],
 (5,
  0.1,
  1e-06):  [0.04949404, 0.50006672, 0.9907368 , 0.00992832, 0.04971299,
        0.05865613, 0.09467537 ],
 (5,
  0.1,
  1e-07):  [4.96171302e-02, 4.99959127e-01, 1.00000000e+00, 8.55594476e-07,
        4.96813101e-02, 4.96820828e-02, 9.46597975e-02 ],
 (5, 1.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (5,
  1.0,
  0.1):  [0.02797965, 0.49820304, 0.99774148, 0.01764321, 0.0504201 ,
        0.06633574, 0.09598944 ],
 (5,
  1.0,
  0.01):  [0.02797664, 0.49814522, 0.79406566, 0.26092038, 0.05318089,
        0.28740771, 0.09968555 ],
 (5,
  1.0,
  0.001):  [0.02612356, 0.48935292, 0.98258649, 0.21173225, 0.06117911,
        0.25002927, 0.11518634 ],
 (5,
  1.0,
  0.0001):  [0.0332569 , 0.45554049, 0.92600897, 0.31744865, 0.06622835,
        0.3476827 , 0.12361568 ],
 (5,
  1.0,
  1e-05):  [0.04442563, 0.49919825, 0.86584727, 0.24057434, 0.05625169,
        0.2716387 , 0.10564024 ],
 (5,
  1.0,
  1e-06):  [0.04843933, 0.500152  , 0.92718733, 0.08484331, 0.05030153,
        0.12669203, 0.09542603 ],
 (5,
  1.0,
  1e-07):  [0.04948918, 0.50000495, 0.99072043, 0.00995741, 0.04971359,
        0.05868296, 0.0946764  ],
 (5, 5.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (5,
  5.0,
  0.1):  [2.48407762e-02, 5.00002995e-01, 1.00000000e+00, 2.05342674e-05,
        4.96822392e-02, 4.97007838e-02, 9.46614840e-02 ],
 (5,
  5.0,
  0.01):  [2.48407762e-02, 5.00002995e-01, 1.00000000e+00, 2.05342674e-05,
        4.96822392e-02, 4.97007838e-02, 9.46614840e-02 ],
 (5,
  5.0,
  0.001):  [0.02484078, 0.50000342, 0.99998363, 0.00211075, 0.04978035,
        0.05168634, 0.09483948 ],
 (5,
  5.0,
  0.0001):  [0.02604827, 0.5031809 , 0.99369906, 0.13750687, 0.05680963,
        0.18004358, 0.10747494 ],
 (5,
  5.0,
  1e-05):  [0.03740783, 0.49565657, 0.90091977, 0.33226241, 0.06588748,
        0.36051403, 0.12279455 ],
 (5,
  5.0,
  1e-06):  [0.04579028, 0.4997605 , 0.84733724, 0.2126409 , 0.05326423,
        0.24417342, 0.10022805 ],
 (5,
  5.0,
  1e-07):  [0.0489951 , 0.50010592, 0.92659815, 0.07971659, 0.05000512,
        0.12179074, 0.09488941 ],
 (5, 10.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (5,
  10.0,
  0.1):  [2.48407762e-02, 5.00002995e-01, 1.00000000e+00, 2.05342674e-05,
        4.96822392e-02, 4.97007838e-02, 9.46614840e-02 ],
 (5,
  10.0,
  0.01):  [2.48407762e-02, 5.00002995e-01, 1.00000000e+00, 2.05342674e-05,
        4.96822392e-02, 4.97007838e-02, 9.46614840e-02 ],
 (5,
  10.0,
  0.001):  [2.48407762e-02, 5.00002995e-01, 1.00000000e+00, 2.50689181e-04,
        4.96931083e-02, 4.99195043e-02, 9.46812129e-02 ],
 (5,
  10.0,
  0.0001):  [0.02522707, 0.49731388, 0.99975451, 0.03515638, 0.05138651,
        0.08307884, 0.09774882 ],
 (5,
  10.0,
  1e-05):  [0.03175041, 0.47780568, 0.95316029, 0.30047023, 0.06649654,
        0.33289671, 0.12431999 ],
 (5,
  10.0,
  1e-06):  [0.04479962, 0.50264634, 0.85930084, 0.24597314, 0.05622754,
        0.27644404, 0.10554861 ],
 (5,
  10.0,
  1e-07):  [0.04844245, 0.5001267 , 0.91106674, 0.10153169, 0.05034284,
        0.14175041, 0.09541341 ],
 (5, 15.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (5,
  15.0,
  0.1):  [2.48407762e-02, 5.00002995e-01, 1.00000000e+00, 2.05342674e-05,
        4.96822392e-02, 4.97007838e-02, 9.46614840e-02 ],
 (5,
  15.0,
  0.01):  [2.48407762e-02, 5.00002995e-01, 1.00000000e+00, 2.05342674e-05,
        4.96822392e-02, 4.97007838e-02, 9.46614840e-02 ],
 (5,
  15.0,
  0.001):  [2.48407762e-02, 5.00002995e-01, 1.00000000e+00, 1.38606305e-04,
        4.96878146e-02, 4.98129899e-02, 9.46716041e-02 ],
 (5,
  15.0,
  0.0001):  [0.02532994, 0.50691128, 0.99993454, 0.01716921, 0.05050219,
        0.06599424, 0.09614835 ],
 (5,
  15.0,
  1e-05):  [0.0292585 , 0.46213805, 0.94867598, 0.31713379, 0.06771057,
        0.34850961, 0.12639952 ],
 (5,
  15.0,
  1e-06):  [0.04502359, 0.51047472, 0.85898989, 0.27915224, 0.05864377,
        0.30795931, 0.10979198 ],
 (5,
  15.0,
  1e-07):  [0.04795543, 0.50017217, 0.91604203, 0.10206729, 0.0506325 ,
        0.14250659, 0.09596094 ],
 (5, 20.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (5, 20.0, 0.1):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (5,
  20.0,
  0.01):  [2.48407762e-02, 5.00002995e-01, 1.00000000e+00, 2.05342674e-05,
        4.96822392e-02, 4.97007838e-02, 9.46614840e-02 ],
 (5,
  20.0,
  0.001):  [2.48407762e-02, 5.00002995e-01, 1.00000000e+00, 3.16569956e-05,
        4.96827644e-02, 4.97113540e-02, 9.46624372e-02 ],
 (5,
  20.0,
  0.0001):  [0.0252094 , 0.50345697, 0.99998363, 0.01055547, 0.0501839 ,
        0.05971152, 0.09557156 ],
 (5,
  20.0,
  1e-05):  [0.0281762 , 0.44207626, 0.98469772, 0.22574689, 0.06234296,
        0.26345253, 0.11726186 ],
 (5,
  20.0,
  1e-06):  [0.04238143, 0.49460932, 0.87553599, 0.28882217, 0.06046868,
        0.31797086, 0.11312444 ],
 (5,
  20.0,
  1e-07):  [0.04757926, 0.50067229, 0.86537266, 0.16037434, 0.05112685,
        0.19539955, 0.09654948 ],
 (10, 0.1, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (10,
  0.1,
  0.1):  [0.42756878, 0.51545849, 0.79332919, 0.23742148, 0.05158131,
        0.26503968, 0.0968646  ],
 (10,
  0.1,
  0.01):  [0.03657163, 0.58291199, 0.78334588, 0.50571879, 0.07651274,
        0.51951166, 0.13940883 ],
 (10,
  0.1,
  0.001):  [0.18521295, 0.74189592, 0.54379562, 0.91217579, 0.24454274,
        0.8938742 , 0.33737105 ],
 (10,
  0.1,
  0.0001):  [0.23003784, 0.7176172 , 0.26827272, 0.96840974, 0.30746145,
        0.93362604, 0.28653335 ],
 (10,
  0.1,
  1e-05):  [0.04782405, 0.49925477, 0.86758208, 0.15200491, 0.05077055,
        0.1875557 , 0.09592746 ],
 (10,
  0.1,
  1e-06):  [0.04888879, 0.49900941, 0.95890478, 0.04493326, 0.04987096,
        0.09034052, 0.09481097 ],
 (10,
  0.1,
  1e-07):  [0.04902435, 0.49906102, 0.96957546, 0.03289162, 0.04980169,
        0.07942726, 0.09473726 ],
 (10, 1.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (10,
  1.0,
  0.1):  [0.02530752, 0.5084451 , 0.99756145, 0.01933216, 0.0504939 ,
        0.06793183, 0.09612234 ],
 (10,
  1.0,
  0.01):  [0.02563297, 0.51488343, 0.9972996 , 0.03247067, 0.05113177,
        0.08040459, 0.09727617 ],
 (10,
  1.0,
  0.001):  [0.02634243, 0.4928982 , 0.98347026, 0.21228497, 0.0612711 ,
        0.25059843, 0.11535545 ],
 (10,
  1.0,
  0.0001):  [0.0336674 , 0.49544055, 0.92466695, 0.32310156, 0.06665424,
        0.3529881 , 0.12434511 ],
 (10,
  1.0,
  1e-05):  [0.04494746, 0.50451312, 0.86615823, 0.24597229, 0.05665077,
        0.27678391, 0.10634602 ],
 (10,
  1.0,
  1e-06):  [0.04784089, 0.49853261, 0.8660764 , 0.15258159, 0.05071964,
        0.18802891, 0.09582739 ],
 (10,
  1.0,
  1e-07):  [0.04890879, 0.49904254, 0.95887205, 0.0449016 , 0.04986777,
        0.09030881, 0.09480505 ],
 (10, 5.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (10,
  5.0,
  0.1):  [2.48411398e-02, 5.00000000e-01, 1.00000000e+00, 2.13898619e-05,
        4.96822796e-02, 4.97015969e-02, 9.46615573e-02 ],
 (10,
  5.0,
  0.01):  [2.48411398e-02, 5.00000000e-01, 1.00000000e+00, 7.87146918e-05,
        4.96849864e-02, 4.97560738e-02, 9.46664704e-02 ],
 (10,
  5.0,
  0.001):  [0.0252645 , 0.50787688, 0.998167  , 0.01759017, 0.05043793,
        0.06630647, 0.09602373 ],
 (10,
  5.0,
  0.0001):  [0.02593833, 0.4869793 , 0.99767602, 0.08864814, 0.05413239,
        0.1338098 , 0.10269282 ],
 (10,
  5.0,
  1e-05):  [0.03830605, 0.50201682, 0.90911918, 0.32712885, 0.06597378,
        0.35604287, 0.12302013 ],
 (10,
  5.0,
  1e-06):  [0.04570967, 0.50110507, 0.86705836, 0.19844658, 0.05352404,
        0.23166406, 0.10082414 ],
 (10,
  5.0,
  1e-07):  [0.04841037, 0.49887968, 0.87124808, 0.14102764, 0.05035552,
        0.17730592, 0.09520829 ],
 (10, 10.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (10,
  10.0,
  0.1):  [2.48411398e-02, 5.00000000e-01, 1.00000000e+00, 2.13898619e-05,
        4.96822796e-02, 4.97015969e-02, 9.46615573e-02 ],
 (10,
  10.0,
  0.01):  [2.48411398e-02, 5.00000000e-01, 1.00000000e+00, 3.08014011e-05,
        4.96827240e-02, 4.97105409e-02, 9.46623639e-02 ],
 (10,
  10.0,
  0.001):  [0.02488514, 0.50081028, 0.99975451, 0.00186947, 0.04975808,
        0.05144567, 0.09479803 ],
 (10,
  10.0,
  0.0001):  [0.02531144, 0.50173895, 0.99975451, 0.03159026, 0.05120697,
        0.07968989, 0.09742394 ],
 (10,
  10.0,
  1e-05):  [0.03458096, 0.52970075, 0.91360348, 0.34567215, 0.06802813,
        0.3738877 , 0.12662742 ],
 (10,
  10.0,
  1e-06):  [0.04386826, 0.49417808, 0.8640961 , 0.24726167, 0.05661486,
        0.27790679, 0.10626718 ],
 (10,
  10.0,
  1e-07):  [0.04785623, 0.49852839, 0.87954568, 0.13844118, 0.05066592,
        0.17526019, 0.09581259 ],
 (10, 15.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (10,
  15.0,
  0.1):  [2.48411398e-02, 5.00000000e-01, 1.00000000e+00, 2.13898619e-05,
        4.96822796e-02, 4.97015969e-02, 9.46615573e-02 ],
 (10,
  15.0,
  0.01):  [2.48411398e-02, 5.00000000e-01, 1.00000000e+00, 2.13898619e-05,
        4.96822796e-02, 4.97015969e-02, 9.46615573e-02 ],
 (10,
  15.0,
  0.001):  [2.48411398e-02, 5.00000000e-01, 1.00000000e+00, 2.72079043e-04,
        4.96941187e-02, 4.99398315e-02, 9.46830469e-02 ],
 (10,
  15.0,
  0.0001):  [0.02750935, 0.54506588, 0.99373179, 0.09949794, 0.05454426,
        0.14392461, 0.10341239 ],
 (10,
  15.0,
  1e-05):  [0.03309276, 0.55014674, 0.89821937, 0.35951053, 0.06830721,
        0.38627427, 0.12695949 ],
 (10,
  15.0,
  1e-06):  [0.04354755, 0.49800792, 0.87493044, 0.26998027, 0.05896165,
        0.30003496, 0.11047817 ],
 (10,
  15.0,
  1e-07):  [0.04752281, 0.49977292, 0.8539655 , 0.17106414, 0.05110473,
        0.20499154, 0.09643821 ],
 (10, 20.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (10, 20.0, 0.1):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (10,
  20.0,
  0.01):  [2.48411398e-02, 5.00000000e-01, 1.00000000e+00, 2.13898619e-05,
        4.96822796e-02, 4.97015969e-02, 9.46615573e-02 ],
 (10,
  20.0,
  0.001):  [2.48411398e-02, 5.00000000e-01, 1.00000000e+00, 2.08765052e-04,
        4.96911281e-02, 4.98796631e-02, 9.46776185e-02 ],
 (10,
  20.0,
  0.0001):  [0.02633641, 0.52815802, 0.99654676, 0.05977354, 0.05250098,
        0.10631362, 0.09974699 ],
 (10,
  20.0,
  1e-05):  [0.03197215, 0.5576708 , 0.98561422, 0.22217992, 0.06212899,
        0.2601083 , 0.11688975 ],
 (10,
  20.0,
  1e-06):  [0.04361094, 0.50362221, 0.88682858, 0.28309482, 0.06074163,
        0.31308908, 0.11369588 ],
 (10,
  20.0,
  1e-07):  [0.04712947, 0.49972103, 0.86591274, 0.16506129, 0.05142953,
        0.19988048, 0.0970924  ],
 (15, 0.1, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (15,
  0.1,
  0.1):  [0.42759531, 0.51594202, 0.79332919, 0.23742148, 0.05158131,
        0.26503968, 0.0968646  ],
 (15,
  0.1,
  0.01):  [0.03765492, 0.58479888, 0.78146378, 0.52915438, 0.07983941,
        0.54168943, 0.14487723 ],
 (15,
  0.1,
  0.001):  [0.18614768, 0.73980733, 0.56055448, 0.90769847, 0.24098191,
        0.89045191, 0.3370614  ],
 (15,
  0.1,
  0.0001):  [0.39841954, 0.79108089, 0.41093581, 0.97576956, 0.46995078,
        0.94770791, 0.43846644 ],
 (15,
  0.1,
  1e-05):  [0.04840996, 0.50031197, 0.90106707, 0.11138642, 0.05034248,
        0.15061876, 0.09535735 ],
 (15,
  0.1,
  1e-06):  [4.96503261e-02, 5.00091660e-01, 9.99901804e-01, 1.30905955e-04,
        4.96828142e-02, 4.98007936e-02, 9.46620877e-02 ],
 (15,
  0.1,
  1e-07):  [4.98141816e-02, 5.00092713e-01, 9.99967268e-01, 5.21912630e-05,
        4.96821885e-02, 4.97292419e-02, 9.46612452e-02 ],
 (15, 1.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (15,
  1.0,
  0.1):  [0.02780217, 0.49773207, 0.99770875, 0.01901388, 0.05048542,
        0.06763668, 0.09610766 ],
 (15,
  1.0,
  0.01):  [0.02779681, 0.49745381, 0.99772512, 0.01896511, 0.05048382,
        0.06759115, 0.09610484 ],
 (15,
  1.0,
  0.001):  [0.02644404, 0.46640487, 0.76933652, 0.45761385, 0.06903429,
        0.47310063, 0.12669954 ],
 (15,
  1.0,
  0.0001):  [0.03445146, 0.52438627, 0.92687637, 0.32321194, 0.06681303,
        0.35320275, 0.1246414  ],
 (15,
  1.0,
  1e-05):  [0.04423963, 0.49812789, 0.87776178, 0.23407696, 0.05652558,
        0.26605604, 0.10621143 ],
 (15,
  1.0,
  1e-06):  [0.04840289, 0.49979001, 0.8928022 , 0.1188609 , 0.05030579,
        0.15731128, 0.09524491 ],
 (15,
  1.0,
  1e-07):  [4.96619083e-02, 5.00062938e-01, 9.99885437e-01, 1.45451061e-04,
        4.96827283e-02, 4.98138030e-02, 9.46618583e-02 ],
 (15, 5.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (15,
  5.0,
  0.1):  [2.48408368e-02, 4.99997861e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (15,
  5.0,
  0.01):  [2.48408368e-02, 4.99997861e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (15,
  5.0,
  0.001):  [0.02484086, 0.49999786, 0.9999509 , 0.00269769, 0.04980664,
        0.0522425 , 0.09488704 ],
 (15,
  5.0,
  0.0001):  [0.02604674, 0.48589249, 0.99828156, 0.08546533, 0.05398514,
        0.1308152 , 0.10243101 ],
 (15,
  5.0,
  1e-05):  [0.0373116 , 0.48294922, 0.89540441, 0.33830206, 0.06606899,
        0.36597961, 0.12305793 ],
 (15,
  5.0,
  1e-06):  [0.0455274 , 0.49846235, 0.87543779, 0.18662398, 0.05327009,
        0.22084512, 0.1004291  ],
 (15,
  5.0,
  1e-07):  [0.04905985, 0.49991869, 0.95183464, 0.05136476, 0.04984048,
        0.09610125, 0.09472111 ],
 (15, 10.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (15,
  10.0,
  0.1):  [2.48408368e-02, 4.99997861e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (15,
  10.0,
  0.01):  [2.48408368e-02, 4.99997861e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (15,
  10.0,
  0.001):  [2.48408368e-02, 4.99997861e-01, 1.00000000e+00, 3.31115062e-04,
        4.96969076e-02, 4.99959346e-02, 9.46881090e-02 ],
 (15,
  10.0,
  0.0001):  [0.02519779, 0.49620027, 0.99862525, 0.061886  , 0.05271693,
        0.1084244 , 0.10014714 ],
 (15,
  10.0,
  1e-05):  [0.03264916, 0.50128755, 0.9508854 , 0.30414245, 0.06667516,
        0.33627346, 0.1246126  ],
 (15,
  10.0,
  1e-06):  [0.04423873, 0.49803883, 0.86838401, 0.2427852 , 0.05656258,
        0.27386574, 0.1062073  ],
 (15,
  10.0,
  1e-07):  [0.04840945, 0.49979166, 0.89517528, 0.11627957, 0.05029285,
        0.1549761 , 0.09523519 ],
 (15, 15.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (15,
  15.0,
  0.1):  [2.48408368e-02, 4.99997861e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (15,
  15.0,
  0.01):  [2.48408368e-02, 4.99997861e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (15,
  15.0,
  0.001):  [2.48408368e-02, 4.99997861e-01, 1.00000000e+00, 1.83952812e-04,
        4.96899562e-02, 4.98560835e-02, 9.46754914e-02 ],
 (15,
  15.0,
  0.0001):  [0.02484593, 0.49998545, 0.99980361, 0.02503983, 0.05088281,
        0.07346733, 0.09683729 ],
 (15,
  15.0,
  1e-05):  [0.03151827, 0.52918422, 0.96821708, 0.27700385, 0.06542931,
        0.3113442 , 0.12257532 ],
 (15,
  15.0,
  1e-06):  [0.04372787, 0.50046029, 0.88186966, 0.2644865 , 0.05898399,
        0.29515888, 0.11057234 ],
 (15,
  15.0,
  1e-07):  [0.04785288, 0.49957461, 0.89375143, 0.12422975, 0.05064965,
        0.16246057, 0.09586647 ],
 (15, 20.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (15, 20.0, 0.1):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (15,
  20.0,
  0.01):  [2.48408368e-02, 4.99997861e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (15,
  20.0,
  0.001):  [2.48408368e-02, 4.99997861e-01, 1.00000000e+00, 4.19241293e-05,
        4.96832492e-02, 4.97211110e-02, 9.46633172e-02 ],
 (15,
  20.0,
  0.0001):  [0.02484296, 0.49999316, 0.9999509 , 0.0141695 , 0.05035704,
        0.06314437, 0.09588534 ],
 (15,
  20.0,
  1e-05):  [0.0332333 , 0.57046534, 0.97748028, 0.25574147, 0.06424921,
        0.29159837, 0.12057321 ],
 (15,
  20.0,
  1e-06):  [0.04342318, 0.5035015 , 0.87414487, 0.29569431, 0.06093163,
        0.32443246, 0.11392239 ],
 (15,
  20.0,
  1e-07):  [0.04738183, 0.49944489, 0.88578115, 0.13859689, 0.05101554,
        0.17571796, 0.09647473 ],
 (20, 0.1, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (20,
  0.1,
  0.1):  [0.42756087, 0.51532551, 0.79332919, 0.23742148, 0.05158131,
        0.26503968, 0.0968646  ],
 (20,
  0.1,
  0.01):  [0.03968937, 0.58179559, 0.78049818, 0.53576984, 0.08079326,
        0.54792825, 0.14642893 ],
 (20,
  0.1,
  0.001):  [0.19077187, 0.74410472, 0.5536971 , 0.91269172, 0.24899173,
        0.89485641, 0.34351044 ],
 (20,
  0.1,
  0.0001):  [0.39526749, 0.77998356, 0.41841511, 0.97420554, 0.45887928,
        0.94659316, 0.43771401 ],
 (20,
  0.1,
  1e-05):  [0.04798339, 0.49862558, 0.92157376, 0.09206539, 0.05038998,
        0.13327642, 0.09555518 ],
 (20,
  0.1,
  1e-06):  [0.04917705, 0.49884608, 0.99615397, 0.00434556, 0.04970496,
        0.05361987, 0.09468541 ],
 (20,
  0.1,
  1e-07):  [0.04932239, 0.4989016 , 0.99602304, 0.00433016, 0.04969802,
        0.05359873, 0.09467223 ],
 (20, 1.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (20,
  1.0,
  0.1):  [0.02787799, 0.50450634, 0.9981179 , 0.01487023, 0.05030333,
        0.06371922, 0.09577954 ],
 (20,
  1.0,
  0.01):  [0.02770178, 0.50412448, 0.79509672, 0.23354478, 0.05144229,
        0.26144339, 0.09663252 ],
 (20,
  1.0,
  0.001):  [0.02629354, 0.4636549 , 0.98985303, 0.16047872, 0.05806108,
        0.20168309, 0.10968827 ],
 (20,
  1.0,
  0.0001):  [0.03535488, 0.46360543, 0.9195771 , 0.32801353, 0.06676402,
        0.35740316, 0.12448972 ],
 (20,
  1.0,
  1e-05):  [0.04397907, 0.49428713, 0.86758208, 0.24362197, 0.05657228,
        0.2746211 , 0.1062184  ],
 (20,
  1.0,
  1e-06):  [0.04804914, 0.49879222, 0.93453569, 0.07798915, 0.05032219,
        0.12054347, 0.09550187 ],
 (20,
  1.0,
  1e-07):  [0.04919276, 0.49889903, 0.99603941, 0.00447219, 0.04970553,
        0.05373451, 0.09468593 ],
 (20, 5.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (20,
  5.0,
  0.1):  [2.48411196e-02, 5.00000856e-01, 1.00000000e+00, 2.13898619e-05,
        4.96822796e-02, 4.97015969e-02, 9.46615573e-02 ],
 (20,
  5.0,
  0.01):  [2.48411196e-02, 5.00000856e-01, 1.00000000e+00, 2.13898619e-05,
        4.96822796e-02, 4.97015969e-02, 9.46615573e-02 ],
 (20,
  5.0,
  0.001):  [0.02484718, 0.50000086, 0.99998363, 0.00186862, 0.04976888,
        0.05145624, 0.09481866 ],
 (20,
  5.0,
  0.0001):  [0.02614214, 0.47366009, 0.98150633, 0.22119513, 0.06181266,
        0.25896835, 0.11630099 ],
 (20,
  5.0,
  1e-05):  [0.03852544, 0.49162626, 0.90126346, 0.33383243, 0.06605603,
        0.36202312, 0.12309044 ],
 (20,
  5.0,
  1e-06):  [0.0455475 , 0.49890142, 0.88661582, 0.17684967, 0.05330754,
        0.21211175, 0.10056843 ],
 (20,
  5.0,
  1e-07):  [0.04866281, 0.49892457, 0.93383195, 0.07274778, 0.05001617,
        0.11552753, 0.09494697 ],
 (20, 10.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (20,
  10.0,
  0.1):  [2.48411196e-02, 5.00000856e-01, 1.00000000e+00, 2.13898619e-05,
        4.96822796e-02, 4.97015969e-02, 9.46615573e-02 ],
 (20,
  10.0,
  0.01):  [2.48411196e-02, 5.00000856e-01, 1.00000000e+00, 2.13898619e-05,
        4.96822796e-02, 4.97015969e-02, 9.46615573e-02 ],
 (20,
  10.0,
  0.001):  [2.48418064e-02, 5.00000856e-01, 1.00000000e+00, 1.95931135e-04,
        4.96905219e-02, 4.98674667e-02, 9.46765183e-02 ],
 (20,
  10.0,
  0.0001):  [0.02524531, 0.49270863, 0.99924716, 0.04610799, 0.05192084,
        0.09346115, 0.09871258 ],
 (20,
  10.0,
  1e-05):  [0.03311664, 0.45893036, 0.93910183, 0.32110461, 0.06743891,
        0.35180749, 0.12584091 ],
 (20,
  10.0,
  1e-06):  [0.04484858, 0.50279463, 0.87283559, 0.23846188, 0.05653163,
        0.26997837, 0.10618585 ],
 (20,
  10.0,
  1e-07):  [0.04806849, 0.49885437, 0.9343393 , 0.0781038 , 0.05031809,
        0.12064266, 0.09549346 ],
 (20, 15.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (20,
  15.0,
  0.1):  [2.48411196e-02, 5.00000856e-01, 1.00000000e+00, 2.13898619e-05,
        4.96822796e-02, 4.97015969e-02, 9.46615573e-02 ],
 (20,
  15.0,
  0.01):  [2.48411196e-02, 5.00000856e-01, 1.00000000e+00, 2.13898619e-05,
        4.96822796e-02, 4.97015969e-02, 9.46615573e-02 ],
 (20,
  15.0,
  0.001):  [2.48411196e-02, 5.00000856e-01, 1.00000000e+00, 1.35183927e-04,
        4.96876530e-02, 4.98097375e-02, 9.46713107e-02 ],
 (20,
  15.0,
  0.0001):  [0.02532284, 0.50994436, 0.99988544, 0.02003374, 0.05063997,
        0.06871402, 0.0963978  ],
 (20,
  15.0,
  1e-05):  [0.0302051 , 0.45496817, 0.97450165, 0.26190774, 0.06456664,
        0.29731031, 0.12110907 ],
 (20,
  15.0,
  1e-06):  [0.04409695, 0.50113404, 0.86902229, 0.27540645, 0.05899968,
        0.30489804, 0.11049746 ],
 (20,
  15.0,
  1e-07):  [0.04757299, 0.49877959, 0.90409479, 0.11551295, 0.05072676,
        0.1546907 , 0.0960636  ],
 (20, 20.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (20, 20.0, 0.1):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (20,
  20.0,
  0.01):  [2.48411196e-02, 5.00000856e-01, 1.00000000e+00, 2.13898619e-05,
        4.96822796e-02, 4.97015969e-02, 9.46615573e-02 ],
 (20,
  20.0,
  0.001):  [2.48411196e-02, 5.00000856e-01, 1.00000000e+00, 2.22454564e-05,
        4.96823200e-02, 4.97024100e-02, 9.46616306e-02 ],
 (20,
  20.0,
  0.0001):  [0.02502787, 0.50392498, 0.99996727, 0.01080616, 0.05019519,
        0.05994894, 0.09559197 ],
 (20,
  20.0,
  1e-05):  [0.02865516, 0.45785623, 0.98661255, 0.21548318, 0.06168991,
        0.25379387, 0.11611924 ],
 (20,
  20.0,
  1e-06):  [0.04469962, 0.5097308 , 0.87759811, 0.29213161, 0.06086855,
        0.32121833, 0.11384128 ],
 (20,
  20.0,
  1e-07):  [0.04713724, 0.49869048, 0.9028346 , 0.12314229, 0.0510779 ,
        0.16187839, 0.0966858  ],
 (25, 0.1, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (25,
  0.1,
  0.1):  [0.42759455, 0.51592061, 0.79332919, 0.23742148, 0.05158131,
        0.26503968, 0.0968646  ],
 (25,
  0.1,
  0.01):  [0.0506042 , 0.66517229, 0.76660338, 0.63640914, 0.0992819 ,
        0.64287735, 0.17579658 ],
 (25,
  0.1,
  0.001):  [0.19684057, 0.74027643, 0.57048869, 0.91158372, 0.25223413,
        0.89463769, 0.34980607 ],
 (25,
  0.1,
  0.0001):  [0.41090472, 0.78756204, 0.41897156, 0.9764104 , 0.48146546,
        0.94871613, 0.44804985 ],
 (25,
  0.1,
  1e-05):  [0.04845929, 0.50153186, 0.9168767 , 0.09846438, 0.050484  ,
        0.13912414, 0.09569874 ],
 (25,
  0.1,
  1e-06):  [0.04967564, 0.50133783, 0.99743053, 0.00293383, 0.04969852,
        0.05234169, 0.09467949 ],
 (25,
  0.1,
  1e-07):  [0.0498473 , 0.5013358 , 0.99746326, 0.00276443, 0.04969205,
        0.05218233, 0.09466789 ],
 (25, 1.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (25,
  1.0,
  0.1):  [0.02785336, 0.4944428 , 0.99785604, 0.01762097, 0.05042451,
        0.06632029, 0.09599797 ],
 (25,
  1.0,
  0.01):  [0.0278539 , 0.49454111, 0.79445845, 0.25878995, 0.05306098,
        0.28540264, 0.09947795 ],
 (25,
  1.0,
  0.001):  [0.02657484, 0.4628261 , 0.78003993, 0.42832001, 0.06658294,
        0.4457939 , 0.12269301 ],
 (25,
  1.0,
  0.0001):  [0.03544047, 0.52173333, 0.91718765, 0.33400184, 0.06716073,
        0.36297525, 0.12515689 ],
 (25,
  1.0,
  1e-05):  [0.04453195, 0.49984222, 0.88250794, 0.2329587 , 0.05673573,
        0.26522913, 0.10661713 ],
 (25,
  1.0,
  1e-06):  [0.04845911, 0.50118041, 0.94005106, 0.07351524, 0.0503721 ,
        0.11656584, 0.09562043 ],
 (25,
  1.0,
  1e-07):  [0.049694  , 0.5013126 , 0.99751236, 0.00284827, 0.04969834,
        0.05226445, 0.09467954 ],
 (25, 5.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (25,
  5.0,
  0.1):  [2.48407964e-02, 5.00001283e-01, 1.00000000e+00, 1.96786729e-05,
        4.96821988e-02, 4.96999707e-02, 9.46614106e-02 ],
 (25,
  5.0,
  0.01):  [2.48407964e-02, 5.00001283e-01, 1.00000000e+00, 1.96786729e-05,
        4.96821988e-02, 4.96999707e-02, 9.46614106e-02 ],
 (25,
  5.0,
  0.001):  [0.0248408 , 0.50000128, 1.        , 0.00174285, 0.04976369,
        0.05133753, 0.09480932 ],
 (25,
  5.0,
  0.0001):  [0.02654951, 0.51262544, 0.99283166, 0.14515075, 0.05724137,
        0.18726461, 0.10824207 ],
 (25,
  5.0,
  1e-05):  [0.04491429, 0.52302749, 0.89221302, 0.34213683, 0.06620744,
        0.36946531, 0.12326769 ],
 (25,
  5.0,
  1e-06):  [0.04566146, 0.49983892, 0.87637066, 0.18920531, 0.05348451,
        0.22334455, 0.10081626 ],
 (25,
  5.0,
  1e-07):  [0.04910267, 0.50135179, 0.93726883, 0.07051296, 0.05007638,
        0.1135745 , 0.09507319 ],
 (25, 10.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (25,
  10.0,
  0.1):  [2.48407964e-02, 5.00001283e-01, 1.00000000e+00, 1.96786729e-05,
        4.96821988e-02, 4.96999707e-02, 9.46614106e-02 ],
 (25,
  10.0,
  0.01):  [2.48407964e-02, 5.00001283e-01, 1.00000000e+00, 1.96786729e-05,
        4.96821988e-02, 4.96999707e-02, 9.46614106e-02 ],
 (25,
  10.0,
  0.001):  [2.48407964e-02, 5.00001283e-01, 1.00000000e+00, 1.90797568e-04,
        4.96902795e-02, 4.98625882e-02, 9.46760782e-02 ],
 (25,
  10.0,
  0.0001):  [0.02522067, 0.49848157, 0.99973814, 0.03172544, 0.05121296,
        0.07981754, 0.0974347  ],
 (25,
  10.0,
  1e-05):  [0.03559411, 0.53936191, 0.92671271, 0.33671835, 0.06806972,
        0.36603002, 0.12682386 ],
 (25,
  10.0,
  1e-06):  [0.0445134 , 0.49958032, 0.86365422, 0.2500261 , 0.05678429,
        0.28051192, 0.10656223 ],
 (25,
  10.0,
  1e-07):  [0.04847633, 0.50123952, 0.93882361, 0.07467115, 0.05036931,
        0.11760334, 0.09560906 ],
 (25, 15.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (25,
  15.0,
  0.1):  [2.48407964e-02, 5.00001283e-01, 1.00000000e+00, 1.96786729e-05,
        4.96821988e-02, 4.96999707e-02, 9.46614106e-02 ],
 (25,
  15.0,
  0.01):  [2.48407964e-02, 5.00001283e-01, 1.00000000e+00, 1.96786729e-05,
        4.96821988e-02, 4.96999707e-02, 9.46614106e-02 ],
 (25,
  15.0,
  0.001):  [2.48407964e-02, 5.00001283e-01, 1.00000000e+00, 1.16360849e-04,
        4.96867641e-02, 4.97918496e-02, 9.46696972e-02 ],
 (25,
  15.0,
  0.0001):  [0.02487313, 0.50067203, 0.99985271, 0.02300009, 0.05078434,
        0.07153137, 0.09665918 ],
 (25,
  15.0,
  1e-05):  [0.03197935, 0.53101063, 0.95808648, 0.30358802, 0.06709632,
        0.33610434, 0.12540997 ],
 (25,
  15.0,
  1e-06):  [0.04368304, 0.49716222, 0.86807306, 0.27958175, 0.05926038,
        0.30881875, 0.1109468  ],
 (25,
  15.0,
  1e-07):  [0.04791947, 0.50103125, 0.92694183, 0.09327263, 0.05073267,
        0.13469038, 0.09620018 ],
 (25, 20.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (25, 20.0, 0.1):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (25,
  20.0,
  0.01):  [2.48407964e-02, 5.00001283e-01, 1.00000000e+00, 1.96786729e-05,
        4.96821988e-02, 4.96999707e-02, 9.46614106e-02 ],
 (25,
  20.0,
  0.001):  [2.48407964e-02, 5.00001283e-01, 1.00000000e+00, 1.96786729e-05,
        4.96821988e-02, 4.96999707e-02, 9.46614106e-02 ],
 (25,
  20.0,
  0.0001):  [0.02488482, 0.5017124 , 0.99998363, 0.01031419, 0.05017227,
        0.05948223, 0.09555049 ],
 (25,
  20.0,
  1e-05):  [0.02862741, 0.47098825, 0.9870708 , 0.21130959, 0.06141026,
        0.24985039, 0.11562684 ],
 (25,
  20.0,
  1e-06):  [0.04294201, 0.49562053, 0.87496318, 0.29714454, 0.06110334,
        0.3258513 , 0.11422943 ],
 (25,
  20.0,
  1e-07):  [0.04744426, 0.50076511, 0.87208275, 0.15620931, 0.05126168,
        0.19177481, 0.09683152 ],
 (30, 0.1, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (30,
  0.1,
  0.1):  [0.42756931, 0.5154718 , 0.79332919, 0.23742148, 0.05158131,
        0.26503968, 0.0968646  ],
 (30,
  0.1,
  0.01):  [0.0378868 , 0.59433872, 0.16406992, 0.94432818, 0.13350113,
        0.90556396, 0.14721539 ],
 (30,
  0.1,
  0.001):  [0.2047343 , 0.74629595, 0.52520376, 0.92519024, 0.26848327,
        0.90531841, 0.355325   ],
 (30,
  0.1,
  0.0001):  [0.39471826, 0.77899404, 0.40290007, 0.97669532, 0.47473774,
        0.94818844, 0.43587882 ],
 (30,
  0.1,
  1e-05):  [0.04822566, 0.49967267, 0.92461785, 0.08898696, 0.05038581,
        0.13050216, 0.09556399 ],
 (30,
  0.1,
  1e-06):  [0.04954005, 0.49999244, 0.99459919, 0.00664369, 0.04974034,
        0.05572657, 0.09474256 ],
 (30,
  0.1,
  1e-07):  [0.04968801, 0.49996748, 0.99487742, 0.006079  , 0.0497267 ,
        0.05520376, 0.09471908 ],
 (30, 1.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (30,
  1.0,
  0.1):  [0.02787922, 0.49676788, 0.99813427, 0.01498745, 0.0503098 ,
        0.06383143, 0.09579134 ],
 (30,
  1.0,
  0.01):  [0.02800107, 0.49707055, 0.9980688 , 0.01541097, 0.05032721,
        0.06423066, 0.09582261 ],
 (30,
  1.0,
  0.001):  [0.02646565, 0.49031425, 0.77855062, 0.42569675, 0.06618077,
        0.44322698, 0.12199162 ],
 (30,
  1.0,
  0.0001):  [0.03630081, 0.53580245, 0.89668096, 0.34399604, 0.06669288,
        0.37145413, 0.12415167 ],
 (30,
  1.0,
  1e-05):  [0.0441443 , 0.49596455, 0.88126412, 0.23266523, 0.05663985,
        0.26488844, 0.10643877 ],
 (30,
  1.0,
  1e-06):  [0.04832911, 0.50003432, 0.91160682, 0.1022752 , 0.05041079,
        0.14248382, 0.09553842 ],
 (30,
  1.0,
  1e-07):  [0.04955506, 0.49999461, 0.99469739, 0.00650337, 0.04973833,
        0.05559811, 0.09473936 ],
 (30, 5.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (30,
  5.0,
  0.1):  [2.48409984e-02, 4.99998717e-01, 1.00000000e+00, 1.88230785e-05,
        4.96821584e-02, 4.96991576e-02, 9.46613373e-02 ],
 (30,
  5.0,
  0.01):  [2.48409984e-02, 4.99998717e-01, 1.00000000e+00, 1.88230785e-05,
        4.96821584e-02, 4.96991576e-02, 9.46613373e-02 ],
 (30,
  5.0,
  0.001):  [0.02484189, 0.49999872, 1.        , 0.00157258, 0.04975563,
        0.05117572, 0.09479469 ],
 (30,
  5.0,
  0.0001):  [0.02598045, 0.49497346, 0.99206245, 0.15822936, 0.05803669,
        0.19965525, 0.10965826 ],
 (30,
  5.0,
  1e-05):  [0.0392665 , 0.48575815, 0.89961049, 0.33527068, 0.06607612,
        0.3633078 , 0.12310986 ],
 (30,
  5.0,
  1e-06):  [0.04571744, 0.50046545, 0.87990573, 0.18533203, 0.05344706,
        0.21983933, 0.10077299 ],
 (30,
  5.0,
  1e-07):  [0.04896123, 0.50004184, 0.95188374, 0.05379636, 0.04996461,
        0.09841448, 0.0949455  ],
 (30, 10.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (30,
  10.0,
  0.1):  [2.48409984e-02, 4.99998717e-01, 1.00000000e+00, 1.88230785e-05,
        4.96821584e-02, 4.96991576e-02, 9.46613373e-02 ],
 (30,
  10.0,
  0.01):  [2.48409984e-02, 4.99998717e-01, 1.00000000e+00, 1.88230785e-05,
        4.96821584e-02, 4.96991576e-02, 9.46613373e-02 ],
 (30,
  10.0,
  0.001):  [2.48409984e-02, 4.99998717e-01, 1.00000000e+00, 1.72830084e-04,
        4.96894309e-02, 4.98455134e-02, 9.46745379e-02 ],
 (30,
  10.0,
  0.0001):  [0.02491118, 0.49064071, 0.99959085, 0.03623443, 0.05143305,
        0.0840952 , 0.09783222 ],
 (30,
  10.0,
  1e-05):  [0.03591864, 0.54358149, 0.95029623, 0.30535226, 0.06674491,
        0.33739389, 0.12472934 ],
 (30,
  10.0,
  1e-06):  [0.04422589, 0.49677201, 0.87558509, 0.23814103, 0.05667709,
        0.26981006, 0.10646279 ],
 (30,
  10.0,
  1e-07):  [0.04834692, 0.50007442, 0.91159046, 0.10227177, 0.05040975,
        0.14247975, 0.09553646 ],
 (30, 15.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (30,
  15.0,
  0.1):  [2.48409984e-02, 4.99998717e-01, 1.00000000e+00, 1.88230785e-05,
        4.96821584e-02, 4.96991576e-02, 9.46613373e-02 ],
 (30,
  15.0,
  0.01):  [2.48409984e-02, 4.99998717e-01, 1.00000000e+00, 1.88230785e-05,
        4.96821584e-02, 4.96991576e-02, 9.46613373e-02 ],
 (30,
  15.0,
  0.001):  [2.48409984e-02, 4.99998717e-01, 1.00000000e+00, 9.66821757e-05,
        4.96858348e-02, 4.97731486e-02, 9.46680105e-02 ],
 (30,
  15.0,
  0.0001):  [0.02486664, 0.49998075, 0.99988544, 0.01878201, 0.05057864,
        0.06752447, 0.09628667 ],
 (30,
  15.0,
  1e-05):  [0.03039267, 0.49522269, 0.96715328, 0.28039628, 0.06565002,
        0.31451524, 0.12295397 ],
 (30,
  15.0,
  1e-06):  [0.04462952, 0.5055778 , 0.86975876, 0.27730074, 0.05919233,
        0.3067348 , 0.11084125 ],
 (30,
  15.0,
  1e-07):  [0.04780498, 0.49997509, 0.91000295, 0.11019287, 0.05075165,
        0.14992845, 0.09614141 ],
 (30, 20.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (30, 20.0, 0.1):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (30,
  20.0,
  0.01):  [2.48409984e-02, 4.99998717e-01, 1.00000000e+00, 1.88230785e-05,
        4.96821584e-02, 4.96991576e-02, 9.46613373e-02 ],
 (30,
  20.0,
  0.001):  [2.48409984e-02, 4.99998717e-01, 1.00000000e+00, 1.88230785e-05,
        4.96821584e-02, 4.96991576e-02, 9.46613373e-02 ],
 (30,
  20.0,
  0.0001):  [0.02485371, 0.49999059, 1.        , 0.00969816, 0.05014341,
        0.05889762, 0.09549821 ],
 (30,
  20.0,
  1e-05):  [0.02893945, 0.42758519, 0.98746359, 0.21143707, 0.06144252,
        0.24999106, 0.11568671 ],
 (30,
  20.0,
  1e-06):  [0.04462171, 0.50912129, 0.87263919, 0.29841424, 0.06105451,
        0.32694247, 0.11412427 ],
 (30,
  20.0,
  1e-07):  [0.04732913, 0.49975088, 0.87457039, 0.15325152, 0.05123004,
        0.18908755, 0.09679035 ],
 (35, 0.1, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (35,
  0.1,
  0.1):  [0.42757347, 0.51553273, 0.79332919, 0.23742148, 0.05158131,
        0.26503968, 0.0968646  ],
 (35,
  0.1,
  0.01):  [0.03756062, 0.58491504, 0.7789925 , 0.54635696, 0.08237713,
        0.55791459, 0.14899798 ],
 (35,
  0.1,
  0.001):  [0.20531303, 0.7431567 , 0.5562993 , 0.91876387, 0.26362283,
        0.90075617, 0.35772469 ],
 (35,
  0.1,
  0.0001):  [0.39916638, 0.78438096, 0.40694249, 0.97648484, 0.4749852 ,
        0.94818925, 0.43833902 ],
 (35,
  0.1,
  1e-05):  [0.04838969, 0.49993195, 0.92754738, 0.08614211, 0.05038799,
        0.12794419, 0.09558351 ],
 (35,
  0.1,
  1e-06):  [0.04973888, 0.50021151, 0.99697228, 0.00354815, 0.04970593,
        0.05290272, 0.09469086 ],
 (35,
  0.1,
  1e-07):  [0.04990949, 0.50025824, 0.99867435, 0.00163761, 0.04969602,
        0.05117166, 0.09468055 ],
 (35, 1.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (35,
  1.0,
  0.1):  [0.02812278, 0.49728054, 0.99779058, 0.01825839, 0.05045246,
        0.06692279, 0.09604831 ],
 (35,
  1.0,
  0.01):  [0.02804991, 0.4970214 , 0.99779058, 0.01822502, 0.05045083,
        0.06689108, 0.09604536 ],
 (35,
  1.0,
  0.001):  [0.02650163, 0.47603506, 0.7799581 , 0.41809223, 0.06548288,
        0.43607019, 0.12082192 ],
 (35,
  1.0,
  0.0001):  [0.03492483, 0.46666298, 0.90095251, 0.34136166, 0.06673925,
        0.36916285, 0.12427282 ],
 (35,
  1.0,
  1e-05):  [0.04475659, 0.50173045, 0.87898923, 0.2355289 , 0.05670156,
        0.26749683, 0.10653105 ],
 (35,
  1.0,
  1e-06):  [0.04849874, 0.50034731, 0.93129521, 0.08197878, 0.05036345,
        0.1241739 , 0.09555916 ],
 (35,
  1.0,
  1e-07):  [0.04975692, 0.50024665, 0.99697228, 0.00353959, 0.04970552,
        0.05289459, 0.09469013 ],
 (35, 5.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (35,
  5.0,
  0.1):  [2.48407560e-02, 4.99998717e-01, 1.00000000e+00, 1.28339171e-05,
        4.96818757e-02, 4.96934660e-02, 9.46608240e-02 ],
 (35,
  5.0,
  0.01):  [2.48407560e-02, 4.99998717e-01, 1.00000000e+00, 1.28339171e-05,
        4.96818757e-02, 4.96934660e-02, 9.46608240e-02 ],
 (35,
  5.0,
  0.001):  [0.02484076, 0.49999872, 0.99998363, 0.00266774, 0.04980677,
        0.05221566, 0.09488742 ],
 (35,
  5.0,
  0.0001):  [0.02645535, 0.50548625, 0.99759419, 0.09655555, 0.05457609,
        0.14132029, 0.10349046 ],
 (35,
  5.0,
  1e-05):  [0.03983075, 0.51499578, 0.90049426, 0.33589013, 0.06619435,
        0.36394038, 0.12332333 ],
 (35,
  5.0,
  1e-06):  [0.04581298, 0.50089352, 0.86799123, 0.19698951, 0.05348652,
        0.23032572, 0.10076386 ],
 (35,
  5.0,
  1e-07):  [0.0491463 , 0.50023323, 0.958512  , 0.04718005, 0.04996323,
        0.09245617, 0.09497577 ],
 (35, 10.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (35,
  10.0,
  0.1):  [2.48407560e-02, 4.99998717e-01, 1.00000000e+00, 1.28339171e-05,
        4.96818757e-02, 4.96934660e-02, 9.46608240e-02 ],
 (35,
  10.0,
  0.01):  [2.48407560e-02, 4.99998717e-01, 1.00000000e+00, 1.28339171e-05,
        4.96818757e-02, 4.96934660e-02, 9.46608240e-02 ],
 (35,
  10.0,
  0.001):  [2.48407560e-02, 4.99998717e-01, 1.00000000e+00, 2.85768555e-04,
        4.96947654e-02, 4.99528409e-02, 9.46842206e-02 ],
 (35,
  10.0,
  0.0001):  [0.0252711 , 0.49758801, 0.99929626, 0.04518052, 0.05187544,
        0.0925822 , 0.09863076 ],
 (35,
  10.0,
  1e-05):  [0.03327486, 0.45705035, 0.94623744, 0.31275144, 0.06714646,
        0.34422383, 0.12539473 ],
 (35,
  10.0,
  1e-06):  [0.04475983, 0.50181964, 0.87064253, 0.24345513, 0.05674871,
        0.2746146 , 0.10655231 ],
 (35,
  10.0,
  1e-07):  [0.04851611, 0.50040034, 0.92681091, 0.08664605, 0.05037638,
        0.12838651, 0.09555871 ],
 (35, 15.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (35,
  15.0,
  0.1):  [2.48407560e-02, 4.99998717e-01, 1.00000000e+00, 1.28339171e-05,
        4.96818757e-02, 4.96934660e-02, 9.46608240e-02 ],
 (35,
  15.0,
  0.01):  [2.48407560e-02, 4.99998717e-01, 1.00000000e+00, 1.28339171e-05,
        4.96818757e-02, 4.96934660e-02, 9.46608240e-02 ],
 (35,
  15.0,
  0.001):  [2.48407560e-02, 4.99998717e-01, 1.00000000e+00, 1.60851761e-04,
        4.96888652e-02, 4.98341302e-02, 9.46735110e-02 ],
 (35,
  15.0,
  0.0001):  [0.02491891, 0.49457824, 0.9999018 , 0.02403365, 0.05083775,
        0.07251602, 0.09675616 ],
 (35,
  15.0,
  1e-05):  [0.03048464, 0.48182592, 0.97471441, 0.26230559, 0.0646124 ,
        0.29769896, 0.12119122 ],
 (35,
  15.0,
  1e-06):  [0.04345521, 0.49467152, 0.87475042, 0.27339923, 0.0592112 ,
        0.30327512, 0.11091466 ],
 (35,
  15.0,
  1e-07):  [0.04795417, 0.50031607, 0.93088606, 0.0887722 , 0.05069872,
        0.13060949, 0.09616028 ],
 (35, 20.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (35, 20.0, 0.1):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (35,
  20.0,
  0.01):  [2.48407560e-02, 4.99998717e-01, 1.00000000e+00, 1.28339171e-05,
        4.96818757e-02, 4.96934660e-02, 9.46608240e-02 ],
 (35,
  20.0,
  0.001):  [2.48407560e-02, 4.99998717e-01, 1.00000000e+00, 4.36353183e-05,
        4.96833300e-02, 4.97227372e-02, 9.46634638e-02 ],
 (35,
  20.0,
  0.0001):  [0.02490524, 0.50065538, 0.99996727, 0.01372716, 0.05033637,
        0.06272482, 0.09584795 ],
 (35,
  20.0,
  1e-05):  [0.0306757 , 0.53278587, 0.98743085, 0.21327489, 0.0615753 ,
        0.25173594, 0.11592181 ],
 (35,
  20.0,
  1e-06):  [0.0441899 , 0.50660081, 0.87578148, 0.29607505, 0.06106975,
        0.3248756 , 0.11417769 ],
 (35,
  20.0,
  1e-07):  [0.04750188, 0.50035976, 0.87795817, 0.14943385, 0.05119931,
        0.18562787, 0.09675616 ],
 (40, 0.1, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (40,
  0.1,
  0.1):  [0.427568  , 0.51545443, 0.79332919, 0.23742148, 0.05158131,
        0.26503968, 0.0968646  ],
 (40,
  0.1,
  0.01):  [0.05481556, 0.5809378 , 0.77184053, 0.60362447, 0.09239356,
        0.61198166, 0.1650319  ],
 (40,
  0.1,
  0.001):  [0.21148374, 0.74921036, 0.5582305 , 0.92151974, 0.27106131,
        0.90347107, 0.36492508 ],
 (40,
  0.1,
  0.0001):  [0.41157313, 0.79353608, 0.42018265, 0.97575331, 0.47533001,
        0.94815185, 0.44605829 ],
 (40,
  0.1,
  1e-05):  [0.04847867, 0.5004543 , 0.91190141, 0.10115693, 0.05036668,
        0.14143575, 0.09546082 ],
 (40,
  0.1,
  1e-06):  [0.04978138, 0.5002183 , 0.99322444, 0.0077226 , 0.04972633,
        0.05668358, 0.0947109  ],
 (40,
  0.1,
  1e-07):  [0.04994536, 0.50021648, 0.99559753, 0.00498983, 0.04970914,
        0.05420448, 0.09469048 ],
 (40, 1.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (40,
  1.0,
  0.1):  [0.02787535, 0.50227132, 0.99808517, 0.01596026, 0.05035467,
        0.06475347, 0.09587246 ],
 (40,
  1.0,
  0.01):  [0.02787204, 0.50224061, 0.99805244, 0.01603127, 0.05035656,
        0.06481933, 0.09587572 ],
 (40,
  1.0,
  0.001):  [0.02943625, 0.56355113, 0.76116985, 0.46512511, 0.06924496,
        0.47983299, 0.1269418  ],
 (40,
  1.0,
  0.0001):  [0.03786231, 0.54120705, 0.92995319, 0.32016773, 0.06673988,
        0.35046265, 0.12454178 ],
 (40,
  1.0,
  1e-05):  [0.04419576, 0.49629632, 0.87011882, 0.24140684, 0.05657203,
        0.27264205, 0.10623692 ],
 (40,
  1.0,
  1e-06):  [0.04852194, 0.50030712, 0.88781055, 0.12560555, 0.05040512,
        0.16347286, 0.09539427 ],
 (40,
  1.0,
  1e-07):  [0.04979702, 0.50023047, 0.99292985, 0.00802804, 0.04972686,
        0.05695922, 0.09471053 ],
 (40, 5.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (40,
  5.0,
  0.1):  [2.48407560e-02, 5.00000428e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (40,
  5.0,
  0.01):  [2.48407560e-02, 5.00000428e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (40,
  5.0,
  0.001):  [0.02484076, 0.50000043, 0.99998363, 0.00204658, 0.04977731,
        0.05162536, 0.09483396 ],
 (40,
  5.0,
  0.0001):  [0.0266873 , 0.51466319, 0.9982161 , 0.08481166, 0.05394531,
        0.13019075, 0.10235897 ],
 (40,
  5.0,
  1e-05):  [0.03902847, 0.49027385, 0.89211482, 0.34121022, 0.06611368,
        0.36857986, 0.12310423 ],
 (40,
  5.0,
  1e-06):  [0.0458867 , 0.50161803, 0.86232857, 0.20168757, 0.05345223,
        0.23450906, 0.10066467 ],
 (40,
  5.0,
  1e-07):  [0.04918425, 0.50027294, 0.98399398, 0.01941429, 0.04984534,
        0.06733584, 0.09488422 ],
 (40, 10.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (40,
  10.0,
  0.1):  [2.48407560e-02, 5.00000428e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (40,
  10.0,
  0.01):  [2.48407560e-02, 5.00000428e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (40,
  10.0,
  0.001):  [2.48407560e-02, 5.00000428e-01, 1.00000000e+00, 2.32721697e-04,
        4.96922596e-02, 4.99024295e-02, 9.46796724e-02 ],
 (40,
  10.0,
  0.0001):  [0.02524504, 0.49335743, 0.99862525, 0.05657875, 0.05243592,
        0.10338082, 0.09963993 ],
 (40,
  10.0,
  1e-05):  [0.03362066, 0.4522596 , 0.91286701, 0.34806268, 0.06820928,
        0.37612287, 0.12693406 ],
 (40,
  10.0,
  1e-06):  [0.04409257, 0.49530719, 0.87016792, 0.24121091, 0.05656126,
        0.27245829, 0.10621829 ],
 (40,
  10.0,
  1e-07):  [0.04853937, 0.50035207, 0.89591175, 0.11712489, 0.05037791,
        0.15581601, 0.09539186 ],
 (40, 15.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (40,
  15.0,
  0.1):  [2.48407560e-02, 5.00000428e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (40,
  15.0,
  0.01):  [2.48407560e-02, 5.00000428e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (40,
  15.0,
  0.001):  [2.48407560e-02, 5.00000428e-01, 1.00000000e+00, 1.45451061e-04,
        4.96881379e-02, 4.98194946e-02, 9.46721908e-02 ],
 (40,
  15.0,
  0.0001):  [0.02488772, 0.49999658, 0.99959085, 0.02915609, 0.05107721,
        0.07736852, 0.09718828 ],
 (40,
  15.0,
  1e-05):  [0.03025122, 0.46582999, 0.88414455, 0.36649389, 0.06800042,
        0.39221144, 0.12628791 ],
 (40,
  15.0,
  1e-06):  [0.04438481, 0.50352959, 0.87815456, 0.26801583, 0.05901666,
        0.29832829, 0.11060038 ],
 (40,
  15.0,
  1e-07):  [0.04798054, 0.50033204, 0.89761383, 0.12210531, 0.05074058,
        0.16063356, 0.09605153 ],
 (40, 20.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (40, 20.0, 0.1):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (40,
  20.0,
  0.01):  [2.48407560e-02, 5.00000428e-01, 1.00000000e+00, 1.71118895e-05,
        4.96820776e-02, 4.96975315e-02, 9.46611906e-02 ],
 (40,
  20.0,
  0.001):  [2.48407560e-02, 5.00000428e-01, 1.00000000e+00, 2.31010508e-05,
        4.96823604e-02, 4.97032231e-02, 9.46617039e-02 ],
 (40,
  20.0,
  0.0001):  [0.02484066, 0.50000257, 0.9999509 , 0.01256013, 0.05027909,
        0.06161495, 0.09574402 ],
 (40,
  20.0,
  1e-05):  [0.02907384, 0.49360651, 0.98599064, 0.21978596, 0.06197236,
        0.25785199, 0.11661513 ],
 (40,
  20.0,
  1e-06):  [0.04307699, 0.49654813, 0.8799712 , 0.29049229, 0.06089069,
        0.31977835, 0.11389993 ],
 (40,
  20.0,
  1e-07):  [0.04751293, 0.5003017 , 0.8947334 , 0.13151685, 0.05110616,
        0.16943442, 0.09668953 ],
 (50, 0.1, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (50,
  0.1,
  0.1):  [0.42756737, 0.51543204, 0.79332919, 0.23742148, 0.05158131,
        0.26503968, 0.0968646  ],
 (50,
  0.1,
  0.01):  [0.04975968, 0.66159035, 0.76825636, 0.62756743, 0.09734299,
        0.63455703, 0.17279211 ],
 (50,
  0.1,
  0.001):  [0.22232549, 0.7494225 , 0.49939446, 0.94044121, 0.30475905,
        0.91852945, 0.37852221 ],
 (50,
  0.1,
  0.0001):  [0.38357882, 0.77937231, 0.39029819, 0.97653275, 0.46509088,
        0.94740788, 0.42442471 ],
 (50,
  0.1,
  1e-05):  [0.04785588, 0.49779774, 0.89180387, 0.1233579 , 0.05049713,
        0.16153527, 0.09558206 ],
 (50,
  0.1,
  1e-06):  [0.04920118, 0.49862193, 0.99168603, 0.0093679 , 0.0497315 ,
        0.05817072, 0.09471327 ],
 (50,
  0.1,
  1e-07):  [0.04936984, 0.49871258, 0.9917024 , 0.00905048, 0.04971714,
        0.05786987, 0.09468731 ],
 (50, 1.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (50,
  1.0,
  0.1):  [0.02823237, 0.49902347, 0.99793787, 0.01747381, 0.05042127,
        0.06618451, 0.09599247 ],
 (50,
  1.0,
  0.01):  [0.02827362, 0.49855893, 0.99795424, 0.0174199 , 0.05041942,
        0.06613409, 0.09598921 ],
 (50,
  1.0,
  0.001):  [0.02634044, 0.49703433, 0.78625904, 0.37598329, 0.06179997,
        0.39636631, 0.11459294 ],
 (50,
  1.0,
  0.0001):  [0.03585649, 0.46203728, 0.93465026, 0.31463118, 0.06654874,
        0.34543451, 0.12425062 ],
 (50,
  1.0,
  1e-05):  [0.0440814 , 0.49543617, 0.87132991, 0.2413478 , 0.05664215,
        0.27264611, 0.10636959 ],
 (50,
  1.0,
  1e-06):  [0.04800896, 0.4983843 , 0.88767962, 0.12727396, 0.05048955,
        0.16505187, 0.09554469 ],
 (50,
  1.0,
  1e-07):  [0.04922719, 0.49869557, 0.99162057, 0.00940726, 0.04973026,
        0.05820487, 0.09471072 ],
 (50, 5.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (50,
  5.0,
  0.1):  [2.48406551e-02, 5.00000428e-01, 1.00000000e+00, 3.76461569e-05,
        4.96830472e-02, 4.97170456e-02, 9.46629505e-02 ],
 (50,
  5.0,
  0.01):  [2.48406551e-02, 5.00000428e-01, 1.00000000e+00, 2.05342674e-04,
        4.96909665e-02, 4.98764107e-02, 9.46773251e-02 ],
 (50,
  5.0,
  0.001):  [0.02484066, 0.50000043, 0.99615397, 0.0467899 , 0.05180355,
        0.09395551, 0.09848551 ],
 (50,
  5.0,
  0.0001):  [0.02716822, 0.52649699, 0.99829793, 0.08596928, 0.05401413,
        0.13129492, 0.10248329 ],
 (50,
  5.0,
  1e-05):  [0.03879661, 0.49034425, 0.90142712, 0.33289213, 0.06598026,
        0.36113767, 0.12296039 ],
 (50,
  5.0,
  1e-06):  [0.04540774, 0.49782603, 0.88566659, 0.17962864, 0.05342426,
        0.2147055 , 0.10076998 ],
 (50,
  5.0,
  1e-07):  [0.04865061, 0.49859879, 0.94271873, 0.06401387, 0.05002075,
        0.10766904, 0.09500076 ],
 (50, 10.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (50,
  10.0,
  0.1):  [2.48406551e-02, 5.00000428e-01, 1.00000000e+00, 1.88230785e-05,
        4.96821584e-02, 4.96991576e-02, 9.46613373e-02 ],
 (50,
  10.0,
  0.01):  [2.48406551e-02, 5.00000428e-01, 1.00000000e+00, 1.93364351e-04,
        4.96904007e-02, 4.98650275e-02, 9.46762982e-02 ],
 (50,
  10.0,
  0.001):  [0.02484066, 0.50000043, 0.99883801, 0.00673524, 0.04994611,
        0.05602416, 0.09513507 ],
 (50,
  10.0,
  0.0001):  [0.02520433, 0.50072473, 0.99968904, 0.03627378, 0.05143983,
        0.08413748, 0.09784497 ],
 (50,
  10.0,
  1e-05):  [0.03285798, 0.47718789, 0.9491997 , 0.30620272, 0.0667493 ,
        0.33814762, 0.12472755 ],
 (50,
  10.0,
  1e-06):  [0.0440487 , 0.4951043 , 0.86553632, 0.24682874, 0.05667312,
        0.27756692, 0.10638071 ],
 (50,
  10.0,
  1e-07):  [0.0480306 , 0.49842827, 0.88741776, 0.12743224, 0.0504841 ,
        0.16518929, 0.09553342 ],
 (50, 15.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (50,
  15.0,
  0.1):  [2.48406551e-02, 5.00000428e-01, 1.00000000e+00, 1.88230785e-05,
        4.96821584e-02, 4.96991576e-02, 9.46613373e-02 ],
 (50,
  15.0,
  0.01):  [2.48406551e-02, 5.00000428e-01, 1.00000000e+00, 1.82241623e-04,
        4.96898754e-02, 4.98544573e-02, 9.46753447e-02 ],
 (50,
  15.0,
  0.001):  [0.02484066, 0.50000043, 0.99947629, 0.00246668, 0.04977322,
        0.05199938, 0.09482426 ],
 (50,
  15.0,
  0.0001):  [0.02484066, 0.50000043, 0.98469772, 0.18118582, 0.05915085,
        0.22110531, 0.111598   ],
 (50,
  15.0,
  1e-05):  [0.03004551, 0.46036715, 0.96315996, 0.28858517, 0.06609969,
        0.3220989 , 0.12370945 ],
 (50,
  15.0,
  1e-06):  [0.04400538, 0.50009386, 0.86537266, 0.28033382, 0.05914502,
        0.30939929, 0.11072256 ],
 (50,
  15.0,
  1e-07):  [0.0475566 , 0.49874324, 0.88247521, 0.13923346, 0.0508705 ,
        0.17615865, 0.09619577 ],
 (50, 20.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (50, 20.0, 0.1):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (50,
  20.0,
  0.01):  [2.48406551e-02, 5.00000428e-01, 1.00000000e+00, 1.63418545e-04,
        4.96889864e-02, 4.98365694e-02, 9.46737311e-02 ],
 (50,
  20.0,
  0.001):  [2.48406551e-02, 5.00000428e-01, 1.00000000e+00, 2.91757716e-04,
        4.96950483e-02, 4.99585325e-02, 9.46847342e-02 ],
 (50,
  20.0,
  0.0001):  [0.02484066, 0.50000043, 0.99031128, 0.13103772, 0.05622906,
        0.17372752, 0.10641591 ],
 (50,
  20.0,
  1e-05):  [0.02879689, 0.43627502, 0.98491048, 0.22275231, 0.06213027,
        0.2606173 , 0.11688705 ],
 (50,
  20.0,
  1e-06):  [0.04515138, 0.51292251, 0.8733102 , 0.29676637, 0.06096414,
        0.3254098 , 0.11397211 ],
 (50,
  20.0,
  1e-07):  [0.04714487, 0.49890625, 0.88386632, 0.14456039, 0.0512476 ,
        0.18129004, 0.09687809 ],
 (100, 0.1, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (100,
  0.1,
  0.1):  [0.42756238, 0.51535586, 0.79331282, 0.23803665, 0.05161979,
        0.26562348, 0.09693233 ],
 (100,
  0.1,
  0.01):  [0.06357168, 0.58658531, 0.76521227, 0.64123555, 0.10031926,
        0.64739487, 0.17738356 ],
 (100,
  0.1,
  0.001):  [0.24099682, 0.75221053, 0.4908841 , 0.95052525, 0.3415434 ,
        0.92768969, 0.4028176  ],
 (100,
  0.1,
  0.0001):  [0.41988588, 0.79709034, 0.43545219, 0.97521514, 0.47875843,
        0.94839903, 0.4560796  ],
 (100,
  0.1,
  1e-05):  [0.04870893, 0.50326709, 0.88589572, 0.13220903, 0.05066526,
        0.16965314, 0.09584882 ],
 (100,
  0.1,
  1e-06):  [0.04983326, 0.50198602, 0.97165396, 0.02998516, 0.04976104,
        0.07676847, 0.09467359 ],
 (100,
  0.1,
  1e-07):  [4.99899963e-02, 5.01896340e-01, 9.99950902e-01, 9.92489592e-05,
        4.96836377e-02, 4.97731486e-02, 9.46638024e-02 ],
 (100, 1.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (100,
  1.0,
  0.1):  [0.02788692, 0.50018495, 0.99765965, 0.01867934, 0.05046672,
        0.06731632, 0.09607354 ],
 (100,
  1.0,
  0.01):  [0.02798656, 0.50092306, 0.79465484, 0.23988901, 0.05182207,
        0.26745048, 0.09729895 ],
 (100,
  1.0,
  0.001):  [0.02679647, 0.51172501, 0.78267487, 0.41459456, 0.06532911,
        0.43288126, 0.12059249 ],
 (100,
  1.0,
  0.0001):  [0.03410313, 0.48165012, 0.88974174, 0.35425376, 0.06719202,
        0.38085748, 0.12494814 ],
 (100,
  1.0,
  1e-05):  [0.04463899, 0.50097261, 0.88124775, 0.23585745, 0.05686202,
        0.26792126, 0.10683084 ],
 (100,
  1.0,
  1e-06):  [0.04856236, 0.50203791, 0.92692547, 0.08816816, 0.05046215,
        0.12983868, 0.09571361 ],
 (100,
  1.0,
  1e-07):  [0.04983524, 0.50189267, 0.97368335, 0.02776147, 0.04975143,
        0.07475607, 0.0946658  ],
 (100, 5.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (100,
  5.0,
  0.1):  [2.48406551e-02, 5.00000856e-01, 1.00000000e+00, 1.79674840e-05,
        4.96821180e-02, 4.96983446e-02, 9.46612640e-02 ],
 (100,
  5.0,
  0.01):  [2.48406551e-02, 5.00000856e-01, 1.00000000e+00, 1.26627982e-04,
        4.96872489e-02, 4.98016067e-02, 9.46705773e-02 ],
 (100,
  5.0,
  0.001):  [0.02484066, 0.50000086, 0.99756145, 0.02744833, 0.05089383,
        0.07564478, 0.09684672 ],
 (100,
  5.0,
  0.0001):  [0.02720349, 0.52770697, 0.99823246, 0.08581014, 0.05400189,
        0.13114044, 0.1024609  ],
 (100,
  5.0,
  1e-05):  [0.04166342, 0.49673644, 0.90753167, 0.33073347, 0.06619743,
        0.35938953, 0.12339421 ],
 (100,
  5.0,
  1e-06):  [0.04588435, 0.50234539, 0.89828484, 0.17018116, 0.05356078,
        0.20635428, 0.10109378 ],
 (100,
  5.0,
  1e-07):  [0.04922257, 0.50198872, 0.92429053, 0.08447541, 0.05013311,
        0.12619849, 0.09510762 ],
 (100, 10.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (100,
  10.0,
  0.1):  [2.48406551e-02, 5.00000856e-01, 1.00000000e+00, 1.79674840e-05,
        4.96821180e-02, 4.96983446e-02, 9.46612640e-02 ],
 (100,
  10.0,
  0.01):  [2.48406551e-02, 5.00000856e-01, 1.00000000e+00, 7.95702862e-05,
        4.96850268e-02, 4.97568869e-02, 9.46665438e-02 ],
 (100,
  10.0,
  0.001):  [0.02484066, 0.50000086, 0.99942719, 0.00355842, 0.04982271,
        0.05303444, 0.09491385 ],
 (100,
  10.0,
  0.0001):  [0.02502494, 0.49329948, 0.99965631, 0.04111474, 0.0516845 ,
        0.0887363 , 0.09828732 ],
 (100,
  10.0,
  1e-05):  [0.03298093, 0.49669628, 0.91810415, 0.34476949, 0.06825263,
        0.37325349, 0.12705955 ],
 (100,
  10.0,
  1e-06):  [0.04446879, 0.4992933 , 0.87088802, 0.24489937, 0.0568662 ,
        0.27599928, 0.10676123 ],
 (100,
  10.0,
  1e-07):  [0.04857297, 0.50204292, 0.9261399 , 0.08889455, 0.05045971,
        0.13048997, 0.09570504 ],
 (100, 15.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (100,
  15.0,
  0.1):  [2.48406551e-02, 5.00000856e-01, 1.00000000e+00, 1.79674840e-05,
        4.96821180e-02, 4.96983446e-02, 9.46612640e-02 ],
 (100,
  15.0,
  0.01):  [2.48406551e-02, 5.00000856e-01, 1.00000000e+00, 4.44909127e-05,
        4.96833704e-02, 4.97235503e-02, 9.46635371e-02 ],
 (100,
  15.0,
  0.001):  [2.48406551e-02, 5.00000856e-01, 1.00000000e+00, 6.85331175e-04,
        4.97136474e-02, 5.03325528e-02, 9.47184930e-02 ],
 (100,
  15.0,
  0.0001):  [0.0250121 , 0.49667436, 0.99988544, 0.0221907 , 0.05074601,
        0.07076381, 0.09658991 ],
 (100,
  15.0,
  1e-05):  [0.0331595 , 0.55323547, 0.97836405, 0.24797352, 0.06368165,
        0.28426025, 0.11957985 ],
 (100,
  15.0,
  1e-06):  [0.04479392, 0.50727184, 0.87676344, 0.27304415, 0.05931211,
        0.30303769, 0.11110788 ],
 (100,
  15.0,
  1e-07):  [0.04803925, 0.50221737, 0.89312952, 0.12993913, 0.05093142,
        0.1678554 , 0.09636742 ],
 (100, 20.0, 1.0):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (100, 20.0, 0.1):  [0.0, 0.0,  1.,  1.,  1.,  1.,  1. ],
 (100,
  20.0,
  0.01):  [2.48406551e-02, 5.00000856e-01, 1.00000000e+00, 1.96786729e-05,
        4.96821988e-02, 4.96999707e-02, 9.46614106e-02 ],
 (100,
  20.0,
  0.001):  [2.48406551e-02, 5.00000856e-01, 1.00000000e+00, 1.99353513e-04,
        4.96906836e-02, 4.98707191e-02, 9.46768117e-02 ],
 (100,
  20.0,
  0.0001):  [0.02504494, 0.50428048, 0.9999509 , 0.01440992, 0.0503687 ,
        0.06337285, 0.09590648 ],
 (100,
  20.0,
  1e-05):  [0.02852381, 0.48507293, 0.98579425, 0.22058167, 0.06202012,
        0.2585984 , 0.11669829 ],
 (100,
  20.0,
  1e-06):  [0.0431234 , 0.49732742, 0.87869464, 0.2949294 , 0.06116699,
        0.3239316 , 0.11437238 ],
 (100,
  20.0,
  1e-07):  [0.0475675 , 0.50212182, 0.88519197, 0.14494968, 0.05134268,
        0.18172586, 0.09705595]})



In [ ]:
df

1                                                        \
          0.1                                                        
  1.000000e+00 1.000000e-01 1.000000e-02 1.000000e-03 1.000000e-04   
0          0.0     0.427567     0.430614     0.169031     0.117108   
1          0.0     0.515401     0.579363     0.738493     0.646487   
2          1.0     0.793329     0.208242     0.571176     0.277569   
3          1.0     0.237421     0.932123     0.896016     0.923495   
4          1.0     0.051581     0.138219     0.223097     0.159433   
5          1.0     0.265040     0.896160     0.879878     0.891405   
6          1.0     0.096865     0.166154     0.320866     0.202533   

                                                                    ...  \
                                                 1.0                ...   
  1.000000e-05 1.000000e-06 1.000000e-07 1.000000e+00 1.000000e-01  ...   
0     0.049716     0.050865     0.051045          0.0     0.024848  ...   
1     0.500050     0.500978     0.501792          0.0     0.500136  ...   
2     0.997594     0.999885     0.999885          1.0     0.998412  ...   
3     0.002960     0.000198     0.000181          1.0     0.013346  ...   
4     0.049708     0.049685     0.049684          1.0     0.050244  ...   
5     0.052375     0.049864     0.049848          1.0     0.062285  ...   
6     0.094697     0.094666     0.094665          1.0     0.095673  ...   

           100                                                      \
          15.0                      20.0                             
  1.000000e-06 1.000000e-07 1.000000e+00 1.000000e-01 1.000000e-02   
0     0.044794     0.048039          0.0          0.0     0.024841   
1     0.507272     0.502217          0.0          0.0     0.500001   
2     0.876763     0.893130          1.0          1.0     1.000000   
3     0.273044     0.129939          1.0          1.0     0.000020   
4     0.059312     0.050931          1.0          1.0     0.049682   
5     0.303038     0.167855          1.0          1.0     0.049700   
6     0.111108     0.096367          1.0          1.0     0.094661   

                                                                    
                                                                    
  1.000000e-03 1.000000e-04 1.000000e-05 1.000000e-06 1.000000e-07  
0     0.024841     0.025045     0.028524     0.043123     0.047567  
1     0.500001     0.504280     0.485073     0.497327     0.502122  
2     1.000000     0.999951     0.985794     0.878695     0.885192  
3     0.000199     0.014410     0.220582     0.294929     0.144950  
4     0.049691     0.050369     0.062020     0.061167     0.051343  
5     0.049871     0.063373     0.258598     0.323932     0.181726  
6     0.094677     0.095906     0.116698     0.114372     0.097056  

[7 rows x 528 columns]

In [ ]:
results_uniform_dist = pd.DataFrame.from_dict({(1,
  0.1,
  1.0):  [0.02827034, 0.51060327, 0.79357468, 0.2354673 , 0.05147135,
        0.26319478, 0.09667251 ],
 (1,
  0.1,
  0.1):  [0.02827034, 0.51060327, 0.79357468, 0.2354673 , 0.05147135,
        0.26319478, 0.09667251 ],
 (1,
  0.1,
  0.01):  [0.03339866, 0.55575068, 0.78778109, 0.42957602, 0.06733731,
        0.4473721 , 0.12406951 ],
 (1,
  0.1,
  0.001):  [0.09344865, 0.69007059, 0.67622991, 0.81629189, 0.16138154,
        0.80933343, 0.26057673 ],
 (1,
  0.1,
  0.0001):  [0.24605426, 0.72730721, 0.37663252, 0.95659997, 0.31209146,
        0.92778645, 0.34133788 ],
 (1,
  0.1,
  1e-05):  [0.06777455, 0.58356923, 0.45437138, 0.68383132, 0.06988024,
        0.67243146, 0.12113107 ],
 (1,
  0.1,
  1e-06):  [0.04627416, 0.51273351, 0.74058132, 0.28143411, 0.05112559,
        0.30424513, 0.09564816 ],
 (1,
  0.1,
  1e-07):  [0.04431661, 0.50268896, 0.7845897 , 0.22126358, 0.05003601,
        0.24925033, 0.09407268 ],
 (1,
  1.0,
  1.0):  [0.        , 0.49998032, 0.00392786, 0.9673103 , 0.00624236,
        0.91944824, 0.00482175 ],
 (1,
  1.0,
  0.1):  [0.00312594, 0.48975158, 0.00430428, 0.9644278 , 0.006286  ,
        0.91672765, 0.00510972 ],
 (1,
  1.0,
  0.01):  [0.00198794, 0.47185394, 0.79755163, 0.20257996, 0.04968906,
        0.23213891, 0.09354978 ],
 (1,
  1.0,
  0.001):  [0.02670004, 0.34951138, 0.79594776, 0.23445171, 0.05155238,
        0.26234755, 0.09683303 ],
 (1,
  1.0,
  0.0001):  [0.03221428, 0.39266136, 0.75179209, 0.35370789, 0.05732627,
        0.37348522, 0.10652938 ],
 (1,
  1.0,
  1e-05):  [0.03892371, 0.48501261, 0.71532847, 0.34465998, 0.05398349,
        0.36307526, 0.10039081 ],
 (1,
  1.0,
  1e-06):  [0.0428938 , 0.50196684, 0.74382181, 0.27400499, 0.05083917,
        0.29734608, 0.09517338 ],
 (1,
  1.0,
  1e-07):  [0.04387747, 0.50160622, 0.78452424, 0.22127727, 0.05003288,
        0.24926009, 0.09406667 ],
 (1, 5.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (1,
  5.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (1,
  5.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (1,
  5.0,
  0.001):  [0.00000000e+00, 4.99543113e-01, 1.96392917e-04, 9.96052287e-01,
        2.59403372e-03, 9.46576902e-01, 3.65141188e-04 ],
 (1,
  5.0,
  0.0001):  [4.53836326e-04, 4.47652272e-01, 2.98517233e-02, 6.48074314e-01,
        4.41489349e-03, 6.17360230e-01, 7.69216171e-03 ],
 (1,
  5.0,
  1e-05):  [0.01639706, 0.26517337, 0.79619325, 0.20427233, 0.04970889,
        0.23367971, 0.09357556 ],
 (1,
  5.0,
  1e-06):  [0.0358256 , 0.45578811, 0.77900887, 0.22828202, 0.05012711,
        0.25564283, 0.09419315 ],
 (1,
  5.0,
  1e-07):  [0.04239531, 0.4967728 , 0.78398416, 0.22142871, 0.0500094 ,
        0.24937718, 0.09402129 ],
 (1, 10.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (1,
  10.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (1,
  10.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (1,
  10.0,
  0.001):  [0.       , 0.4999153, 1.       , 1.       , 1.       , 1.       ,
        1.        ],
 (1,
  10.0,
  0.0001):  [4.09963680e-05, 4.84372985e-01, 3.76419757e-03, 9.03756744e-01,
        2.04050853e-03, 8.59043972e-01, 2.64643106e-03 ],
 (1,
  10.0,
  1e-05):  [0.0032064 , 0.26638727, 0.37916926, 0.24791107, 0.02567965,
        0.25443214, 0.04810157 ],
 (1,
  10.0,
  1e-06):  [0.0311842 , 0.40638627, 0.79139799, 0.21038726, 0.049788  ,
        0.23925261, 0.09368231 ],
 (1,
  10.0,
  1e-07):  [0.04095849, 0.49074865, 0.78331315, 0.22162207, 0.04998052,
        0.2495276 , 0.09396544 ],
 (1, 15.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (1,
  15.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (1,
  15.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (1,
  15.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (1,
  15.0,
  0.0001):  [0.00000000e+00, 4.92658572e-01, 6.71009132e-04, 9.59668132e-01,
        8.69012293e-04, 9.12023937e-01, 7.57281912e-04 ],
 (1,
  15.0,
  1e-05):  [0.00150401, 0.32495848, 0.14107558, 0.34763317, 0.01117895,
        0.33737113, 0.02071632 ],
 (1,
  15.0,
  1e-06):  [0.02771098, 0.36643739, 0.79278911, 0.20632404, 0.04962851,
        0.23546037, 0.09340959 ],
 (1,
  15.0,
  1e-07):  [0.03974899, 0.48470091, 0.78645544, 0.21771885, 0.04993313,
        0.2459744 , 0.09390416 ],
 (1, 20.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (1, 20.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (1,
  20.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (1,
  20.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (1,
  20.0,
  0.0001):  [0.00000000e+00, 4.95741706e-01, 2.45491146e-04, 9.77524389e-01,
        5.70689393e-04, 9.28971932e-01, 3.43304419e-04 ],
 (1,
  20.0,
  1e-05):  [0.0009493 , 0.36382408, 0.06389316, 0.45750604, 0.00611952,
        0.43795086, 0.01116927 ],
 (1,
  20.0,
  1e-06):  [0.02416602, 0.33288124, 0.78747013, 0.20735589, 0.04937299,
        0.2361767 , 0.09292005 ],
 (1,
  20.0,
  1e-07):  [0.03870221, 0.47872428, 0.78604628, 0.21789168, 0.04991893,
        0.24611832, 0.09387613 ],
 (5, 0.1, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (5,
  0.1,
  0.1):  [0.02745583, 0.492274  , 0.79678243, 0.20924333, 0.05004091,
        0.23843302, 0.09416773 ],
 (5,
  0.1,
  0.01):  [0.02750894, 0.49874263, 0.79090701, 0.31541576, 0.05695778,
        0.33903877, 0.10626295 ],
 (5,
  0.1,
  0.001):  [0.03268678, 0.43552135, 0.75509803, 0.38298034, 0.06013056,
        0.40146762, 0.11139077 ],
 (5,
  0.1,
  0.0001):  [0.03191948, 0.3896803 , 0.74972996, 0.31907599, 0.05442822,
        0.34047143, 0.10148866 ],
 (5,
  0.1,
  1e-05):  [0.03390878, 0.42572668, 0.77825603, 0.22803047, 0.05006559,
        0.25536638, 0.09407903 ],
 (5,
  0.1,
  1e-06):  [0.04073729, 0.48237027, 0.77902524, 0.22210377, 0.04974979,
        0.24977234, 0.09352681 ],
 (5,
  0.1,
  1e-07):  [0.04290996, 0.49216197, 0.7773559 , 0.22322631, 0.04971666,
        0.25075617, 0.09345622 ],
 (5, 1.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (5,
  1.0,
  0.1):  [0.00318465, 0.48760753, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (5,
  1.0,
  0.01):  [0.00315225, 0.48356797, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (5,
  1.0,
  0.001):  [0.02607596, 0.28739789, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (5,
  1.0,
  0.0001):  [0.02594193, 0.24810461, 0.79784622, 0.20016975, 0.04956414,
        0.22986308, 0.09333037 ],
 (5,
  1.0,
  1e-05):  [0.02799877, 0.31508784, 0.79715885, 0.20107411, 0.04957683,
        0.23068836, 0.09334817 ],
 (5,
  1.0,
  1e-06):  [0.03731694, 0.45681543, 0.7976171 , 0.20053766, 0.04957228,
        0.23020132, 0.09334324 ],
 (5,
  1.0,
  1e-07):  [0.04239023, 0.48942906, 0.77825603, 0.22173758, 0.04968088,
        0.24938612, 0.0933995  ],
 (5, 5.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (5,
  5.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (5,
  5.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (5,
  5.0,
  0.001):  [0.00000000e+00, 4.99065263e-01, 3.92785834e-04, 9.93698547e-01,
        3.24807146e-03, 9.44349855e-01, 7.00822006e-04 ],
 (5,
  5.0,
  0.0001):  [4.72135011e-04, 4.33671876e-01, 3.85093778e-02, 5.69820787e-01,
        4.65814225e-03, 5.43424562e-01, 8.31097823e-03 ],
 (5,
  5.0,
  1e-05):  [0.01746798, 0.21112922, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (5,
  5.0,
  1e-06):  [0.03020769, 0.36585339, 0.79782986, 0.20013039, 0.04956086,
        0.22982486, 0.09332444 ],
 (5,
  5.0,
  1e-07):  [0.04033985, 0.4773565 , 0.79774803, 0.20032803, 0.04956767,
        0.23000862, 0.09333595 ],
 (5, 10.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (5,
  10.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (5,
  10.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (5,
  10.0,
  0.001):  [0.        , 0.49981434, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (5,
  10.0,
  0.0001):  [8.43867585e-05, 4.77263130e-01, 6.33367157e-03, 8.70100224e-01,
        2.54252321e-03, 8.27187205e-01, 3.62847084e-03 ],
 (5,
  10.0,
  1e-05):  [0.00350253, 0.21876555, 0.45422408, 0.2159452 , 0.02939606,
        0.2277832 , 0.05521854 ],
 (5,
  10.0,
  1e-06):  [0.02750614, 0.30437849, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (5,
  10.0,
  1e-07):  [0.03823981, 0.46251186, 0.79776439, 0.20024846, 0.04956395,
        0.22993381, 0.09332947 ],
 (5, 15.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (5,
  15.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (5,
  15.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (5,
  15.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (5,
  15.0,
  0.0001):  [3.71888732e-05, 4.88513694e-01, 1.96392917e-03, 9.40525917e-01,
        1.72334559e-03, 8.93896966e-01, 1.83578870e-03 ],
 (5,
  15.0,
  1e-05):  [0.0016242 , 0.28161014, 0.17915944, 0.28571551, 0.01294298,
        0.28042167, 0.02414187 ],
 (5,
  15.0,
  1e-06):  [0.02642843, 0.27436129, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (5,
  15.0,
  1e-07):  [0.03654291, 0.44824345, 0.79781349, 0.20016718, 0.04956206,
        0.22985901, 0.09332645 ],
 (5, 20.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (5, 20.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (5,
  20.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (5,
  20.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (5,
  20.0,
  0.0001):  [3.03748253e-05, 4.92966640e-01, 4.90982292e-04, 9.68001622e-01,
        8.01517540e-04, 9.19934465e-01, 6.08945408e-04 ],
 (5,
  20.0,
  1e-05):  [0.00102769, 0.32674833, 0.08256686, 0.38127087, 0.00692802,
        0.36643087, 0.01278341 ],
 (5,
  20.0,
  1e-06):  [0.02497819, 0.2557531 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (5,
  20.0,
  1e-07):  [0.03513375, 0.43444985, 0.79782986, 0.20013809, 0.04956131,
        0.22983218, 0.09332524 ],
 (10, 0.1, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (10,
  0.1,
  0.1):  [0.02720371, 0.48650828, 0.79774803, 0.20092267, 0.04960272,
        0.23057371, 0.0933981  ],
 (10,
  0.1,
  0.01):  [0.02566553, 0.43117653, 0.79305096, 0.2468912 , 0.0521787 ,
        0.27402511, 0.09791508 ],
 (10,
  0.1,
  0.001):  [0.02745038, 0.31900384, 0.78910674, 0.22793294, 0.05072213,
        0.25581276, 0.09531745 ],
 (10,
  0.1,
  0.0001):  [0.02644102, 0.26013656, 0.7976171 , 0.20057616, 0.04957455,
        0.23023791, 0.09334726 ],
 (10,
  0.1,
  1e-05):  [0.03198902, 0.2866871 , 0.99998363, 0.        , 0.0496805 ,
        0.04968046, 0.09465825 ],
 (10,
  0.1,
  1e-06):  [0.03509781, 0.43225464, 0.79784622, 0.20010815, 0.04956051,
        0.22980453, 0.09332394 ],
 (10,
  0.1,
  1e-07):  [0.04317556, 0.49130897, 0.79782986, 0.20015692, 0.04956242,
        0.22985007, 0.09332721 ],
 (10, 1.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (10,
  1.0,
  0.1):  [0.0030806 , 0.49701965, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (10,
  1.0,
  0.01):  [0.00315589, 0.48360122, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (10,
  1.0,
  0.001):  [0.02558537, 0.32196497, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (10,
  1.0,
  0.0001):  [0.02581056, 0.24021248, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (10,
  1.0,
  1e-05):  [0.02609015, 0.25367216, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (10,
  1.0,
  1e-06):  [0.03253117, 0.40126149, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (10,
  1.0,
  1e-07):  [0.04242786, 0.48727755, 0.79782986, 0.20015606, 0.04956237,
        0.22984925, 0.09332712 ],
 (10, 5.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (10,
  5.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (10,
  5.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (10,
  5.0,
  0.001):  [0.00000000e+00, 4.98999382e-01, 3.76419757e-04, 9.94040784e-01,
        3.29135661e-03, 9.44674277e-01, 6.75576443e-04 ],
 (10,
  5.0,
  0.0001):  [0.00061716, 0.42580435, 0.04459756, 0.53173828, 0.00495437,
        0.50753651, 0.00891804 ],
 (10,
  5.0,
  1e-05):  [0.01876965, 0.20981188, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (10,
  5.0,
  1e-06):  [0.02784203, 0.31365134, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (10,
  5.0,
  1e-07):  [0.03958563, 0.46966179, 0.79782986, 0.20013895, 0.04956136,
        0.22983299, 0.09332533 ],
 (10, 10.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (10,
  10.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (10,
  10.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (10,
  10.0,
  0.001):  [0.       , 0.4997861, 1.       , 1.       , 1.       , 1.       ,
        1.        ],
 (10,
  10.0,
  0.0001):  [1.19406012e-04, 4.75037916e-01, 7.90481490e-03, 8.53409287e-01,
        2.81116317e-03, 8.11403552e-01, 4.14740015e-03 ],
 (10,
  10.0,
  1e-05):  [0.00422957, 0.20198381, 0.49528657, 0.21194957, 0.03181162,
        0.22602612, 0.05978343 ],
 (10,
  10.0,
  1e-06):  [0.02647007, 0.27113237, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (10,
  10.0,
  1e-07):  [0.03684952, 0.44820214, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (10, 15.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (10,
  15.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (10,
  15.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (10,
  15.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (10,
  15.0,
  0.0001):  [1.80639576e-05, 4.88167234e-01, 2.02939347e-03, 9.39879087e-01,
        1.76156381e-03, 8.93285524e-01, 1.88601761e-03 ],
 (10,
  15.0,
  1e-05):  [0.00185254, 0.26571674, 0.21136788, 0.26966712, 0.01490458,
        0.26677074, 0.02784563 ],
 (10,
  15.0,
  1e-06):  [0.02603981, 0.25384819, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (10,
  15.0,
  1e-07):  [0.03478074, 0.4279397 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (10, 20.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (10, 20.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (10,
  20.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (10,
  20.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (10,
  20.0,
  0.0001):  [2.92348107e-05, 4.92692027e-01, 4.90982292e-04, 9.67715854e-01,
        7.94428409e-04, 9.19662894e-01, 6.06888181e-04 ],
 (10,
  20.0,
  1e-05):  [0.0011688 , 0.313259  , 0.10047134, 0.34918864, 0.00800607,
        0.33683205, 0.01483038 ],
 (10,
  20.0,
  1e-06):  [0.02545332, 0.24422365, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (10,
  20.0,
  1e-07):  [0.03317366, 0.40911294, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15, 0.1, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (15,
  0.1,
  0.1):  [0.02721065, 0.48620875, 0.79782986, 0.20090214, 0.04960635,
        0.23055827, 0.09340508 ],
 (15,
  0.1,
  0.01):  [0.03097275, 0.4929042 , 0.65485581, 0.39874039, 0.05387128,
        0.41146453, 0.0995529  ],
 (15,
  0.1,
  0.001):  [0.02675881, 0.2958754 , 0.79159438, 0.21042662, 0.0498021 ,
        0.23929977, 0.09370865 ],
 (15,
  0.1,
  0.0001):  [0.0260955 , 0.24485805, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15,
  0.1,
  1e-05):  [0.03110278, 0.25255509, 0.99998363, 0.        , 0.0496805 ,
        0.04968046, 0.09465825 ],
 (15,
  0.1,
  1e-06):  [0.03086754, 0.37568679, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15,
  0.1,
  1e-07):  [0.04137005, 0.48250505, 0.79779713, 0.20017146, 0.04956134,
        0.22986226, 0.09332508 ],
 (15, 1.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (15,
  1.0,
  0.1):  [0.00295421, 0.49974482, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (15,
  1.0,
  0.01):  [0.00315722, 0.48360158, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15,
  1.0,
  0.001):  [0.01948648, 0.29206498, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15,
  1.0,
  0.0001):  [0.02581141, 0.23918237, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15,
  1.0,
  1e-05):  [0.02581782, 0.23980376, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15,
  1.0,
  1e-06):  [0.02943243, 0.34978348, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15,
  1.0,
  1e-07):  [0.04060965, 0.47765321, 0.79779713, 0.20016804, 0.04956114,
        0.22985901, 0.09332472 ],
 (15, 5.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (15,
  5.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (15,
  5.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (15,
  5.0,
  0.001):  [0.00000000e+00, 4.98991254e-01, 5.40080521e-04, 9.93746460e-01,
        4.49468810e-03, 9.44402706e-01, 9.64291976e-04 ],
 (15,
  5.0,
  0.0001):  [0.00073223, 0.42236433, 0.04796897, 0.51758076, 0.00517139,
        0.49424985, 0.00933627 ],
 (15,
  5.0,
  1e-05):  [0.01827611, 0.2068406 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15,
  5.0,
  1e-06):  [0.02685904, 0.28543363, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15,
  5.0,
  1e-07):  [0.03772433, 0.45619723, 0.79784622, 0.200109  , 0.04956056,
        0.22980535, 0.09332403 ],
 (15, 10.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (15,
  10.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (15,
  10.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (15,
  10.0,
  0.001):  [0.        , 0.49978396, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (15,
  10.0,
  0.0001):  [1.04717128e-04, 4.73554115e-01, 8.77221695e-03, 8.44899545e-01,
        2.94806781e-03, 8.03359677e-01, 4.41304813e-03 ],
 (15,
  10.0,
  1e-05):  [0.00439145, 0.19850444, 0.50273313, 0.21144563, 0.03225451,
        0.22591716, 0.06061975 ],
 (15,
  10.0,
  1e-06):  [0.02612541, 0.2569516 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15,
  10.0,
  1e-07):  [0.03505066, 0.43082147, 0.79784622, 0.20010815, 0.04956051,
        0.22980453, 0.09332394 ],
 (15, 15.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (15,
  15.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (15,
  15.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (15,
  15.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (15,
  15.0,
  0.0001):  [3.44475736e-05, 4.87598199e-01, 2.40581323e-03, 9.36138428e-01,
        1.96558225e-03, 8.89749406e-01, 2.16353053e-03 ],
 (15,
  15.0,
  1e-05):  [0.00193202, 0.26177595, 0.22092567, 0.26632603, 0.01549826,
        0.26407048, 0.02896461 ],
 (15,
  15.0,
  1e-06):  [0.02588534, 0.24570062, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15,
  15.0,
  1e-07):  [0.03307784, 0.40762598, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15, 20.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (15, 20.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (15,
  20.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (15,
  20.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (15,
  20.0,
  0.0001):  [2.81216461e-05, 4.92402408e-01, 6.21910903e-04, 9.66276744e-01,
        9.63171368e-04, 9.18301786e-01, 7.55805281e-04 ],
 (15,
  20.0,
  1e-05):  [0.00119601, 0.30828734, 0.10785244, 0.34108787, 0.00848448,
        0.32950044, 0.01573142 ],
 (15,
  20.0,
  1e-06):  [0.02539059, 0.23907226, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (15,
  20.0,
  1e-07):  [0.03160086, 0.38700791, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (20, 0.1, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (20,
  0.1,
  0.1):  [0.02719044, 0.4858975 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (20,
  0.1,
  0.01):  [0.02561882, 0.42868975, 0.7929364 , 0.22622602, 0.05084906,
        0.25438092, 0.09556948 ],
 (20,
  0.1,
  0.001):  [0.02661425, 0.2779604 , 0.79576773, 0.20382827, 0.04965731,
        0.23323658, 0.09348123 ],
 (20,
  0.1,
  0.0001):  [3.10522207e-02, 2.41040450e-01, 1.00000000e+00, 2.56678343e-06,
        4.96813909e-02, 4.96837090e-02, 9.46599441e-02 ],
 (20,
  0.1,
  1e-05):  [3.08914744e-02, 2.41775470e-01, 1.00000000e+00, 8.55594476e-07,
        4.96813101e-02, 4.96820828e-02, 9.46597975e-02 ],
 (20,
  0.1,
  1e-06):  [0.02866419, 0.33341691, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (20,
  0.1,
  1e-07):  [0.04084649, 0.47578453, 0.79784622, 0.20011157, 0.04956071,
        0.22980779, 0.0933243  ],
 (20, 1.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (20,
  1.0,
  0.1):  [0.        , 0.49998032, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (20,
  1.0,
  0.01):  [0.00315494, 0.48360109, 0.25063009, 0.34650806, 0.01965599,
        0.34174472, 0.0364531  ],
 (20,
  1.0,
  0.001):  [0.00291517, 0.25923741, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (20,
  1.0,
  0.0001):  [0.02580827, 0.24039338, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (20,
  1.0,
  1e-05):  [0.02574163, 0.23516699, 0.79784622, 0.20010815, 0.04956051,
        0.22980453, 0.09332394 ],
 (20,
  1.0,
  1e-06):  [0.02786409, 0.31438021, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (20,
  1.0,
  1e-07):  [0.04001056, 0.47026474, 0.79784622, 0.20011157, 0.04956071,
        0.22980779, 0.0933243  ],
 (20, 5.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (20,
  5.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (20,
  5.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (20,
  5.0,
  0.001):  [0.00000000e+00, 4.98990826e-01, 4.74616216e-04, 9.94183669e-01,
        4.24783946e-03, 9.44814941e-01, 8.53832678e-04 ],
 (20,
  5.0,
  0.0001):  [0.00072628, 0.42257074, 0.04787077, 0.51631362, 0.00514741,
        0.49304078, 0.00929532 ],
 (20,
  5.0,
  1e-05):  [0.01813398, 0.20603982, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (20,
  5.0,
  1e-06):  [0.02640985, 0.26895321, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (20,
  5.0,
  1e-07):  [0.03693147, 0.44643676, 0.79784622, 0.20011157, 0.04956071,
        0.22980779, 0.0933243  ],
 (20, 10.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (20,
  10.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (20,
  10.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (20,
  10.0,
  0.001):  [0.        , 0.49978396, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (20,
  10.0,
  0.0001):  [9.25186625e-05, 4.74660379e-01, 8.31396681e-03, 8.51591149e-01,
        2.92012761e-03, 8.09696068e-01, 4.32217198e-03 ],
 (20,
  10.0,
  1e-05):  [0.00470957, 0.19484841, 0.51193087, 0.2108142 , 0.03279985,
        0.22577406, 0.06164974 ],
 (20,
  10.0,
  1e-06):  [0.02597701, 0.24943658, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (20,
  10.0,
  1e-07):  [0.03415783, 0.41885769, 0.79784622, 0.20010986, 0.04956061,
        0.22980616, 0.09332412 ],
 (20, 15.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (20, 15.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (20,
  15.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (20,
  15.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (20,
  15.0,
  0.0001):  [3.50610487e-05, 4.87815524e-01, 2.12758993e-03, 9.39198034e-01,
        1.82599657e-03, 8.92643185e-01, 1.96528996e-03 ],
 (20,
  15.0,
  1e-05):  [0.00205452, 0.25642179, 0.23612975, 0.26184442, 0.01644839,
        0.26056688, 0.03075447 ],
 (20,
  15.0,
  1e-06):  [0.02581922, 0.24171032, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (20,
  15.0,
  1e-07):  [0.03218748, 0.39428161, 0.79784622, 0.20010815, 0.04956051,
        0.22980453, 0.09332394 ],
 (20, 20.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (20, 20.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (20,
  20.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (20,
  20.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (20,
  20.0,
  0.0001):  [2.87703550e-05, 4.92573922e-01, 6.05544827e-04, 9.67243566e-01,
        9.65502844e-04, 9.19219761e-01, 7.44287094e-04 ],
 (20,
  20.0,
  1e-05):  [0.00126073, 0.30373558, 0.11502078, 0.33287844, 0.00893301,
        0.322055  , 0.01657846 ],
 (20,
  20.0,
  1e-06):  [0.0253992 , 0.23681304, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (20,
  20.0,
  1e-07):  [0.03076053, 0.37291403, 0.79784622, 0.20010815, 0.04956051,
        0.22980453, 0.09332394 ],
 (25, 0.1, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (25,
  0.1,
  0.1):  [0.02720681, 0.48617271, 0.79782986, 0.20081829, 0.0496014 ,
        0.23047858, 0.09339631 ],
 (25,
  0.1,
  0.01):  [0.03040876, 0.48003229, 0.64096102, 0.38564124, 0.05172125,
        0.39832585, 0.09571864 ],
 (25,
  0.1,
  0.001):  [0.02644233, 0.27270419, 0.79642238, 0.20204521, 0.04959059,
        0.23157463, 0.0933675  ],
 (25,
  0.1,
  0.0001):  [3.10337106e-02, 2.39609657e-01, 1.00000000e+00, 1.54007006e-05,
        4.96819968e-02, 4.96959053e-02, 9.46610440e-02 ],
 (25,
  0.1,
  1e-05):  [0.03080904, 0.23702648, 0.99996727, 0.        , 0.04967972,
        0.04967964, 0.09465677 ],
 (25,
  0.1,
  1e-06):  [0.02733212, 0.29986075, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (25,
  0.1,
  1e-07):  [0.03842257, 0.45579846, 0.79784622, 0.20014066, 0.04956243,
        0.22983543, 0.09332733 ],
 (25, 1.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (25,
  1.0,
  0.1):  [0.        , 0.49998032, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (25,
  1.0,
  0.01):  [0.00315217, 0.48360297, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (25,
  1.0,
  0.001):  [0.00220199, 0.2817266 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (25,
  1.0,
  0.0001):  [0.02581796, 0.23838872, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (25,
  1.0,
  1e-05):  [2.57152827e-02, 2.32841933e-01, 9.99983634e-01, 8.55594476e-07,
        4.96805374e-02, 4.96812697e-02, 9.46583216e-02 ],
 (25,
  1.0,
  1e-06):  [0.02692491, 0.28739473, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (25,
  1.0,
  1e-07):  [0.03764505, 0.44977601, 0.79784622, 0.20013381, 0.04956202,
        0.22982893, 0.09332662 ],
 (25, 5.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (25,
  5.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (25,
  5.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (25,
  5.0,
  0.001):  [0.00000000e+00, 4.98990399e-01, 4.58250139e-04, 9.94339387e-01,
        4.21432872e-03, 9.44962110e-01, 8.26617070e-04 ],
 (25,
  5.0,
  0.0001):  [0.00065302, 0.42581248, 0.04518674, 0.52875311, 0.00498786,
        0.50472892, 0.00898403 ],
 (25,
  5.0,
  1e-05):  [0.01714502, 0.20265687, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (25,
  5.0,
  1e-06):  [0.02614781, 0.25769957, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (25,
  5.0,
  1e-07):  [0.03487484, 0.42444093, 0.79784622, 0.20012269, 0.04956137,
        0.22981836, 0.09332546 ],
 (25, 10.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (25,
  10.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (25,
  10.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (25,
  10.0,
  0.001):  [0.        , 0.49978353, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (25,
  10.0,
  0.0001):  [9.51426414e-05, 4.75361872e-01, 7.56112730e-03, 8.59228185e-01,
        2.80011879e-03, 8.16916285e-01, 4.08677768e-03 ],
 (25,
  10.0,
  1e-05):  [0.00446145, 0.19637954, 0.49790514, 0.21207021, 0.03197917,
        0.22627086, 0.06009838 ],
 (25,
  10.0,
  1e-06):  [0.02588954, 0.24438259, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (25,
  10.0,
  1e-07):  [0.03245047, 0.39619132, 0.79784622, 0.20011157, 0.04956071,
        0.22980779, 0.0933243  ],
 (25, 15.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (25, 15.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (25,
  15.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (25,
  15.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (25,
  15.0,
  0.0001):  [3.61903263e-05, 4.88196266e-01, 1.86573271e-03, 9.41119699e-01,
        1.65380375e-03, 8.94456370e-01, 1.75338758e-03 ],
 (25,
  15.0,
  1e-05):  [0.0019731 , 0.25954891, 0.22706294, 0.26472093, 0.01588774,
        0.26285003, 0.02969752 ],
 (25,
  15.0,
  1e-06):  [0.02576413, 0.23881796, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (25,
  15.0,
  1e-07):  [0.03077574, 0.37196392, 0.79784622, 0.20011071, 0.04956066,
        0.22980697, 0.09332421 ],
 (25, 20.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (25, 20.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (25,
  20.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (25,
  20.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (25,
  20.0,
  0.0001):  [2.94724433e-05, 4.92751031e-01, 4.41884063e-04, 9.67810825e-01,
        7.17150522e-04, 9.19750707e-01, 5.46829906e-04 ],
 (25,
  20.0,
  1e-05):  [0.00123634, 0.30590984, 0.1119276 , 0.3358773 , 0.00873379,
        0.3247512 , 0.01620322 ],
 (25,
  20.0,
  1e-06):  [0.02531185, 0.23481015, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (25,
  20.0,
  1e-07):  [0.02959993, 0.35173191, 0.79784622, 0.200109  , 0.04956056,
        0.22980535, 0.09332403 ],
 (30, 0.1, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (30,
  0.1,
  0.1):  [0.0271923 , 0.48589993, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (30,
  0.1,
  0.01):  [0.025472  , 0.4138575 , 0.79036693, 0.22480317, 0.0506043 ,
        0.2529011 , 0.09511851 ],
 (30,
  0.1,
  0.001):  [0.02586667, 0.26505214, 0.797568  , 0.2008046 , 0.04958512,
        0.23045256, 0.09336566 ],
 (30,
  0.1,
  0.0001):  [3.10452751e-02, 2.39591918e-01, 9.99983634e-01, 2.82346177e-05,
        4.96818301e-02, 4.97072885e-02, 9.46606680e-02 ],
 (30,
  0.1,
  1e-05):  [0.03072601, 0.23397262, 0.99998363, 0.        , 0.0496805 ,
        0.04968046, 0.09465825 ],
 (30,
  0.1,
  1e-06):  [0.02667003, 0.278857  , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (30,
  0.1,
  1e-07):  [0.03589702, 0.43566143, 0.79784622, 0.20011243, 0.04956076,
        0.2298086 , 0.09332439 ],
 (30, 1.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (30,
  1.0,
  0.1):  [0.        , 0.49998032, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (30,
  1.0,
  0.01):  [0.00315053, 0.48360298, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (30,
  1.0,
  0.001):  [0.00196767, 0.2953733 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (30,
  1.0,
  0.0001):  [0.02581264, 0.24083843, 0.79784622, 0.20010815, 0.04956051,
        0.22980453, 0.09332394 ],
 (30,
  1.0,
  1e-05):  [0.02569798, 0.23178228, 0.99996727, 0.        , 0.04967972,
        0.04967964, 0.09465677 ],
 (30,
  1.0,
  1e-06):  [0.02645366, 0.27078602, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (30,
  1.0,
  1e-07):  [0.03527843, 0.42963158, 0.79784622, 0.20011157, 0.04956071,
        0.22980779, 0.0933243  ],
 (30, 5.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (30,
  5.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (30,
  5.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (30,
  5.0,
  0.001):  [0.00000000e+00, 4.98991254e-01, 3.43687604e-04, 9.94410401e-01,
        3.20415014e-03, 9.45023905e-01, 6.20787513e-04 ],
 (30,
  5.0,
  0.0001):  [5.25110879e-04, 4.28084574e-01, 4.31900756e-02, 5.38097911e-01,
        4.86451613e-03, 5.13510261e-01, 8.74417248e-03 ],
 (30,
  5.0,
  1e-05):  [0.01668516, 0.20122317, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (30,
  5.0,
  1e-06):  [0.02600954, 0.25094539, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (30,
  5.0,
  1e-07):  [0.0330183 , 0.40443374, 0.79784622, 0.200109  , 0.04956056,
        0.22980535, 0.09332403 ],
 (30, 10.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (30,
  10.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (30,
  10.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (30,
  10.0,
  0.001):  [0.        , 0.49978396, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (30,
  10.0,
  0.0001):  [6.31339444e-05, 4.76342756e-01, 7.10287716e-03, 8.63382097e-01,
        2.71063644e-03, 8.20841058e-01, 3.92383777e-03 ],
 (30,
  10.0,
  1e-05):  [0.00442616, 0.19613332, 0.49543386, 0.2124578 , 0.03184066,
        0.22651641, 0.05983578 ],
 (30,
  10.0,
  1e-06):  [0.025842  , 0.24143729, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (30,
  10.0,
  1e-07):  [0.03105074, 0.37702137, 0.79784622, 0.20010815, 0.04956051,
        0.22980453, 0.09332394 ],
 (30, 15.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (30, 15.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (30,
  15.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (30,
  15.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (30,
  15.0,
  0.0001):  [3.78419260e-05, 4.88712192e-01, 1.68570587e-03, 9.42718805e-01,
        1.53612122e-03, 8.95967086e-01, 1.60744105e-03 ],
 (30,
  15.0,
  1e-05):  [0.00197012, 0.25913414, 0.2281922 , 0.26458147, 0.01596251,
        0.2627736 , 0.02983781 ],
 (30,
  15.0,
  1e-06):  [0.02573549, 0.23721288, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (30,
  15.0,
  1e-07):  [0.02969872, 0.35400851, 0.79784622, 0.20010815, 0.04956051,
        0.22980453, 0.09332394 ],
 (30, 20.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (30, 20.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (30,
  20.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (30,
  20.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (30,
  20.0,
  0.0001):  [3.05287581e-05, 4.93002148e-01, 4.58250139e-04, 9.68425997e-01,
        7.58170643e-04, 9.20336130e-01, 5.71236216e-04 ],
 (30,
  20.0,
  1e-05):  [0.00122193, 0.30639958, 0.11042192, 0.33689631, 0.00863044,
        0.32564478, 0.01600959 ],
 (30,
  20.0,
  1e-06):  [0.02529587, 0.23378724, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (30,
  20.0,
  1e-07):  [0.0287533 , 0.33510541, 0.79784622, 0.20010815, 0.04956051,
        0.22980453, 0.09332394 ],
 (35, 0.1, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (35,
  0.1,
  0.1):  [0.02719038, 0.48589719, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (35,
  0.1,
  0.01):  [0.02548575, 0.41829905, 0.78832117, 0.23122184, 0.05087995,
        0.25889924, 0.0955903  ],
 (35,
  0.1,
  0.001):  [0.02626401, 0.27464924, 0.7966024 , 0.2017526 , 0.04958396,
        0.23130549, 0.09335699 ],
 (35,
  0.1,
  0.0001):  [3.10340669e-02, 2.39064038e-01, 1.00000000e+00, 1.71118895e-06,
        4.96813505e-02, 4.96828959e-02, 9.46598708e-02 ],
 (35,
  0.1,
  1e-05):  [3.06812567e-02, 2.32342173e-01, 1.00000000e+00, 8.55594476e-07,
        4.96813101e-02, 4.96820828e-02, 9.46597975e-02 ],
 (35,
  0.1,
  1e-06):  [0.02639378, 0.26849495, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (35,
  0.1,
  1e-07):  [0.03648939, 0.44440555, 0.79784622, 0.20011414, 0.04956087,
        0.22981023, 0.09332457 ],
 (35, 1.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (35,
  1.0,
  0.1):  [0.        , 0.49998032, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (35,
  1.0,
  0.01):  [0.00314785, 0.48360026, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (35,
  1.0,
  0.001):  [0.0021064 , 0.28673242, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (35,
  1.0,
  0.0001):  [0.02580938, 0.24138963, 0.79784622, 0.20011157, 0.04956071,
        0.22980779, 0.0933243  ],
 (35,
  1.0,
  1e-05):  [3.07141588e-02, 2.31057745e-01, 9.99983634e-01, 1.71118895e-06,
        4.96805778e-02, 4.96820828e-02, 9.46583949e-02 ],
 (35,
  1.0,
  1e-06):  [0.02624888, 0.26241824, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (35,
  1.0,
  1e-07):  [0.03577748, 0.43747569, 0.79784622, 0.20011328, 0.04956081,
        0.22980941, 0.09332448 ],
 (35, 5.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (35,
  5.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (35,
  5.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (35,
  5.0,
  0.001):  [0.00000000e+00, 4.98990399e-01, 5.72812674e-04, 9.93909879e-01,
        4.89305187e-03, 9.44559632e-01, 1.02556589e-03 ],
 (35,
  5.0,
  0.0001):  [0.00073738, 0.42122238, 0.04862361, 0.51193897, 0.00518132,
        0.48892087, 0.00936474 ],
 (35,
  5.0,
  1e-05):  [0.0160086 , 0.19912161, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (35,
  5.0,
  1e-06):  [0.02593925, 0.24719892, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (35,
  5.0,
  1e-07):  [0.03321608, 0.40854344, 0.79784622, 0.200109  , 0.04956056,
        0.22980535, 0.09332403 ],
 (35, 10.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (35,
  10.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (35,
  10.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (35,
  10.0,
  0.001):  [0.        , 0.49978353, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (35,
  10.0,
  0.0001):  [9.70274829e-05, 4.73645829e-01, 8.91951164e-03, 8.44359665e-01,
        2.98705427e-03, 8.02853937e-01, 4.47535680e-03 ],
 (35,
  10.0,
  1e-05):  [0.00462787, 0.19405494, 0.49664495, 0.21230892, 0.03191017,
        0.2264351 , 0.05996735 ],
 (35,
  10.0,
  1e-06):  [0.02581283, 0.23966904, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (35,
  10.0,
  1e-07):  [0.03104679, 0.37768933, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (35, 15.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (35, 15.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (35,
  15.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (35,
  15.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (35,
  15.0,
  0.0001):  [1.68927237e-05, 4.87346292e-01, 2.45491146e-03, 9.36053724e-01,
        2.00296439e-03, 8.89671350e-01, 2.20602834e-03 ],
 (35,
  15.0,
  1e-05):  [0.00208112, 0.25466699, 0.24028673, 0.26093407, 0.01671284,
        0.25990828, 0.031252   ],
 (35,
  15.0,
  1e-06):  [0.0256865 , 0.23607991, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (35,
  15.0,
  1e-07):  [0.02960492, 0.35256405, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (35, 20.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (35, 20.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (35,
  20.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (35,
  20.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (35,
  20.0,
  0.0001):  [2.68341115e-05, 4.92037465e-01, 6.71009132e-04, 9.66420484e-01,
        1.04357565e-03, 9.18440823e-01, 8.16814424e-04 ],
 (35,
  20.0,
  1e-05):  [0.00127578, 0.30043898, 0.12097804, 0.32777653, 0.00932072,
        0.31750252, 0.01730795 ],
 (35,
  20.0,
  1e-06):  [0.0251784 , 0.2327674 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (35,
  20.0,
  1e-07):  [0.02862394, 0.33246065, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40, 0.1, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (40,
  0.1,
  0.1):  [0.02718984, 0.48589668, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40,
  0.1,
  0.01):  [0.02744919, 0.45643729, 0.75380511, 0.27560153, 0.051594  ,
        0.29935929, 0.09657777 ],
 (40,
  0.1,
  0.001):  [0.02626427, 0.26485281, 0.7956368 , 0.20306936, 0.04960461,
        0.23250886, 0.09338694 ],
 (40,
  0.1,
  0.0001):  [3.10558960e-02, 2.39586979e-01, 1.00000000e+00, 1.02671337e-05,
        4.96817545e-02, 4.96910268e-02, 9.46606040e-02 ],
 (40,
  0.1,
  1e-05):  [0.03054996, 0.22950034, 0.99998363, 0.        , 0.0496805 ,
        0.04968046, 0.09465825 ],
 (40,
  0.1,
  1e-06):  [0.02619026, 0.25960102, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40,
  0.1,
  1e-07):  [0.03518883, 0.42888997, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40, 1.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (40,
  1.0,
  0.1):  [0.        , 0.49998032, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (40,
  1.0,
  0.01):  [0.00315521, 0.48360024, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (40,
  1.0,
  0.001):  [0.0017944 , 0.30542295, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40,
  1.0,
  0.0001):  [0.0205705 , 0.22470726, 0.79784622, 0.20010986, 0.04956061,
        0.22980616, 0.09332412 ],
 (40,
  1.0,
  1e-05):  [3.06929099e-02, 2.30572631e-01, 9.99983634e-01, 8.55594476e-07,
        4.96805374e-02, 4.96812697e-02, 9.46583216e-02 ],
 (40,
  1.0,
  1e-06):  [0.02609911, 0.25527109, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40,
  1.0,
  1e-07):  [0.03454827, 0.42212962, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40, 5.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (40,
  5.0,
  0.1):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (40,
  5.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (40,
  5.0,
  0.001):  [0.00000000e+00, 4.98990399e-01, 3.76419757e-04, 9.94408690e-01,
        3.50716682e-03, 9.45023905e-01, 6.79869938e-04 ],
 (40,
  5.0,
  0.0001):  [5.28433503e-04, 4.28944356e-01, 4.29773166e-02, 5.40285666e-01,
        4.86359343e-03, 5.15578756e-01, 8.73830345e-03 ],
 (40,
  5.0,
  1e-05):  [0.01522028, 0.19702876, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40,
  5.0,
  1e-06):  [0.0258921 , 0.24411143, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40,
  5.0,
  1e-07):  [0.03226304, 0.39442135, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40, 10.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (40, 10.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (40,
  10.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (40,
  10.0,
  0.001):  [0.        , 0.49978353, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (40,
  10.0,
  0.0001):  [8.07254283e-05, 4.76227989e-01, 7.15197539e-03, 8.62684787e-01,
        2.71550010e-03, 8.20180831e-01, 3.93640499e-03 ],
 (40,
  10.0,
  1e-05):  [0.00419532, 0.19836056, 0.47851134, 0.21416214, 0.03085127,
        0.22729535, 0.05796533 ],
 (40,
  10.0,
  1e-06):  [0.0257953 , 0.23828515, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40,
  10.0,
  1e-07):  [0.03034564, 0.36531517, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40, 15.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (40, 15.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (40,
  15.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (40,
  15.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (40,
  15.0,
  0.0001):  [1.87895131e-05, 4.88624550e-01, 1.65297372e-03, 9.42196037e-01,
        1.49273584e-03, 8.95468664e-01, 1.56877364e-03 ],
 (40,
  15.0,
  1e-05):  [0.00191142, 0.26072519, 0.22189126, 0.26675382, 0.01557388,
        0.26452499, 0.02910498 ],
 (40,
  15.0,
  1e-06):  [0.02564126, 0.23527376, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40,
  15.0,
  1e-07):  [0.02908495, 0.34196918, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40, 20.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (40, 20.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (40,
  20.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (40,
  20.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (40,
  20.0,
  0.0001):  [3.07618493e-05, 4.93055226e-01, 4.25517986e-04, 9.68025579e-01,
        6.95242934e-04, 9.19953979e-01, 5.27924141e-04 ],
 (40,
  20.0,
  1e-05):  [0.00120354, 0.30693961, 0.11016006, 0.33818912, 0.00862682,
        0.32686034, 0.01600061 ],
 (40,
  20.0,
  1e-06):  [0.02502686, 0.23190834, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (40,
  20.0,
  1e-07):  [0.02822653, 0.32327927, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (50, 0.1, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (50,
  0.1,
  0.1):  [0.0271909 , 0.48590048, 0.79784622, 0.20011328, 0.04956081,
        0.22980941, 0.09332448 ],
 (50,
  0.1,
  0.01):  [0.02705167, 0.44280391, 0.77025302, 0.264602  , 0.05191373,
        0.28972339, 0.09727152 ],
 (50,
  0.1,
  0.001):  [0.02600951, 0.25973185, 0.79506399, 0.20355534, 0.04959942,
        0.23294224, 0.09337379 ],
 (50,
  0.1,
  0.0001):  [3.10232763e-02, 2.39138133e-01, 9.99885437e-01, 1.45451061e-04,
        4.96827283e-02, 4.98138030e-02, 9.46618583e-02 ],
 (50,
  0.1,
  1e-05):  [3.03430973e-02, 2.25128160e-01, 9.99983634e-01, 1.71118895e-06,
        4.96805778e-02, 4.96820828e-02, 9.46583949e-02 ],
 (50,
  0.1,
  1e-06):  [0.03097365, 0.24755122, 0.99998363, 0.        , 0.0496805 ,
        0.04968046, 0.09465825 ],
 (50,
  0.1,
  1e-07):  [0.03143292, 0.38308724, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (50, 1.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (50,
  1.0,
  0.1):  [0.        , 0.49998032, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (50,
  1.0,
  0.01):  [0.00314784, 0.48360316, 0.00451704, 0.96276367, 0.0063018 ,
        0.91515676, 0.0052622  ],
 (50,
  1.0,
  0.001):  [0.00186669, 0.30096394, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (50,
  1.0,
  0.0001):  [0.01501353, 0.20547475, 0.79784622, 0.20011414, 0.04956087,
        0.22981023, 0.09332457 ],
 (50,
  1.0,
  1e-05):  [3.05641253e-02, 2.27929596e-01, 9.99983634e-01, 1.71118895e-06,
        4.96805778e-02, 4.96820828e-02, 9.46583949e-02 ],
 (50,
  1.0,
  1e-06):  [0.03093581, 0.24542452, 0.99998363, 0.        , 0.0496805 ,
        0.04968046, 0.09465825 ],
 (50,
  1.0,
  1e-07):  [0.03103776, 0.3769929 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (50, 5.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (50, 5.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (50,
  5.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (50,
  5.0,
  0.001):  [0.00000000e+00, 4.98990399e-01, 5.07348368e-04, 9.94009983e-01,
        4.40841866e-03, 9.44651511e-01, 9.09971527e-04 ],
 (50,
  5.0,
  0.0001):  [0.00070954, 0.42275132, 0.04798534, 0.51594058, 0.00515571,
        0.49269197, 0.00931101 ],
 (50,
  5.0,
  1e-05):  [0.0148427 , 0.19587716, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (50,
  5.0,
  1e-06):  [0.02581738, 0.23948769, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (50,
  5.0,
  1e-07):  [0.02964494, 0.3529916 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (50, 10.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (50, 10.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (50,
  10.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (50,
  10.0,
  0.001):  [0.        , 0.49978353, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (50,
  10.0,
  0.0001):  [9.78391522e-05, 4.73865282e-01, 9.00134202e-03, 8.44055928e-01,
        3.00852232e-03, 8.02569356e-01, 4.50974926e-03 ],
 (50,
  10.0,
  1e-05):  [0.00450329, 0.19420589, 0.48443586, 0.21377541, 0.03120645,
        0.22722217, 0.05863569 ],
 (50,
  10.0,
  1e-06):  [0.02575162, 0.23599907, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (50,
  10.0,
  1e-07):  [0.02848656, 0.32911859, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (50, 15.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (50, 15.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (50,
  15.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (50,
  15.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (50,
  15.0,
  0.0001):  [1.67737652e-05, 4.87256455e-01, 2.53674184e-03, 9.36229121e-01,
        2.07527213e-03, 8.89842098e-01, 2.28292008e-03 ],
 (50,
  15.0,
  1e-05):  [0.00205663, 0.2549339 , 0.23761906, 0.26227992, 0.01656002,
        0.26105474, 0.03096224 ],
 (50,
  15.0,
  1e-06):  [0.02555549, 0.23371188, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (50,
  15.0,
  1e-07):  [0.02773272, 0.3108741 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (50, 20.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (50, 20.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (50,
  20.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (50,
  20.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (50,
  20.0,
  0.0001):  [2.66625397e-05, 4.91986161e-01, 6.87375209e-04, 9.66432462e-01,
        1.06938256e-03, 9.18453020e-01, 8.36845094e-04 ],
 (50,
  20.0,
  1e-05):  [0.00126136, 0.30019079, 0.12150175, 0.32703302, 0.00935044,
        0.31682197, 0.01736456 ],
 (50,
  20.0,
  1e-06):  [0.0249369 , 0.23079239, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (50,
  20.0,
  1e-07):  [0.02722584, 0.2968348 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (100, 0.1, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (100, 0.1, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (100,
  0.1,
  0.01):  [0.02559855, 0.42155822, 0.77383719, 0.2670182 , 0.05230559,
        0.29219761, 0.09798793 ],
 (100,
  0.1,
  0.001):  [3.08740132e-02, 2.87081522e-01, 1.00000000e+00, 1.45451061e-04,
        4.96881379e-02, 4.98194946e-02, 9.46721908e-02 ],
 (100,
  0.1,
  0.0001):  [3.10413059e-02, 2.41727834e-01, 9.99983634e-01, 5.64692354e-05,
        4.96831632e-02, 4.97341204e-02, 9.46630878e-02 ],
 (100,
  0.1,
  1e-05):  [0.02958642, 0.21085997, 0.99998363, 0.        , 0.0496805 ,
        0.04968046, 0.09465825 ],
 (100,
  0.1,
  1e-06):  [0.03074248, 0.233997  , 0.99998363, 0.        , 0.0496805 ,
        0.04968046, 0.09465825 ],
 (100,
  0.1,
  1e-07):  [0.0272604 , 0.29855393, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (100, 1.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (100, 1.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (100,
  1.0,
  0.01):  [0.00316551, 0.48360237, 0.00451704, 0.96268581, 0.00628873,
        0.91508277, 0.00525764 ],
 (100,
  1.0,
  0.001):  [0.00157906, 0.32146198, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (100,
  1.0,
  0.0001):  [2.53744622e-02, 2.50121028e-01, 1.00000000e+00, 5.98916133e-06,
        4.96815525e-02, 4.96869613e-02, 9.46602374e-02 ],
 (100,
  1.0,
  1e-05):  [0.03041406, 0.22582404, 0.99998363, 0.        , 0.0496805 ,
        0.04968046, 0.09465825 ],
 (100,
  1.0,
  1e-06):  [0.03076912, 0.23428047, 0.99998363, 0.        , 0.0496805 ,
        0.04968046, 0.09465825 ],
 (100,
  1.0,
  1e-07):  [0.02715864, 0.2955236 , 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (100, 5.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (100, 5.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (100,
  5.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (100,
  5.0,
  0.001):  [0.00000000e+00, 4.98990399e-01, 5.72812674e-04, 9.93827741e-01,
        4.82825217e-03, 9.44481575e-01, 1.02412547e-03 ],
 (100,
  5.0,
  0.0001):  [0.00062732, 0.42536111, 0.04598867, 0.5236281 , 0.0050216 ,
        0.49989836, 0.00905451 ],
 (100,
  5.0,
  1e-05):  [0.01113322, 0.18809404, 0.79778076, 0.20011328, 0.04955695,
        0.22980616, 0.09331718 ],
 (100,
  5.0,
  1e-06):  [0.02572596, 0.23353953, 0.99996727, 0.        , 0.04967972,
        0.04967964, 0.09465677 ],
 (100,
  5.0,
  1e-07):  [0.02680262, 0.28409549, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (100, 10.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (100, 10.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (100,
  10.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (100,
  10.0,
  0.001):  [0.        , 0.49978353, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (100,
  10.0,
  0.0001):  [8.98619656e-05, 4.73908430e-01, 9.24683316e-03, 8.42156509e-01,
        3.05324536e-03, 8.00776499e-01, 4.59067808e-03 ],
 (100,
  10.0,
  1e-05):  [0.00354056, 0.20738423, 0.42183562, 0.22156817, 0.02754949,
        0.23151771, 0.05172115 ],
 (100,
  10.0,
  1e-06):  [0.02547029, 0.23197319, 0.79776439, 0.20010729, 0.04955563,
        0.22979966, 0.09331473 ],
 (100,
  10.0,
  1e-07):  [0.02650489, 0.27331264, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (100, 15.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (100, 15.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (100,
  15.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (100,
  15.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (100,
  15.0,
  0.0001):  [3.32649681e-05, 4.87156698e-01, 2.78223299e-03, 9.35490743e-01,
        2.24965924e-03, 8.89152600e-01, 2.48776240e-03 ],
 (100,
  15.0,
  1e-05):  [0.00178838, 0.26369836, 0.21225164, 0.27226471, 0.01501859,
        0.26928318, 0.02805224 ],
 (100,
  15.0,
  1e-06):  [0.02477045, 0.22926592, 0.79769893, 0.20010815, 0.04955182,
        0.22979722, 0.09330752 ],
 (100,
  15.0,
  1e-07):  [0.02630736, 0.26528413, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385 ],
 (100, 20.0, 1.0):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (100, 20.0, 0.1):  [ 0.0,  0.0,  1.,  1.,  1.,  1.,  1. ],
 (100,
  20.0,
  0.01):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (100,
  20.0,
  0.001):  [0.        , 0.49998032, 1.        , 1.        , 1.        ,
        1.        , 1.         ],
 (100,
  20.0,
  0.0001):  [2.63784017e-05, 4.91899746e-01, 8.01937743e-04, 9.66273321e-01,
        1.24151211e-03, 9.18307477e-01, 9.74445660e-04 ],
 (100,
  20.0,
  1e-05):  [0.00118262, 0.30563789, 0.11241858, 0.33640435, 0.00877867,
        0.32527645, 0.01628561 ],
 (100,
  20.0,
  1e-06):  [0.02339639, 0.22453213, 0.79720795, 0.20010815, 0.04952283,
        0.22977282, 0.09325276 ],
 (100,
  20.0,
  1e-07):  [0.02617322, 0.25928689, 0.79784622, 0.20010729, 0.04956046,
        0.22980372, 0.09332385]})

In [ ]:
results_uniform_dist

1                                                        \
          0.1                                                        
  1.000000e+00 1.000000e-01 1.000000e-02 1.000000e-03 1.000000e-04   
0     0.028270     0.028270     0.033399     0.093449     0.246054   
1     0.510603     0.510603     0.555751     0.690071     0.727307   
2     0.793575     0.793575     0.787781     0.676230     0.376633   
3     0.235467     0.235467     0.429576     0.816292     0.956600   
4     0.051471     0.051471     0.067337     0.161382     0.312091   
5     0.263195     0.263195     0.447372     0.809333     0.927786   
6     0.096673     0.096673     0.124070     0.260577     0.341338   

                                                                    ...  \
                                                 1.0                ...   
  1.000000e-05 1.000000e-06 1.000000e-07 1.000000e+00 1.000000e-01  ...   
0     0.067775     0.046274     0.044317     0.000000     0.003126  ...   
1     0.583569     0.512734     0.502689     0.499980     0.489752  ...   
2     0.454371     0.740581     0.784590     0.003928     0.004304  ...   
3     0.683831     0.281434     0.221264     0.967310     0.964428  ...   
4     0.069880     0.051126     0.050036     0.006242     0.006286  ...   
5     0.672431     0.304245     0.249250     0.919448     0.916728  ...   
6     0.121131     0.095648     0.094073     0.004822     0.005110  ...   

           100                                                      \
          15.0                      20.0                             
  1.000000e-06 1.000000e-07 1.000000e+00 1.000000e-01 1.000000e-02   
0     0.024770     0.026307          0.0          0.0      0.00000   
1     0.229266     0.265284          0.0          0.0      0.49998   
2     0.797699     0.797846          1.0          1.0      1.00000   
3     0.200108     0.200107          1.0          1.0      1.00000   
4     0.049552     0.049560          1.0          1.0      1.00000   
5     0.229797     0.229804          1.0          1.0      1.00000   
6     0.093308     0.093324          1.0          1.0      1.00000   

                                                                    
                                                                    
  1.000000e-03 1.000000e-04 1.000000e-05 1.000000e-06 1.000000e-07  
0      0.00000     0.000026     0.001183     0.023396     0.026173  
1      0.49998     0.491900     0.305638     0.224532     0.259287  
2      1.00000     0.000802     0.112419     0.797208     0.797846  
3      1.00000     0.966273     0.336404     0.200108     0.200107  
4      1.00000     0.001242     0.008779     0.049523     0.049560  
5      1.00000     0.918307     0.325276     0.229773     0.229804  
6      1.00000     0.000974     0.016286     0.093253     0.093324  

[7 rows x 528 columns]

In [ ]:
results_normal_dist.to_csv('results_normal_dist.csv',index=False)

In [ ]:
results_uniform_dist.to_csv('results_uniform_dist.csv',index=False)